In [20]:

from config.database import collection, collection_weather, collection_searchTrack
from routes.root.root_class import Root_class, Fetching_Mechanism, Source_links_and_api, Root_source_links
from bson import ObjectId
import requests
import datetime as dt
import json
import pickle
from pymongo import UpdateOne


# get synopsis of the collection:
# x = [i for i in collection_searchTrack.find({})]

# Delete all documents from the collection
# x = collection_searchTrack.delete_many({})

# [i for i in collection_searchTrack.find_one({})]



In [ ]:
for data in x:

    update_query = {
        "$setOnInsert": {"email": data['email']},  # Only set email on document creation
        "$inc": {f"searches.{data['searchTerm']}": 1},  # Increment count for this search term
        "$set": {"lastUpdated": data['timestamp']}  # Update timestamp
    }
    # This single operation will:
    # 1. Create document if email doesn't exist
    # 2. Create searchTerm with count 1 if it doesn't exist
    # 3. Increment count if searchTerm exists
    result = collection_searchTrack.update_one(
        {"email": data['email']},
        update_query,
        upsert=True
    )

    print({"status": "success", "matched_count": result.matched_count, "modified_count": result.modified_count})

{'status': 'success', 'matched_count': 0, 'modified_count': 0}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 0, 'modified_count': 0}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_cou

In [ ]:
# searches
y = [i for i in collection_searchTrack.find({})]
y

[{'_id': ObjectId('67ab9648f07a4d921a95d37e'),
  'email': 'ujasvaghani@gmail.com',
  'lastUpdated': datetime.datetime(2025, 2, 11, 7, 27, 5, 798000),
  'searches': {'newa': 3,
   'jfk': 1,
   '4433': 1,
   '443': 1,
   'n': 2,
   'newar': 3,
   'newark': 2,
   'me': 1,
   'new': 1,
   'Newark ': 1,
   'Newark Liberty International Airport (EWR)': 1,
   'Newark Liberty International Airport (EWR) ': 1}},
 {'_id': ObjectId('67ab9648f07a4d921a95d37f'),
  'email': 'ismailsakhani879@gmail.com',
  'lastUpdated': datetime.datetime(2025, 2, 10, 14, 36, 0, 784000),
  'searches': {'new york': 1,
   'New York Stewart International Airport (SWF)': 1,
   '414': 7}}]

In [ ]:
import requests

# extrac the data from the response - all flight numbers
db = requests.get("http://3.15.228.76:8000/db")

response = db.json()
result = response['db'] 

db = [i for i in result]
print("total flights", len(db))


total flights 35205


In [25]:
import pickle
with open('flight_data.pkl', 'rb') as f:
    xx = pickle.load(f)

# with open('flight_data.pkl', 'wb') as f:
#     pickle.dump(x, f)

{'flight_number': 'SKW5332',
 'route': 'KMRY.MRY5.MRY..BSR..LIBBO..TOKIO.HUULL2.KLAX/1725',
 'timestamp': '2025-02-20T16:36:07.053Z',
 'registration': 'N606UX',
 'arrivalPoint': 'KLAX',
 'departurePoint': 'KMRY',
 'runwayPositionAndTime': None}

In [ ]:
gjs_flight_numbers = [i for i in x if i[:3] == 'GJS']

def gjs_returns(flightNumber):
    return requests.get(f"http://3.15.228.76:8000/flightData/{flightNumber}")


gjs_flight_data = [gjs_returns(i).json() for i in gjs_flight_numbers]

In [16]:
registration = []
for i in a:
    registration.append(i['flightData']['registration'])

print(len(registration),len(set(registration)))

169 38


In [17]:
set(registration)

{'N503GJ',
 'N504GJ',
 'N506GJ',
 'N508GJ',
 'N511GJ',
 'N520GJ',
 'N521GJ',
 'N522GJ',
 'N523GJ',
 'N524GJ',
 'N526GJ',
 'N534GJ',
 'N535GJ',
 'N536GJ',
 'N537GJ',
 'N538GJ',
 'N539GJ',
 'N540GJ',
 'N541GJ',
 'N543GJ',
 'N544GJ',
 'N546GJ',
 'N548GJ',
 'N549GJ',
 'N551GJ',
 'N552GJ',
 'N554GJ',
 'N556GJ',
 'N557GJ',
 'N559GJ',
 'N561GJ',
 'N563GJ',
 'N564GJ',
 'N566GJ',
 'N569GJ',
 'N578GJ',
 'N579GJ',
 None}

In [1]:

for i in a:
    data = i['flightData']
    if data['runwayPositionAndTime'] is not None:
        # print(i['flightData']['flight_number'], i['flightData']['runwayPositionAndTime'], data['timestamp'], data['time'])
        print(data['flight_number'], data['registration'])

NameError: name 'a' is not defined

In [1]:
flights= ["AAL649","JBU1302","N811WE","ASA307","AAL1786","ASA488","ASH6305","LXJ398","N803SD","DAL990","UAL1320","VAR520","N738RL","UAL2393","CKS287","AAL1798","DAL2140","DAL31","JBU2162","UAL1652","FLE2401","ENY3532","SKW5260","NKS307","IRO8393","ENY3600","N517GL","AAY1397","SJJ334","DAL1702","PDT5828","LXJ593","CPA885","UAL1441","JSX400","CAL5142","AAY1127","UAL313","QXE2027","N479MM","RDN095","DLH440","EDV5515","AAL632","LXJ487","SWA3171","MXY242","EJM44","N46SM","NKS396","SKW5819","DAL3011","ENY3901","CXK242","N118V","N253JS","SUB323","AAL2644","CMP189","ACA1240","SWA2320","N4520L","MXY184","X703SR","EDV4960","NKS2002","SWA3478","N797PG","FDX30","N91447","AAL34","UAL1189","FFT2350","SWA1289","ASH6211","FDX1073","AAL2468","DAL2458","SKW5318","JBU1133","EJA706","SWA2587","AAL1168","N600BM","GJS4394","SWA4588","EJA941","SKW5599","BXR8604","DWI401","N46WD","UAL2344","N37SD","DAL618","JTZ374","DAL1231","EDV4868","LXJ443","JBU932","NKS3891","LAN532","CXK123","AAL1034","N86AT","PDT5851","LXJ388","AWI6180","TSC844","N133PE","N79VH","JBU829","DAL450","DAL377","ENY3618","DAL3069","AMX622","AAL2551","SKW5386","EDV4998","RPA5695","UAL2326","PLZ4507","N35629","JZA798","JBU670","N2201M","ENY4168","UPS1346","DAL3144","JZA568","NKS147","VJA535","ASA399","AAL1861","N41395","RPA5830","N9255C","VXP322","N950TD","AAL2761","N9835G","N953AJ","GPD103B","N608AB","AAL1514","SWA1753","JBU466","DAL2462","DAL1147","AAL1647","AAL35","N816WG","N913MK","ENY4191","AAL1487","UCA4369","UAL354","EJA674","AAL1610","AZY322","EDV5091","UAL2459","CVR1499","UAL1391","BYA297","SWA2632","ASP548","LXJ351","AAL2010","N595P","SWA3674","AAR285","UAL2455","JIA5540","DAL2013","N941CD","UPS5661","JBU126","SWA2542","RPA4473","CPA879","JIA5322","BWA528","JIA5227","AAL1952","FDX1451","ARE4406","N300KZ","JZA985","AAL76","N328PF","ENY3427","FDX1639","NKS2988","CGJKX","JBU1230","TSC106","N263SN","N52398","XAVGE","NKS881","EJA6532","SKW3627","EDV5384","ASA896","MXY255","AAL631","N7155P","SWA1513","GXA6111","SWA2034","BTQ355","EJA391","SKW5324","AAL79","PDT5888","DAL2441","DAL2972","JBU1902","ABX403","ASA459","AJT880","LFA457","ENY3905","LRC655","AAL2028","VJA58","ACA91","QXE2045","ABX3159","EJA597","UAL1893","N100PM","DAL291","SKW5416","DAL1087","DAL2597","N25SF","NCR313","SKW3310","CJT952","EDV4832","GGT700","RPA4667","CPT8822","VJA350","ABX3151","SWA726","SVL22","DAL8844","N3844Q","ABX845","GJS4537","N125DJ","AAL2353","DAL894","AWI6107","FFT1450","SWA3557","AMF1532","WUP904","N70000","UAL1633","ETH519","UAL2266","N6980B","N8238W","N250AL","SWA3952","N710CA","JIA5096","N694JL","DAL2079","JIA5345","GPD153B","JTZ317","AAL1871","UAL2255","AIC144","JBU323","N881AB","SWA1199","EJA937","UAL450","DAL3125","SKW5289","DAL1382","VIV740","PAL113","UPS2331","EDV5311","DAL1542","EDV5425","AAL762","AAL782","SWA1306","VOS4327","EJA629","FFT1994","ENY4170","N19GR","CJT1925","JZA689","RPA4367","JSX171","YEL2","SWA215","SWA2443","NKS1034","FLE1101","SWA1957","GTI531","SWA8502","EDV5423","GXA6130","SWA2864","SWA3514","UAL2082","ACA1088","ANA11","AAL602","SKW4994","N638NG","QXE2065","AMF2852","N755TV","N885M","N59VK","UAL706","EJM326","UAL2054","DAL2273","N9385L","UPS1476","VXP473","N2AC","SKW5033","VAR518","STB940","N5337F","AAL1205","UCA4310","ASP812","ASA799","LXJ547","LXJ573","SWA147","DAL1746","UCA4887","PDT5779","AAL2723","SWA495","DAL700","UAL1564","ASA129","RPA4565","AAL2920","SWA4877","N795SC","N38331","QXE2221","SWA333","N121AP","EDV5073","DAL1119","NKS327","UAL1643","AAL1186","SWA4849","ASA1070","TSC651","TSC607","SKW5392","RPA4797","UPS5126","UAL2475","DAL430","SKW5831","UAL399","DAL299","N794SB","FDX889","ENY3687","JBU2924","DAL1624","ASA616","WUP985","ENY3555","EPI241","N2456V","N786QA","SKW4038","N30HV","N645BD","SWA3121","AAL1924","UPS1296","IBE2629","ATN3333","N97CP","AAL1784","RPA3677","SWA1841","JZA780","UPS5773","UAL2856","AAL1231","JBU598","EJM618","N459VP","N404DC","JSX905","WJA711","DAL1411","ENY3400","NKS1350","N798RS","SWG121","UAL402","WIA303","AAY118","ABX520","UPS9749","N895CA","XBFXT","ETD5SJ","EJA860","FFT3381","N199TT","NDU13","SWA1497","WJA1510","N9394X","SKW6208","SKW5608","JBU1190","UAL2435","DAL792","N606MC","ENY3726","AAL1152","DAL2723","CPT8573","AAL1431","N851QS","RPA3656","N58HD","N57GP","CFRQT","SWA2803","SCX3013","SWA1352","UAL1085","DAL690","N529DB","SKW3819","N3155S","JUS603","AAL2986","N451SP","N812LP","N443PR","N758RF","UAL1917","N699QS","NKS341","SWA4047","CGPMM","ASA324","SWA2640","DAL67","ENY3535","NKS812","ENY3934","CFS8971","DAL1105","ENY3354","DAL744","CXK164","N18KW","LXJ333","EJA645","YEL82","DAL762","JBU1769","DAL1782","GJS4417","DAL1751","N12JS","N15RK","EJA792","AAL1651","DAL2052","MXY263","EJA638","UAL1751","EJA432","AAL1122","SWA2730","MIIII","UAL1435","AAL1678","SKW3878","DAL1338","AAL1845","UPS671","ENY3898","UAL1513","AAL2578","NKS527","ACA1043","AAL1624","BHS201","N43LJ","SKW3939","UAE213","UPS802","UAL1561","AAL1198","AWI6050","CSQ725","JIA5409","DAL1215","DAL788","UAL1979","JBU316","AAL3228","RPA4601","SWA4348","NKS253","BL6ER","SKW5058","AAL794","SWA1928","N600JT","LBQ790","DAL2790","N551LJ","RPA4507","SWA3018","JBU8398","SKW5931","EJA626","LBQ865","UAL1768","SWA1081","JBU2465","AWI6090","SIL0125","FFL588","LN113MT","CXK127","N30644","N7848T","JIA5101","RNI218","JZA691","UCA4330","N500WF","DAL9961","EJA459","UAL1842","N478BK","DAL617","FFT4754","N199BP","AAL3216","N2QW","N664FL","UAL2855","SWA4964","N522SK","ASH6161","N190PA","EJA528","T7JIM","UAL792","ASH6261","DAL2055","DAL1615","AAL3013","RPA4553","DAL1639","ENY4053","JZA931","TAF700","DAL2964","DAL9963","N722AP","NKS176","PDT5704","N9258P","N545SB","QXE2287","UCA4815","SKW5020","UAL997","FDX630","SKW4769","N350HA","CXK595","UPS340","N4386N","SWA1680","VXP188","AAL92G","N362BC","ZN212MF","AAL2475","N731AA","N910J","EJA948","BHS212","UAL1950","N140WH","SWA1171","UAL279","CLX14P","FTO115","ASH6025","DAL639","UAL383","MTN8311","AAL569","DAL353","UAL3896","SWA317","N39DJ","N815TG","NIT289","UPS2310","N459BE","UAL801","RPA3607","SWA1989","JBU557","UPS5133","JTL611","N27TJ","AIP1542","NKS520","POE669","N351TS","SWA548","ERU821","N741TS","SWA3947","SHH851","DAL1293","SWA4253","RPA3502","NKS879","GXA1914","SWA1056","N994MB","ARG1325","DAL3085","LTA515","KES215W","UAL1041","NKS1310","AAL2186","DAL1760","NKS833","N194TH","EDV5165","N33KF","VOI7843","KPO518","SWA677","SKW4103","AAL1742","UAL2637","N24MN","JRE736","GXA3140","AAL946","DAL655","JBU144","N2818T","N317XT","N71NX","LXJ409","QFA7","AAL2129","UAL2105","N81QC","FDX1365","CPT8574","ASA706","UAL916","BWA266","AAL1309","SWA473","N3330M","AVA262","N789HU","N324FL","N60KC","N855SC","SD4","EJA776","N459RH","AAL2128","KFB60","XLS500","N98CL","SWA1102","FDY393","N335RJ","DAL2375","JBU225","AAL1200","FLE651","SKW3922","SCX3069","OAE3051","FFT2284","JIA5624","DAL2783","SIL97","JIA5022","JBU2586","NDU42T","N1137T","DAL2656","N911MF","UPS945","SWA837","AAL1196","JBU106","AAY2980","AMF1932","SWA2719","FFT2147","DAL1598","AAL1326","AAL813","SWA563","SKQ37","DCM4102","UAL2070","N751LB","N75866","PDT5928","DAL1762","CSI532","N733YD","ENY3755","SKW4928","JIA5549","UPS9862","NKS580","LXJ322","N464PG","DAL1013","NUS531","SDE757","N481SC","NKS985","JZA801","SKW5269","SKW4158","AAY2839","ENY3824","N944TH","N777PB","AAY703","JBU2064","AAL1177","ENY3757","ASA877","SKW3743","JBU987","SKW6262","EJA621","UAL2119","N818RC","N311NB","FDY328","AAL1054","JIA5348","DAL775","SWA121","JIA5132","HRT445","N58739","BBQ609","UAL608","N194CS","AAL1556","SKW5371","AVA200","TSC484","SWA1659","FDX58","SWA2489","AAL1371","N3687W","DAL2638","AAL1954","N188WR","SKW4747","DAL2376","UAL1175","FFT4026","FFL748","N850ED","N126NG","SWA3120","LXJ607","FDX1413","AAL1260","SKW3750","TZP24","JBU2592","AAL1661","N3364Z","UAL2166","RPA4453","CFS8718","N1072Q","AAY2687","N67901","AAY41","UAL2412","N455RS","SWA3566","SWA3584","SKW5573","ZN8498C","SWA1582","N55TJ","CXA850","AAL2729","VIV610","FFT3412","N9239N","UAL1394","JZA772","TWY67","DAL1558","SKW4923","SKW5100","UAL419","N720ML","N728MH","SWA1707","AAL2428","VJA357","FDX886","AAL1294","VTE3052","AAL2061","LTA656","N2102B","N563M","JIA5051","JSX123","N75RC","N16RP","SWA364","AAL1214","AAL1702","AAL2588","AAL2456","RPA4456","N211BB","KAL023","DAL662","UCA4821","CPT8591","UAL2729","AAL2253","N624DP","SKW5332","DAL8923","VIV587","NKS1639","N75CC","SWA1745","SWA451","UPS9715","JTZ287","IBE0364","UAL2163","AAL2991","JIA5309","FFT3002","SWA655","WWI84","DAL1608","NKS290","N64WG","CKS9715","PWA8TA","ILU45A","VOI1800","AAL2242","MTN8521","NKS1676","FFT3566","SWA2390","UAL770","TOM065","SWA588","BMA223","DAL1066","AAL439","N700PX","DAL1405","DAL2775","JIA9958","AMF1978","PIT8","ENY3363","N584LQ","N92844","SWA265","DAL1226","N123","MXY503","ENY4030","SWA1339","RPA5623","EJA676","FFT4998","EJA900","ENY4035","DAL649","N788RR","DAL947","ACA1059","DAL17","CMP862","DAL2534","TCF658","N680CM","AAL2830","SWA4104","EDV4981","AIP4570","SWA3208","SWA3930","RPA3551","VOI1820","AWI2037","N228PG","SWA513","UAL1795","GPD649B","WMN532","N414KS","N96WF","LXJ502","RPA5637","N8949C","SKW5023","FFT3440","DAL2351","JBU1485","N705VS","AAL2618","EJA760","DAL972","AAL2833","N990SP","ASP668","CXK160","VJA508","UPS871","N1NT","SWA1637","UAL1369","SKW5605","N3NC","ENY3985","NKS502","JAS82","EDV5052","DAL729","FLE1775","UAL567","N565HT","SWA1263","DAL1944","UAL366","ATN3381","AAL2696","ATN3329","VOS4338","AAL2682","AMF1908","LN335RC","N350AL","SWA34","AVA245","RPA4552","SWA1345","FDX1506","UAL2321","NKS884","TAM8195","UAL3739","UPS5844","KAL006","NKS1640","DAL1913","UAL2129","N79BF","AAL2836","AAL717","WMA5263","EJA336","SKW4144","N575JJ","FDX1814","RPA3300","N526CW","WJA2454","N930CT","VJA121","N711TQ","N92CJ","BVN8281","SKW3786","PDT5837","UAL1289","HAL1641","SWA5999","N70LJ","CSB7517","AAL2561","OFT9","GPD202","EJA426","UPS582","AAL1621","UAL927","SWA1527","LXJ341","AAL2065","SWR38K","HAL17","JCM155","N780TW","N345TR","NKS713","FDX1540","N831PB","ENY4065","VIV501","GTI8228","FDX1491","CGGMK","DAL2502","PDT5870","UAL984","N531RC","AIP1599","DAL85","N508RH","JBU2627","N245GR","UAL1538","N72849","AAL1409","UAL2194","N85SL","SKW6236","VIV730","N810RD","AAL2362","DAL1761","WJA1314","UCA4336","UAL2126","AAL1279","N120QS","DAL3162","SWA3015","SKW6487","SKW4314","SWA313","UAL219","OKC393","AMX449","AAY3783","N553TS","DAL1193","SWA1800","RPA3405","SWA292","CSI504","TSC186","VIV623","JIA5560","N4185J","SWA2112","AAL1616","SWA626","AAL953","N90KM","FDX1475","RPA3633","FDX1578","UAL949","ASH6223","NKS1473","FFT1216","AAL102","FFT4157","LXJ323","GTI8638","N746CX","UAL690","CNS303","N13941","FFT1334","SKW5767","SWA4089","N1895C","NJE922K","N4951N","DAL516","DAL1062","N601EA","DAL1367","DAL3108","AAL2329","DAL1360","EDV4871","N6222P","UAL2259","LN854AL","FDY802","UAL3915","AAL712","ACA1103","UAL1416","AAL2398","EJA740","ASA359","N4197X","RPA5618","RPA5799","N70AD","AIP1828","AAL1873","SGX907","N509SF","AAL2375","UAL8129","CFIRK","SWA1340","UAL2135","GTI3621","SWA2075","GJS4452","UAL1859","AAL2339","RPA5601","UAL221","CXK636","SWA658","JBU745","THY3","RPA4595","FDY126","N167KM","SWA932","AAL547","N363VP","LXJ969","UAL1164","SWA653","CXK693","N5555C","AMF1558","FFT685","N2562M","AAL2848","ACA35","SWA8500","AAL1240","SWA2180","FFT3311","AAL1895","FDX1479","VOI1832","CJT920","AAY549","SWA365","GTI8995","ASA1400","SKW3318","UAL2647","N5331D","LLN206","SWA301","SWA1059","UAL1529","LN463AK","SWA102","SKW3782","DAL121","SYB088","AAL2683","SWA1798","SWG4307","UPS903","AAL947","EJA925","POE506","BHS281","POE502","UCA4370","FDX1431","BHS203","UPS872","EJA429","AAL2944","N623SR","SKW4132","DAL2553","ASH6280","RPA5795","AAL2809","AIP1844","PHM100","SCX601","SKW5049","VOS4313","AAL69","N905PC","AAL2408","NKS920","N913SP","AAL3077","AMX1740","DAL810","DAL3154","N685LM","BOE181","JBU688","UPS706","N119AF","UPS761","DAL2227","WMA2404","RPA3645","N2124Y","ENY3590","PDT5732","VJA550","SWA246","N115BA","KAP556","ANA179","N612SL","XEN89","N75801","N125RF","AAL871","GJS4168","SWA290","AAL82","RPA3400","CLX35W","FDX1750","GGW33","DAL1279","UPS962","AIC184","AAL2333","SKW3458","ASP642","N687SB","CMP393","PCM8709","N379EC","N246BJ","DAL861","N499AT","SWA927","CGFCB","UAL2355","BEZ407","UAL2066","AAL1169","DAL1400","AAL1352","DAL2218","N456ER","UAL3033","PDT5865","MSR986","SWG7469","N773DV","N716BS","OKC471","SWA1470","UAL135","SKW3744","SWA1103","SWG6415","AAL1720","FDX1489","DAL945","RPA5875","AAL2755","N12AN","WEN3750","N260HS","NKS308","CGSFU","SWA1566","N1897S","LXJ390","SWA766","ANA118","AAL2007","JBU1791","UPS5787","N817TT","JIA5559","DAL2976","SKW3683","MXY203","AAL2134","ACA794","RPA5759","VJA362","N696NG","FDX1234","MLN280","UAL2133","DAL669","N917AP","UAL323","AAY607","SKW4809","NKS539","EJA369","UAL664","DAL773","JBU39","CYO500","DAL925","UAL2388","SWA2644","NKS1679","LPE2468","UPS1111","SWA1318","DAL479","AAL1118","DAL1204","N347GA","SKW3649","JBU1734","UPS806","EJA596","SKW3979","SWA3907","ZN6498C","DAL2219","LXJ367","N514DH","UAL1209","N319JM","RPA4664","JBU2879","AAY2659","LYM4235","UAL1155","SCX350","RPA4591","SWA4543","DAL1524","N559PB","GXA6166","NKS575","UPS2066","UPS1376","ASA1020","AAY502","RAM204F","RPA4776","UPS1306","N124C","JBU546","UAL1568","UAL2454","DAL755","DAL2852","NKS320","GTI3615","AAY1877","SWA3146","DAL923","SKW5325","JBU1803","UAL2030","N905GW","LTA670","AAL993","RPA5720","AAL2817","AAL1603","DHK362","BHS383","N400AX","SWA4503","SWA1050","SKW157E","N641KC","JSX461","UAL2323","KLM627","UAL240","N510FS","JZA821","LBQ825","N9690U","ASH6196","SWA1564","UAL1530","SWA1451","LTA660","N56MN","N162EC","APZ102","UAL1689","N242MD","JIA5224","N188HA","N351CC","ASA810","UAL681","FDX14","SKW4083","UAL2257","N652CR","UAL231","N940LT","JIA5229","XACDM","BBQ2202","N669GL","DAL1875","N96LA","UAL1499","UPS304","LPE2482","JIA5041","CXK233","FFT91","RPA5710","N214TD","N620WB","UAL728","SWA467","FDX6014","N138BC","JIA5410","AAL1281","SWA4267","SWA2439","UAL1759","FFT1100","UAL2261","SKW6285","KAP550","PHM018","DAL11","SWA228","GTI8609","ASA1280","N987ST","NKS1025","JIA5145","DAL2250","MXY602","UCA4970","UPS2111","SWA113","SWA608","NKS1683","RPA5651","N311FP","SKW5293","UAL1304","PCM8204","N206PK","AAL3141","UPS892","GTI8030","PDT5694","N1822F","NKS2964","AAL2509","BVN8560","DAL2025","MXY817","AAL811","SKW3697","EDV5236","N6SM","SWA591","N636CH","EJA442","UAL1325","DAL1067","N156RK","EJA110","FRG9508","UCA4226","GTX323","LN680TW","N253PC","UPS598","UAL1186","SKW5272","EJA848","LXJ541","N474MJ","AAL2543","ZKP319B","ACA1174","N823HL","N210MY","N412CR","TSC1831","FDY66","N500M","N508GE","UAL740","GTI3613","UAL2484","N33BJ","N930MC","ABX901","JBU583","ASA1292","EJA603","DAL2280","AAL2974","MDIVE","ASA27","RPA3639","N122KA","DAL2346","UAL1552","N55QC","ASH6045","AAY4923","ASA850","EDV5069","BMJ52","N233SV","RPA5755","AAL605","N644AT","NKS2973","FTN3","SWA1290","UAL1992","SKW3702","AMX002","N403SP","N456V","UAL455","LN821TN","EDV4951","SWA3704","ETH508","NKS548","ASH6201","AAY575","RAX698","UAL2426","UAL2122","DAL2535","N737EG","JCM26","SKW3805","JIA5395","SWA2512","UAL2354","EJA627","DAL2078","N613WM","LXJ531","NKS837","FFT3232","SKW6232","N66111","JBU207","SKW5764","N65841","UAL2190","AMX653","AAL2119","LN968SR","ASA124","JBU1","AAL1802","GTI076","JIA5242","LN481HC","N437P","CFS8692","AAL2709","ENY3865","AAL2673","N737PY","JZA848","XSR487","FNX989","IBE0303","AAL2868","NKS907","SWA369","N5273K","JBU458","DAL1626","N203CK","SCX3071","ENY4083","LXJ652","XSR722","JIA5503","N1397Z","UAL641","KFS21","AEA089","MTN8464","FDX121","NKS868","ASA321","DAL1100","SWA109","EDV5086","AAL2537","UCA4823","SWA1628","SWA2745","FEX850","RPA5764","QXE2071","ACA7243","FFT2387","UAL2045","N15KJ","ASA1156","AAL1319","N984JP","AAL3283","BMA452","SKW5089","EJA847","JBU540","X8575U","DAL742","N8939N","JBU2839","SD20","EJA408","UPS496","SIA32","SKW5345","NCR314","AAY3231","N935S","N975MT","JBU1883","UAL1198","FDY423","UPS1366","N44SW","N707SQ","REE600","PDT5989","JRE751","SWA2049","SWA1524","UAL558","SNC384","N528MX","ERH553","TTT683","ASA287","AMX640","CNS119","ENY3927","AWI6114","N90HK","ASA9803","N709VM","UAL1956","N476BE","N7273M","TTTCERT","AAL2688","FFT1602","AAL3007","N628GB","AAL807","AAL1731","SWA1295","AAL492","MTU94","UAL1687","N968AB","UAL1610","UAL662","CTL13","UPS379","DAL53","N8273A","UPS219","ASA1365","JBU1107","EJA745","AAL3050","RPA4576","UPS492","NOAA42","UCA4952","N364EA","N805PH","AAL2279","PDT5800","IBE0312","DAL834","UPS5945","NKS361","N303LM","N46RT","EJA825","ENY3565","ATN3325","N220KA","HPN8805","SWA1363","DAL1809","AAY2818","SKW3654","SKW4172","PJC65","FFT2894","NKS927","GJS4510","DAL8801","PDT5924","UAL1089","JZA877","N194CD","AAL3122","N4868E","AAY9201","JBU769","N952PC","N347CP","N815QS","N5809S","N6502H","DAL940","ACA940","AAL2411","DAL522","N626N","WUP510","WUP992","SKW5045","UAL622","ASH6088","SKW5736","VTM680","KFS20","FDX9","FFT3371","DAL397","N33UM","FFT2548","N283JD","UCA4294","LTA674","UAL1247","UAL1631","VTM691","CGAPC","N1KA","AAL281","SKW3725","N889AH","UAL2053","AAL175P","QXE2090","VTE3755","LYM232","AVA230","DAL629","BYA229","N498GF","VJA320","UCA4301","JIA5177","UPS712","JBU1736","WJA2198","FDX1495","AMF1944","JBU1730","EDV4937","N58GL","AAL2534","DAL1747","ENY3519","SCX761","RPA3695","SWA2297","N384BC","LKF16","N315QS","N598TP","FDY436","AAL2446","AAL2874","SWA2359","SWA447","SWA1309","SWA2420","AAL2999","ASA53","SKW5126","UCA4888","FDX1429","DHK317","UPS1018","N996SR","N850WM","ILU45G","AMX025","ASA290","SWA1983","SKW4724","FFT2014","ATN3357","N10VZ","CXK261","Z6ZG","MVP438","LYM5110","FDX1412","WJA1118","N24KK","ERU314","LXJ589","EJA202","ACA601","SWA2047","SWA1635","N648AJ","N660MW","SWA149","FDX1157","RPA3555","DAL2975","SKW3769","CAL5382","QXE2005","SKW5791","LBQ651","WMA2112","ASA68","JBU2270","DAL157","VXP858","N22XT","UAL2210","SKW6348","DAL779","UAL2241","AAL816","WUP32","FFT4671","JBU1070","N709WY","UAL1745","N60AJ","GJS4407","MTN8505","DAL2032","DAL3183","JIA5327","JRE774","DAL1337","EJM461","N512GV","UPS5993","GJS4187","AAL1206","EDV5044","DCM125","MXY536","PDT5967","UAL2357","N212LV","DAL878","AAL1992","DLH452","SWA2148","N506DZ","N80149","N253SC","JIA5522","COL901","SCX417","SWA1509","DAL2906","N8077E","ROK30","DAL778","EJA111","UAL284","FFT163","SKW6436","UAL2449","GJS4502","N738E","RRR9110","JBU395","SKW5740","VIV520","KAL011","ENY3554","CXK285","N358EA","AAL1938","UAL786","SWA1020","N707DT","ATN3449","N730C","AAL2671","UAL320","EDV5022","DAL2662","SWA924","SNC493","N1070W","XFL550","N5405T","WJA2162","UAL1439","FFT3022","N704JM","GJS4395","JBU1687","NKS924","N6097F","UAL190","UAL2309","TAM8126","SKW5814","DAL1004","DAL1536","N450GA","LYM4710","AAL2098","WST500","SKW3475","YEL5","SKW5042","N901JF","BAW29G","NKS1681","ROU1760","SWA1503","UAL38","SCX101","JBU2416","AAY624","N242SV","ENY3501","N819LM","QXE2133","QXE2024","DAL2261","VAR834","KAP9","SWA1880","EJA256","UAE5V","QXE2204","N84HT","RPA5792","SKW6260","DAL1600","N68AL","AAL1900","SKW5133","DAL3054","SWA425","UAL554","AWI6074","N5175L","NKS288","SWA2978","AAL2470","EJA104","DHK003","SWA4291","AAL1329","AAL1230","SWA3449","KAL035","ROU1677","SKW117W","CFS8657","EJA433","NKS199","N492DM","ACA193","UAL892","FFT1790","N601CB","LFA471","UAL1307","FFT1370","SWG445","SWA2709","DAE2560","FDX5986","UCA4299","NKS711","SKW5984","N450GG","UAL1392","JIA5122","SKW5329","ABX588","EDV5534","DAL2403","SKW3712","N6PG","MXY614","AAL264","N971DB","N525PZ","SKW4656","EDV5223","FDX1458","EDV5249","SWA1282","IWY234","UAL1696","EDV4921","AAL170","N9156G","ASA1390","SKW6341","BMJ68","WUP832","SWA3295","N390K","N832BH","UAE9933","RPA5692","DAL2254","JBU2095","DAL1925","UAL247","UAL1429","SWA3195","DAL1024","N889CA","SWA399","AAL2125","AAL370","N1992A","N82045","VTE3053","AIP1407","UAL2068","ENY3653","N10TR","AAL1453","SKW3980","CMP473","ASA880","SWA114","ROU1689","N788MG","N13HM","AAL328","TTT540","FFT2410","N214WT","SKW5088","EJA556","SWA4419","N40TW","UPS5604","ASH6153","N80616","DZR353","LTA316","CFALD","BMJ46","JIA5223","PDT5710","AAL1558","DAL677","SWA3821","FDX1440","N414CS","AAL2575","SWA3404","ASH6024","N318AT","TCF623","SKW5330","SWA1739","UAL1899","GJS4531","SWA4885","ASA1351","UPS237","SWA4191","KAP117","ERH189","DAL2917","FDX1151","ZN702DL","RPA3697","N765WM","UPS5320","UAL15","FDX1710","N65PX","SWA1789","EJA923","DAL229","JBU2874","EJA188","FDX1477","SWA142","N241LJ","SKW3478","UPS1014","DAL2826","DAL451","FFT207","SWA2718","N365SZ","UAL1970","PCM8688","N509CQ","DAL538","AAL61","DAL987","DAL615","N339SP","EJA812","ASH6028","AWI6031","UCA4327","UAL668","N109TY","JBU66","SWA4976","SKW3835","LYM5730","RPA4656","EDW22","AAL1986","SWG5464","UAL1348","AAL1715","JBU2456","N179PF","JIA5267","N500TD","N39HL","SWA4264","N43BB","LXJ536","ASP556","ENY3545","SWA748","DAL1897","VIV651","SKW4245","EJA478","SWA1729","SWA1931","UAL2043","SWA1968","SWA2879","N208BC","UAL888","UPS1154","SCX242","LN400EM","JBU547","AAL1609","RPA4773","BHS355","SWA3447","AAL305","UPS5013","N235LC","UPS920","EJA623","UAL1862","N441E","EJM907","N24XP","UAL1895","PDT5853","SKW159X","JBU816","LXJ670","SWA3050","SWA808","AIP1802","LFA315","SWA4962","UAL1592","DAL3160","DAL1308","TTT688","RPA4343","DAL1346","NKS271","NDU96T","FDX1460","GJS4449","SWA1790","DAL199","UAL2636","SWA1459","CSI300","BOX395","N36EP","AAL1530","N978AP","SWA4282","ZN80338","N397NC","EDV5511","SKW3983","UAL2854","ACA975","UPS1270","AAL924","FDJ268","DAL473","FFT3155","N9152E","AAL2504","SKW4921","SWA461","DAL1417","RPA3664","LXJ302","PDT5723","AAL935","DAL839","PIT2","DAL1522","CPT8810","N231PS","SWA1756","KQA003","ACA1316","DAL1993","AIP1827","LTA906","SWA3236","N801AS","COL511","N244DS","PDT5849","LXJ461","EJM221","RPA5744","N16425","TTT815","DAL1310","MXY150","SWA1529","UCA4880","AAL2484","SWA3189","UAL1337","QXE2149","ASA311","JBU522","DAL406","POE239","N413BM","DAL786","UAL2499","FFT3294","AAL653","N454X","PDT5819","SWA2374","N513KD","N560CE","N65516","Z288SP","SKW4274","FFT1544","N320CD","NKS923","SWA3058","ENY3486","AIP1408","EJA957","GTI4436","SWA2403","UAL215","N962PM","UAL552","N649JC","AAL371","VIV765","UPS1344","N34012","AAL1093","SWA5004","NKS257","UAL1579","AAY188","UAL1772","UPS1286","TPA4033","AAL645","SKW3745","GJS4591","DAL1128","GJS4401","EDV5202","N964MS","DAL1369","N60CF","DAL492","UAL417","RPA5766","SWA1255","AAL772","AAL2166","MTN8328","RPA4549","NKS375","RPA5666","EJA480","ANA126","AAY3230","NDU93T","JBU909","PDT5930","RPA5595","BXR1572","BHS222","DAL2436","N502RF","SWA2700","UAL783","SKW3906","CXK520","FFT1115","N308KP","UPS5275","QTR756","BVN8760","N85FC","FSP602","N301HN","DAL2476","RPA5643","N7854R","AVA120","N74058","TCN808","AAL3058","SWA3452","N6652R","UPS983","UAL1975","FFT3021","N113MT","UPS1194","N321JT","DAL1546","SWA294","SKW4692","BHS61K","FDX1508","MXY308","SWA888","UAL333","N466DT","SWA2473","JBU654","ATN3335","UAL1640","UAL2328","AAL1375","N178MH","AAL488","SWA4130","DAL868","LXJ512","AWI6066","TRF80","DAL2874","PTR2324","FFT4292","JBU2817","TAI580","DAL1230","N562PC","ACA518","SWA4766","SWA3201","ASA8","ASH6266","N980CC","AMF1088","DAL2259","RPA5847","FFL1973","JZA7721","LN605PC","AAY2045","VOS4335","UAL2090","N712LU","N283CE","UAL2118","UAL1271","EJA633","N910ME","DAL2456","LXJ561","DAL1520","N525MM","SNC1294","GTI8838","ROU1878","N4000K","UAL2274","DAL1881","GXA6132","UPS9860","VJT785","N39VF","N477HC","DAL8932","FDX1079","JIA5260","BHS238","UAL1194","SKW6406","KOW992","AAL2433","SKW3898","SWA991","SWA3415","UPS334","DAL1072","N529RN","VTE3051","CXK167","SWA442","J9","SKW6270","UAL1876","AAL2656","ACA97","WUP449","FLE1888","AAL152","DAL2908","UPS1056","EDV5554","ACA563","CKK231","SLI633","N887BM","N421AJ","SWA2659","AAL3323","ENY4187","UAL2222","N447LA","VXP831","N505LP","JIA5505","FDX1465","N79TJ","SLH816","UAL957","ACA1002","SWA324","N75JM","ATN3347","SKW4066","EDV4940","LXJ459","UAL1207","JZA873","SKW5634","VTM662","DAL506","QXE2171","EIN124","DAL1421","EJA456","AKT1587","N469DM","SWA736","UAL462","SKW4922","N546CS","NKS513","PDT5914","DAL1903","SKW4074","NKS870","N284MW","CSQ101","SWA176","N4413S","HRT373","TLAES","UAL466","HAL51","N222GY","ZN999UP","AAL3251","DAL3090","N4776E","UAL1451","AAL599","SIA31","AAL2246","EJA180","N608ST","N4203P","EJA536","UAL2013","N9844S","FDX1492","NKS187","N80HA","EJA632","DAL1941","FFT2622","SKW4932","N295RA","N246WA","UPS993","N88CH","DAL577","CMP134","N601AL","UPS904","N602LF","KSU34","JBU2233","NKS2990","SKW5313","SWA1348","UPS5555","UPS1128","N990MC","APC995","FFT3356","N829AR","FFT1759","AAL2089","SKW6295","JIA5087","GXA527","UAL650","UAL518","N133RC","UAL2341","DAL1394","CGLNP","EJA212","RPA4370","SKW3757","PDT5684","UAL1427","SWA2045","SWA1569","ASA1344","CXK554","EJA262","SWA706","N6299X","N74737","N876GH","N47354","RPA4535","EJM832","ASA644","N673JH","SKW3921","SKW4017","ROU1624","JBU265","SWA1535","CXK616","CNS519","N292KW","UAL2337","N245XL","DAL2709","N761YD","DAL787","RPA4737","N421ML","UAL492","SKW3472","FFT3861","VTE321","EDV5277","DAL1301","KLM623","JBU1151","ASA1","UAL3892","DAL1463","N4523J","AAL2586","UAL1907","N94901","MXY713","DSPTEST","N724HS","N587DV","N221CL","FDX1237","JBU1078","N242FB","N737YE","PDT5899","SWG6552","EJA775","AAL1403","EJM280","NKS859","UAL1016","ACA1000","RRR9413","SKW5696","PDT5872","VTM642","N199CM","SKW6393","N41WC","N727KF","UAL1274","N10FV","DAL1970","N5KA","KII833","DAL1918","SWA1695","RPA5782","SWA3496","SWA386","FDX57","N10J","ZN891L","SWA684","LN281H","FDY244","AMX496","N738SJ","EJA800","RPA3632","AAL1771","ACA2170","DAL707","AAL2790","N80XX","SGX999","TAI440","JBU1754","PVL2267","N810ET","EDV5278","UPS514","POE318","N415P","MXY246","JIA5536","GPD154B","DAL2369","YEL13","AAL976","SWA508","UAL1048","N20DJ","N889TA","UAL1878","N6466T","N560RE","BAW6AC","UPS5871","DAL582","UAL784","GXA6115","AAL1014","JBU959","SKW3664","AMF1544","RPA5617","TSC187","GTI8602","N850MJ","NKS558","JSX720","JSX202","DAL2611","FFT1403","N618WP","N122DT","N334TC","SKW4060","N572CP","N8249N","CMP808","RPA4357","FFT3347","RPA4350","DAL2428","CMP226","N86VP","JIA9961","UAL2151","DAL858","N4554","EDG471","DAL1750","SWA1189","AAL3174","ASA972","ASA1166","N777MX","AAL1341","N115MH","N456CC","N221VR","AAL1635","AAL412","SKW3362","ENY3967","N2475L","DAL1673","JBU2484","N92AJ","SKW5102","BOE903","SWA5027","SWA2796","UPS1088","KES214X","RFD31","N1MN","UAL2023","N32EA","UAL277","DAL2618","FFT2510","N813MR","MTN8553","UAL3749","UAL2367","AWI1002","RPA3644","DAL60","UAL1375","RPA5664","N426MP","RPA5734","JIA5365","ENY3826","LXJ337","ENY3471","EDV5036","AMF1918","AAL575","PDT5946","SWA1144","SWA4134","SWA4360","SWA2143","AAL2131","CAY103","LN71NX","EJA243","N4460X","MRA703","N875CF","N801PR","JBU271","SWA1342","UPS606","N602SF","FFT2582","N4976H","UPS1112","SCX777","TIV707","GJS4169","SWG669","FFL1050","N1HZ","N772QS","LN41GJ","DAL446","PDT5973","DAL535","P362ME","ENY3566","N928FE","SWA3479","LXJ608","EJA527","DAL2419","EJA575","HAL1627","UAL1006","JBU2051","N877BR","NKS1642","AAY920","N1RJ","AAL2738","AAL455","MXY830","N4172P","EJA260","BHS142","N304QS","AAL1998","N689C","UAL2092","N711NV","N190PM","AAL255","FFT1815","N1822Q","EDV4894","VOI1750","N54KJ","NKS164","N30CJ","ENY4005","PDT5871","UCA4313","N184RB","N729SC","SKW3409","AAL2130","LXJ440","AWI6156","MXY716","GPD155B","TWY508","LAK676","N901HD","N955JA","SIA37","SWA1080","N6935L","SWA3289","N858EE","CKS612","AAY903","EDV5176","JBU2321","JZA958","AAL2514","N356BG","AAL1694","AAL1001","UPS1488","SWD3","JBU1142","UAL491","MTN8127","NKS1367","VIV650","JBU1214","EDV5162","N121BD","UAL1750","SWA2765","AFR174","PTR2388","AMX021","WIN902","UAL1688","N787RJ","MXY508","ENY3923","N68ZB","AAY994","N5253C","N221MJ","EDV5419","N4285T","CSB7511","AAL2255","VIV153","AAL922","ASH6192","AAL2025","EJA331","UAL782","ABX183","UAL1119","JBU164","SKW4303","LXJ319","UAL652","N96AM","N228ME","SWA1953","SWA1303","DAL2164","MTN8477","SWA3331","UAL2156","N100EX","SWA2315","ZN1348L","AAY661","N895VA","AAL2614","FFT4383","N1942K","TSC888","AAL1995","N492DY","RPA5817","SKW4318","N5518W","FFT4966","N2887Q","SKW5813","THY4JD","LCO1400","DAL1227","SWA985","ENY3906","AAL1056","EJA757","AAL3128","DAE1557","AAY1893","RPA5780","FFT2343","EPI555","DAL1402","UPS5919","KLM759","SWA1131","UAL2168","N247JP","DAL2036","J5","AWI6059","N789KG","N926RC","UPS1216","N284PC","N345P","N416AM","JBU202","ASA1298","SKQ78","ABX6127","VJA552","N951WA","JBU353","UPS1336","UPS80","MXY100","AIP1842","N98MH","N620QS","VOI7830","N235HP","SKW5473","DAL2396","DAL3089","UAL1052","DAL2384","UAL2405","WIS1576","RVJ1980","DAL2471","DAL668","N154TS","SWA2201","JSX775","OPT463","SHH780","JBU520","N529KF","N5237R","SWA3158","N615AK","SKW5364","N689QS","SWA297","JBU872","AAL1314","JBU2000","DAL3199","AIC174","VIV670","N764DB","GJS4178","LXJ544","FLE402","JIA5370","N144LC","SWA4941","BOG73","FFT1564","AAY1383","JBU865","SWA2137","ASA358","DAL2508","EJM14","DAL447","N308EL","NKS588","JRE714","DAL462","UAL370","WSN841","AAL1683","N95AJ","N119G","GJS4495","N97VS","FFT1499","VXP669","N48Q","SDE800","UAL1644","FFT1615","BL2FA","AAL242","N256H","AAL1377","N590ND","JTL780","SKW5624","PDT5877","N711BS","UAL773","ASA966","DAL2238","N494BH","QTR8834","LOT4TL","DAL2492","EDV5080","RPA5604","ENY3872","ACA997","AUA93","AAL3229","N76AW","GTI8857","SWA1420","NKS800","SKW3701","JIA5083","STB402X","EDV5142","TSC610","DAL3060","TSC399","JBU954","N64875","SKW4068","AAL1040","EDV5130","JSX655","UAL1575","DAL1705","SWA1086","LXJ363","JTM766","WUP316","JIA5372","DAL480","CXK308","CRE623","N610GB","UPS974","SKW5621","FFT9470","SWA103","FFT3016","UCA4271","N571BC","VOI7837","AMF1304","SWA422","N833CL","AAL2422","AAL826","N10RA","N265AV","N80FG","JAL66","TWY722","SCX342","LXJ517","UPS1132","N800SB","SWA4078","JBU461","DAL3186","UAL362","DRL75","N620DX","N1807N","N8054B","UAL1470","SWA3985","N979KC","SKW4972","UPS321","AVA126","DAL1418","N828FC","SWA642","UAL1112","FFT3999","SWA1202","JCY600","SWA3572","N563RF","JIA5366","TAI568","JIA5341","RPA5691","UAL2371","N564QS","N497PF","DAL2237","SKW3938","LYM5745","JBU861","DAL1352","TAM8194","AAL1428","EJA476","ZN624AL","AAL1452","SWA2470","YEL51","UPS76","MRA644","SWA3929","ENY3623","AAL900","N984BF","LN581EA","UAL1577","ENY3739","SWA1764","SWA776","SKW4688","LXJ507","N566EA","ENY3656","N6341R","FFT1288","AAL1220","JBU1483","MXY215","BVN8580","WJA2216","FDX264","AMF2284","ASH6212","UAL2198","FFT2210","VIV2220","CMF2587","EJA589","AAL932","AAY847","FLE2651","PDT5827","ACA631","PIT7","DLH422","N44VR","JIA5386","N136DS","UAL2445","FFT2495","UAL1987","N3555K","UAL1022","AAL1801","KLM787","AAL962","DAL2491","DAL1264","N52KT","EDV4983","DAL1658","SKW4741","RPA4345","N251JL","UCA4834","EDV5120","VXP307","N62JC","N554SL","ASH6186","SKW3703","SWA1955","VOI1881","GXA400","UAL625","N141L","SWA1495","FDX1583","EJA697","UAL1083","DAL966","NKS583","ATN3361","AAL987","ACA189","ACA782","CXK547","KAL274","N74FF","ACA1295","UAL2102","N648SP","SWA3271","DAL995","SVA042","AAY87","PDT5733","SKW4792","JBU857","N401RR","DAL444","NKS844","SKW5121","NKS2021","WAL201","JBU1851","N3804Q","EDV4976","ASA20","N820MJ","UAL315","JIA5384","ASA594","N43NC","N229K","DAL459","LXJ392","POE656","RPA4444","JSX106","UAL803","N849QS","AAL2517","SWA1900","ENY3940","UAL973","DAL2409","AAY3201","SKW4092","ENY3365","N904MA","JIA5175","KLM163","UAL2476","RPA4420","N625SM","LTA913","AAL2954","VJA359","PAL112","DAL1862","QXE2220","QUE247","N778CH","ROU1693","N132","N124UV","FDY807","N80B","N357ER","N42TJ","AAL1170","NKS387","ENY3355","AAL637","DAL1796","N990MS","N7038F","GJS4400","N139EA","N366EA","UAL1819","JTZ377","DAL1963","N500DE","AAL1917","N4053H","EDV5355","SWA1769","N161B","UAL2600","BVN8283","DAL1729","SKW5807","AAL1224","N9674D","MXY752","JBU1405","UAL1829","JIA5316","SWA2117","AMF1612","N78WF","SKW3454","RPA4684","PCM8654","UAL371","JBU1333","SFY775","N217GL","N777QS","N2GQ","N44087","DAL8935","ROU1826","AAL2821","FDX1730","SKW3987","SWA1935","AMX497","HAL19","ENY3370","AAL1944","JIA5570","SWA185","NKS3351","JIA5144","FDX1517","KPO495","N497NG","PDT5808","EJA328","JRE592","DAL3048","SWA223","UCA4251","EDV4968","PDT5948","DAL891","POE641","N62GA","AAL1509","SWA2454","WST704","N357EA","N149CF","ENY3744","UAL893","AEA055","ENY3858","SWA3638","AAL1612","SD14","AAL2747","SWA565","WJA1399","SWA1985","IWY131","UAL945","SWA1305","ASA210","AAL169","PDT5897","N738KA","ENY3822","N189QS","N124GA","N700DM","STY743","SWA840","AAL1740","N575HA","UCA4275","UAL1141","WJA1246","N201DY","SWA1437","SWA2308","N400BC","AAL2719","AAL669","CPT8564","WJA1499","N1174H","JZA605","UAL1864","KLM733","TSC398","N810GT","NKS2975","NKS1693","AZG1330","N89RP","SWA4697","FFT1402","N361MB","DAL2488","N888LB","N390AB","SKW3650","UAL1887","CJT2487","UAL2277","WJA2128","UAL1060","UAL2442","JKR54","ASI74","UAL1457","SWA601","N9317R","AAY1193","N500CB","GJS4543","N736RC","Z7279E","AAL2754","N168JG","N7524R","SWA1011","SWA1843","AVA205","AAY935","N101AF","P1484B","GJS4425","UAL1624","FFT4753","NAC802","XARCL","N345FM","CXK109","AAL1667","N909PM","XADLA","EJM352","ENY3852","EDV4893","MXY110","CJT626","HAL62","UAL281","DAL989","BXR1462","SWA570","FFT4286","N5216Y","SWA3078","ATN3319","TSC760","N470U","DAL133","DAL2632","N805BL","RPA5641","NKS353","N823CA","N506HG","DAL7","DAL8919","N305LM","UAL765","CXK584","SLI2709","N4721M","UAL1974","JIA5020","N653BB","AAL1670","N322AM","FDX1732","JKR29","RPA5700","UAL1403","AMX484","N554TS","RPA5784","N395FG","RPA3430","SNC383","N69YK","N357MJ","N945DM","SWA614","POE543","JBU143","ROU1671","QXE2060","AAL2777","AAL2555","EDV5427","UAL606","N81981","N797CS","LXJ662","DAL1540","REE300","AAY2633","N80181","SWA1639","PRE50","SKW6222","NASA923","JBU2427","UAL1532","N443DB","ERU27","NKS839","GPD150B","DAL1332","VOI750","LBQ425","JBU1468","N650AL","DAL831","JBU2719","SKW5137","N405T","NKS1675","ANA159","UPS9307","N501QS","JBU333","EDV4920","ACA1302","BCS372","SHH921","TSC845","SWA1735","PHM059","AAL2921","N1234","N979CF","AAL95","N7HC","AAL1395","AAL1209","BHS236","ENY4001","JAS123","N207AH","TPA4037","DAL3172","N14WG","UAL1919","UAL752","NKS2976","CXK500","N1098R","AAL1419","ABB277","UAL687","N604MH","SKW6293","UAL1809","FDX247","DAL1249","ENY9872","N54855","SWA2008","SCM82","UAL1068","ENY3706","SWA782","SWA438","FDX1718","WJA1226","AAL1163","SWA4205","N5341M","BAW4E","AAL2703","UAL2489","JZA988","FFS42","AAY596","N559BE","NJM725","AAL2174","AWI6106","UAL1128","N100VE","DAL2124","N239K","ENY3952","ENY3568","DAL295","NKS875","CMP404","N872RV","ROU1680","CXK1118","SWA2974","ASA284","RPA5698","N547DT","JBU266","FDX447","AAL1039","UAL1018","TAI582","FFT44","LXJ438","N8149T","OCN193","N423CA","KFB69","SKW4118","SKW5739","RPA4696","EJA608","CXK685","NJZ2","RPA5745","UPS2770","N280WD","AAY4913","SWA2222","NKS2972","NKS688","CVR1016","SWA2865","FDX455","N577DH","NKS712","SWA4826","AAL1547","EDV4978","HOB2","RPA4483","JBU1709","UAL1234","UAL2110","AAL1335","DAL563","NKS554","UAL1310","GTI9342","AAL2760","NKS687","SKW3360","UAL2331","STV11","LRC626","TJJ101","JIA5546","ASA1012","N421MP","SKW3772","ENY3589","EDV5313","RPA3529","ATN3385","NKS1708","EJA390","N900TV","UAL131","AAY433","EVA642","SWA1882","DCM4131","SKW3858","AAL1665","NKS591","N450MH","FDX1713","SWA2673","N484EM","UAL428","REH22","UAL1740","EJM256","N27KV","CPT8807","FFT3372","AAL1095","SWA705","ESF615","DAL3039","ASA1021","SWA470","SKW5897","N5215F","N59AB","RPA5626","DAL1847","AAY239","ASA1415","EDV4884","UAL345","SWA1247","AAL3129","UAL2145","JBU2096","UAL2453","EDV4797","DAL996","DAL1364","JBU2021","N8574X","UAL251","N715LU","JBU289","AAL2542","CTF164","EJA870","QXE2114","UAL2111","SKW5500","N727PJ","DAL2844","AAL773","UAE2J","N7616J","GLF27","CSB421","DAL3058","PRE507","NJE099B","SWA1283","DAL926","JAS59","JIA5262","AAY2532","WIS900","DAL491","N10MD","ASP877","SKW3736","SWA2666","AAY1262","ANA106","FDX1600","UAL6","UAL3891","FFT1113","SWA1544","GJS4558","N526LP","GJS4588","N2649J","ACA1021","N371SF","SWA1772","SWA1013","MRA604","PHM024","SWA1674","SWA4975","UPS982","JBU677","IRO8579","NKS1313","SD11","UAL2055","FFT1238","PWC328","JBU137","GJS4165","SAS907","AAL1674","ASA1075","N286PM","ENY3886","LXJ354","JBU871","AAL2568","DAL2453","JBU113","N300AJ","LFA452","N877CR","N828PG","SKW4048","SKW4735","SWA1573","SHH920","AWI1037","N550WM","AAL2006","AMF1686","EDW4K","UAL716","BBQ8261","SKW3730","N947HC","POE681","N219AB","SKW5338","N234TT","PJC60","N241CJ","N5080R","DAL825","DAL334","LXJ349","NASA918","ACA125","ASA1089","SKW3495","WUP996","UPS9865","LN500CR","N158JD","BAW23K","DAL1197","EDV4859","CPT8555","SWA2251","N9812S","CNS556","LN48AP","AAL1297","UAL1079","UAL189","LTA389","WJA2512","KSZ230","FDX1645","SKW3440","DAL1445","N420AM","JBU2715","AAL2492","AAL3056","DAL898","CMP433","UAL1944","HPN8791","N67701","TSC198","AAL1581","DAL2634","LYM3418","NJM325","UAL482","DAL1240","AAL798","AAL1982","UPS972","N24BE","AAL3029","SWA267","SWA1537","FDX1400","N6906Q","PRE81","AAL1436","SWA1887","N3LH","UAL1285","DAL338","HAL1613","SWA203","SWA2775","AAL2970","UAL1284","JAL9","SWG273","SKW5955","LXJ320","AAY981","DLH498","JBU2293","AAL1262","N55K","UAL663","N586DZ","DAL165","AAL1046","UAL1479","CYO975","ASA453","JIA5220","UPS5682","UAL1396","UAL233","N244LG","N2017E","N522GL","LXJ457","EDV4816","ROU1644","SWA2889","AAL1190","ARF7PR","QXE2040","VXP951","SWA300","FRG374","AAL1426","N9159Q","ZKP4542","UAL2224","GJS4610","AAL617","SWA3798","EJA726","ENY3979","ASH6015","N52381","UAL1491","UPS636","AAL1796","JIA5392","DAL2167","N3GT","OTA8","N18144","AAL518","SWA2835","LYM3912","DAE4926","QXE2008","SKW3415","IRO8972","UAL2461","ROU1650","SKW5920","RPA4419","AAL3256","FDX1721","KNT911","N975","SWA835","SGX703","N284WB","KAP230","NKS1650","PJC35","GXA4630","CAY204","ROU1642","VOI1702","UAL1734","AAL2420","UAL1930","DAL2452","IRO8582","N698RS","SWA371","RPA3553","AAL1848","SWA663","N111XX","AAL1276","TWY47","N1713H","N34WX","DAL9927","UAL1440","AMX022","JSX623","N414BT","UAL2725","N864AM","DAL2207","N884AE","N525RC","AAL1741","NAC803","DAL2633","QXE2402","RPA4499","SKW5191","JIA5092","SWA3040","RPA5705","SNC382","CXK522","CXK634","SWA2671","UPS5198","AAL517","AAL2984","ENY3679","DAL2173","BMA301","UAL2735","SKW4055","DAL1579","UAL1527","SWA460","JBU2880","N402XT","PDT5782","N40992","N747LG","RPA5761","UAL2002","JBU223","TSC159","N726TM","DAL3127","UPS2803","CGUNH","UAL3929","UPS5872","SCX3029","NKS1696","N513QS","UAL846","ACA964","PTR2325","AAL3027","BOX582","RPA3581","SIL93","N65295","N708NC","FFT1612","UAL1731","ASA377","AWI6144","AAL1538","SWA853","AAL899","VTE3803","SWA1097","AIC104","FFT3571","CAL021","JBU374","SKW4175","SKW4775","EJA691","ATN3339","ENY3453","SWA1489","N508MT","AMX694","AAL1628","N2537T","SLI430","BVN8286","N33302","ABX152","N984XR","N71HR","DAL598","DAL993","ENY3422","JIA5033","BAW3G","FDX1755","UAL1298","N1549E","N40EW","N2121P","AAL902","UPS809","EJA929","SWA2289","LXJ535","UAL1621","SWA4031","N1948W","MTN8512","SWA2404","SWA826","DAL330","SCX241","SWA523","N378WT","UPS801","GJS4580","QXE2298","CBC868","N3639W","SWA2493","AAL1047","SKW4887","N2516W","N576DC","WJA9145","DAL713","AAL2626","N925EM","N454L","ASH6108","N523CB","IWL135","UAL714","AIP2598","VIV113","SWA1587","LXJ359","IWY250","UAL1044","N627QS","DAL146A","SWA2772","SWA568","N99AZ","SCX381","EJA190","N699KR","AAL2589","SKW4042","UAL3738","N35RF","JIA5362","AAL1738","N200GW","ACA548","DAL2746","DAL2090","UAL1755","SWA3883","SKW6499","SWA3717","AAL585","AMX668","SWA2717","SWA9001","GJS4419","CXK619","UAL1801","UAL1273","PDT5984","UAL342","SWA3879","UAL2308","DAL625","LAE1831","DAL621","SKW6539","N963FE","ERU66","N616MB","N365WW","ARR22","DAL2970","N462CB","TSC635","BMA602","UAL2138","AAY9350","SWA580","ASA519","N65641","N808ER","SKW5514","DAL1514","UAL398","N218KD","FRG373","LN90HG","REE310","FFT3885","N383MV","SKW4022","N40216","TN48W","SWA805","UAL1597","AAL2316","AAL319","LTA438","UAL2378","N696GM","QXE2031","AAY408","IBE0265","N7421W","ENY3930","SWA393","WJA655","JBU988","UAL876","N965SA","UAL436","AAL515","LXJ522","DAL2868","SKW5266","CRE824","DAL685","DAL9959","AAL1035","DAL1567","EJA742","JBU602","CXK217","N661BW","VJA296","SWA2732","AAL1835","SKW4051","JBU1051","EJA449","JIA5156","N789FE","SWA271","SWA372","CPT8764","EJA919","N111SM","SKW5056","N7005","N2845Y","N85HD","SWA105","UPS5921","N683SL","UAL1874","UAL2275","SLI632","KFS88","N1097G","GTI8709","N162JC","BL354","UAL1471","UPS5738","N342SG","SWA6099","EIN9MY","UAL2079","PJV598","SWA2922","AAL1088","JBU1536","UAL1498","IWY212","N535GB","AAL1623","VTE021","SWA681","N1039A","UPS5701","WJA2436","DAL2132","JBU187","ATN3323","DAL623","SWA1704","UAL368","EJA953","MXY734","DAL40","SLI419","UAL427","N846PW","CFKCA","SWR39","FFT4672","SWA1275","NKS655","AAL2008","AAL2148","AAL720","UAL467","FFT4521","N82B","PAL124","SWA4156","SWA1751","N8243H","WJA2261","AAL2535","CPT8594","UAL2348","AAL940","N75HS","AAY11","N9406S","ENY3960","VOI1731","MXY843","AAL3197","XSR117","AAL3020","FFT3974","N4319Y","UAL1847","PDT5950","GJS4166","UPS2949","AFR028","MTN8302","CXK421","SKW3773","PDT5696","DAL2845","AAL2674","ENY3746","QXE2284","N94HL","N12345","JBU1723","AAL1611","FDX457","TSC611","DAL59","SWA2735","UAL2113","LN519W","FLE650","JBU1303","BAW253","WJA1885","AAL243","SWA1187","DLH456","NKS263","VTE3457","AAL519","EJA550","UAL480","PCM8730","SWA234","SKW6321","ASA223","AAL1444","SNC600","N511AT","N643PE","PCM8682","EDG96","DAL812","N252PJ","N83TF","UPS5196","TSC326","DAL137","SWA104","DAL3068","UAL1960","N1414P","CMD13","CMP440","JZA553","EDV4908","JIA5221","FFT1323","CXK369","JTL61","UPS9703","AAY2998","SCX8188","N27GR","AAL1","ACA93","PDT5964","DAL725","EDV5538","FDX85","UAL2127","NKS873","DAL57","N480CB","HI955","N400EM","NKS850","UAL1482","PSMAM","PDT5907","SKW5784","CMP466","N4883G","SKW3781","FDX1548","JBU337","ASA908","JIA5102","REY32","AIC173","FFT1016","EDV5229","SWA1018","DAL1611","MXY712","SCX547","PDT5925","GJS4383","N755SP","N495EC","RPA5631","SKW4892","CNS226","N164CD","N392SM","N340YC","N458EA","AAL2571","FDX1421","NEW605","SWA4330","JBU209","DAL8840","SWA4917","JBU1891","ASA144","UAL1733","DAL1178","N501JD","N261PJ","XSR732","AAY1006","N121HB","DAL2922","ENY3337","NKS3890","UAL1480","WFL8436","CXK674","JIA5257","AAL2318","UAL2681","N400PV","TSC622","AAL553","FFT3133","ENY3408","UAL1943","UAL1512","AAL829","MXY502","SWA2571","MXY262","SWA4597","SWA947","HYS102","SWA1594","GPD90B","DAL1701","UAL33","N327TW","EDV5482","RPA4489","UAL1265","UPS5961","SWG4473","AAL954","N678MS","N35543","MXY534","UAL1567","EJA353","N214NX","AAL563","VOI1860","SWA785","UAL527","UPS1332","UPS1075","N32531","EDV5402","DAL2717","DAL392","N855QS","N279D","JBU2066","SKW4951","RPA5849","SKW5404","AMX789","SKW5340","N7880G","N70DT","N800ZR","SWA3028","RPA5748","THT8","SKW6327","DAL8839","AAL2308","FDX6030","JBU2549","ENY3976","FFT3238","AAY561","REH8","FDX1402","EDV4861","AAY28","UAL1413","KAY53","SKW125X","FFT2290","DCM1000","UAL63","SWA1272","UAL1254","SWA3771","UAL727","GJS4460","JIA5032","TSC890","GJS4445","N63CJ","ATN3353","PTR2125","SWA4866","SWR16F","LXJ594","AVA005","WIS300","CPA2070","CLX84G","AAL1225","AWI6140","SKW5389","RPA4338","N500EU","TSC840","JIA5520","FDY127","N1089U","N8538M","PDT5983","DAL3004","VJA503","CFG3843","PDT5769","EJM194","LXJ447","AAL501","N1019L","SWA137","UAL1088","UPS223","N24PH","AMF1420","EJM86","N205DS","N84JH","UAL1381","JBU768","N975MS","MXY753","UAL732","VOI1714","AAL2241","UAL1414","DAL439","RPA4422","NKS544","DAL2098","AAL199","AAR587","SWA792","UPS1152","N770FE","EJA214","SKW3465","UAL210","ERH290","DAL1440","UPS9321","DAL2741","SKW4900","CAL5266","CKK227","JBU1229","AIP1603","CNS811","N350J","FFL1242","UAL995","SNC2207","GPD257","DAL375","N3JG","ACA934","LYM414","WJA1540","FDX1708","UAL1481","RCA242","UPS5905","EDV4995","BAW26E","N2459Y","JBU1471","UPS9856","UPS2965","N525AF","N460CA","N184CC","PVL2266","FFT3483","ENY3763","FDY801","N463FA","SWA2498","UAL286","SWA379","SWA1593","ACA590","ASH6209","CSB107","GJS4172","N583K","AIP1856","N560VX","DAL1755","DOW64","N603HC","SCX575","DAL401","AAL2661","N54VM","N20217","EJA386","N552CM","NKS614","SWA4007","SWA3159","DAL864","WCC94","UAL1330","DAL164","JBU1067","FDX1586","N257SJ","EDV4876","EJM161","N818AB","JIA5416","N884CF","IRO8970","AAL2527","JBU1193","DAL1151","SWA15","SWA3049","JBU617","LXJ650","JBU627","UPS5325","SFY439","DAL1255","UAL934","SWG383","RPA4538","EDV5250","LN9WW","SNC201","N9689Q","ARR829","BMJ70","N34FE","DAL1409","N617BG","UAL1191","FDX1654","UAL2295","JBU2475","N482DM","SKW5537","JBU312","N136JX","AAL391","SWA3759","UAL2456","LET6249","ASH6068","SWA3393","NKS1782","RKK18","N55462","UAL1626","SKW5238","SKW3680","N4302N","AAL2849","CPA870","AMX499","ENY3911","SKW5507","LTA975","N177MJ","ZN270AV","SWA22","AAL908","AAL784","N638HP","SWA685","MTN8500","CSJ915","AAL961","NKS1692","ACA994","RPA4738","JBU319","UAL2606","AWI6053","ATN3343","SWG389","N472AR","ROU1645","AAL542","AAL424","SWA3998","UAL1711","UPS921","UPS5151","DAL22","SWA2300","N305FC","DAL2876","SWA1752","UAL3748","SWA1266","SKW5141","ATN3463","N70LX","ACA159","AAL2928","N600NB","AAL2765","KII118","SWA3456","N587SA","PDT5829","UAL264","DAL2183","RPA4619","VJT768","RPA3670","AMX782","LTA902","UAL2614","UAL1875","UPS490","J13","UAL707","EJA332","N420DD","NKS1638","NKS243","AAL994","UCA4843","RPA5663","ENY3737","N1290F","N58HP","AAL1315","DHK375","N45JE","ENY4058","DAL1880","FDX1345","SCX1647","N74RF","ATN3337","N1826K","CMP197","AAL3064","N807FE","CSI571","SKW6328","LN777LP","RPA5674","JBU1551","N81MJ","N486LP","SWA322","DAL2648","DAL748","N883JA","AAL2360","SCX941","MRA609","LXJ365","SWA408","DAL2926","N2122J","RPA3544","DAL1383","N528SP","ENY3909","SWA856","SWA6189","EDV5007","RPA3617","SWA3101","ASA1159","AAL2269","N69DH","EJA932","SKW3916","JBU8358","PDT5761","N463SD","DAL1442","VOI5514","UAL1219","ARE4402","JBU1634","JBU1669","SWA2949","ENY3502","N83CC","UAL704","CVU71","GJS4567","SKW5259","N76015","NKS1522","N75EM","EDV4839","UPS797","N68HA","FFT4630","PDT5878","N990DP","DAL319","EDV4846","ASA145","JBU1098","SKW5561","FFT3230","SWA3341","AAL2574","N591SS","SKW3366","FDX1423","BTQ863","AAY2261","N868WU","N311FT","JBU572","SWA10","ENY3595","SCW3923","LJY674","DAL1526","DAL1601","N915EB","SWA5104","SWA4029","N862CD","AIC101","N594ER","SWA4238","DAL1512","NKS1450","SKW4936","FFT2415","SWA3540","N903J","UAL1649","SWA1995","N232RV","N504CP","N940MC","AAL1125","N1314S","SWA769","THY9","SWA1454","N940KD","FLE2664","RPA4642","POE663","OXF8571","ANZ1","N123Q","SKW4670","N690CH","UPS5957","UAL674","FFT1287","N357BC","AAL1327","AAL2094","N9845K","N503GS","SWA279","SKW5766","N123AB","UPS776","SKW3678","AAL874","AAY373","JSX460","AAL1380","BBQ2160","N817MS","ABX814","UAL1042","SKW5361","N849PA","N29DA","KLM757","N43VB","SNC1367","N5714F","N115BB","SCW4823","N4GA","CNS342","N327MP","N285FW","SNC491","DAL372","SKW6939","DAL2792","N73407","SCX3021","JBU2466","GPD950B","N184LW","AAL2310","N207JB","N321CV","NKS182","EDV5026","UAL410","UAL2229","AMF2870","FDX1570","AAL1423","JIA5328","AAL2540","N787GP","DAL1786","UAL2195","RPA3456","NDU87TM","N51EM","SWA2687","UAL1309","FDX1738","UAL1346","N969VS","EJA307","DAL1955","HER348","EDV4840","SWA1443","EJA565","PPBFB","SWA714","SWA2044","UPS5984","CAK1247","LXJ439","N4371V","SWA2451","N787LC","N914BD","SWA5297","CXK130","UAL1370","N213AL","UAL1344","AAL783","DAL2041","SWA1636","AAL2081","DAL2080","MXY746","EDV4853","SWA1804","ROU1649","AAL436","PDT5838","RPA4433","SWA573","KAP109","SKW3498","SWA1058","JIA5584","DAL745","SWA4420","AAL2369","DAL681","N710SC","SWA2650","N803MG","N911TC","GJS4474","AAL1607","NKS640","UAL2101","AAL827","UPS952","JIA5346","NOAA67","IWY122","FFT2148","WUP824","N69FP","AIP2569","UPS992","RPA3688","SWA1650","N4023P","GPD547","JZA875","PTR2305","N113RF","SWA576","JIA5325","N560ML","CKS867","AAL863","JBU299","N730FA","BYA561","NKS339","AAY3154","SWA2729","SWA2987","N3CT","FDX1989","N604EX","JBU2137","UAL1716","ASA503","SWA1087","UAL601","LN274AH","SKW5795","SKW3801","VPBIZ","ENY3883","UAL382","FFT1993","STB105S","DAL2818","N729QS","CSB5455","DAL1829","JBU216","UPS772","SWA3237","N1VA","GXA1814","FFT3614","DAL2655","VJA151","GEC8222","PSTAR","SWA2331","PDT5706","ENY3367","SWA4093","ACA603","UAL2406","PCM8731","HRT458","AAL2268","NKS631","AAY429","ENY3368","ENY3522","N524WF","PTR2122","FDX1723","EJA404","ANA8","DAL2302","N31JV","VTE3713","SWA3124","N773CP","N3742B","BMJ54","ASA100","JJ69","JBU1077","UAL2419","ENY4097","UAL528","PDT5932","POE305","N338N","N713KB","CXK200","AAL1631","NKS1443","SNC2103","DAL541","SKW5811","APC1974","N1875A","CPA830","AMF2924","DAL640","AAL419","VV101","NKS454","DAL355","SWA6483","N56DL","N444DH","AAL3098","SWA735","PHM112","NKS2003","UAL886","DAL808","AAL3273","N645KA","MXY111","N516KG","FDY360","N2625C","N895CC","UAL1578","DAL2408","RPA5855","BWA551","N717PK","JZA7918","AAL856","FFT3229","N59982","N35FE","CFS4790","EJA316","N6291W","N775E","N149CC","LXJ343","UAL2253","N300MW","UAL1282","N575CW","UAL573","NKS252","SWA3746","EDG206","CXK493","JJ72","EDV4890","VAR496","SWA2702","SWA1101","AAL177","N141HL","ASA876","RPA4584","N1DC","CPT8766","JBU133","UAL1721","RPA3442","N419VT","LFA461","SKQ22","N97CK","AAL1974","N969WR","AMX689","PDT5742","JBU457","DAL728","FFT3176","THY6070","SJJ176","JBU1265","DAL2869","JZA7720","AAL2655","ACA602","QXE2368","DAL2764","N107MY","AAL2819","ERU923","SWA701","DAL620","KII837","N207NX","DAL601","GJS4509","N6311S","DAL2129","AMF1022","MAX240","SWA3361","SWA1118","SD10","AAL1153","AAL2085","SWA1472","AAL2356","ACA634","DAL2940","N776DA","UAL2728","NKS1677","NKS392","FDX1803","DAL2664","SWA485","SWA901","AAL2816","DAL198","SWA4804","NGF5565","N1076K","SWA2260","DAL561","AVA046","AAL536","AAL3156","SWA4966","UAL2191","JBU743","ASA1111","WJA1727","DAL549","UAL1264","UAL1110","EDV5303","N19ZC","N999GK","ACA101","CJT9005","RPA3565","OME71","VTM663","JJ08","J17","AAL142","DAL3","N9429Q","LXJ370","N23SC","JBU102","SWA1240","TFF938","FDX1702","ACA624","FDX5319","RPA5743","ROU1830","JIA9962","N129EE","N446BG","POE525","SKW6235","N96HD","UAL2061","DAL405","N168PX","JCM458","N916AF","RPA3701","N74218","TWY20","DAL263","AAL549","GTI8127","EPI245","RPA4736","SWA1148","SWA3908","N1705C","SWA3123","N28MK","EJA961","EJA508","UAL1220","SCX3011","JBU1329","EDV5479","LCO1902","SWA3192","MXY940","LBQ640","SKW5008","UPS5789","SWA669","AAL1804","ENY3778","LN194CS","SWA869","SWA1504","SKW4962","SKW3122","SWA4144","STEPPER","UAL1102","DAL984","SKW4739","N302AZ","SKW6287","IRO8433","MXY764","SWA908","SWG461","DAL3150","N331BR","ENY3742","FDY482","JBU372","AKT1586","N4262H","UAL2218","AAL1280","AAL115","UAE221","NKS899","UCA4302","ATN6743","DAL663","DAL2290","GTI8252","UAL556","UAL433","UAL1948","EDV5216","MAA3618","UAL252","SCX8215","N977LC","ANA6","FSR816","AAL2938","GRP14","SKW3401","AAL2908","AWI6023","NOAA68","N748RM","AAL793","SWA4879","ROU1637","TSC357","N923ED","JBU1812","N114CG","N506LP","SWA1250","CSN493","ASA618","CFS8771","SKW5408","UAL203","N3471M","AAY2521","UAL2318","N960MM","N541Z","JBU178","PDT5889","AMX090","UAL2673","JBU148","JBU525","LXJ509","FDY419","SWA6050","ASA902","RPA4532","DAL804","N412CU","EJA393","UPS9875","SWA2245","N3747G","DAL1610","PDT5724","N582JM","ASA37","DAL798","AAL1411","JIA5602","N38SB","SWA3213","AAL700","N330D","NKS203","DAL1311","RPA4762","STB106S","CXK222","DAL2667","AAL2050","SKW6463","THY1","UAL2205","AAL2480","AAL2029","TSC939","JBU1316","DAL348","FDX1305","EJA833","AAL1394","UAL2270","AAL2046","RPA4660","N424LJ","TSC634","N87341","UAL2291","GJS4380","SKW4290","MTU84","N1922G","UAL2619","AAL2612","N906V","N582MM","N301BE","AAY2090","UAL2297","VIV100","AAY929","N793CE","UPS2195","N679QS","ACA2313","LN856LF","N5253T","UAL2727","AAL2658","AAL656","FDX60","AAY3054","AAL3205","AIP5283","UAL1723","N48MW","UAL347","SWA6168","N681GW","SWA4854","N86LB","VIV622","LKF81","PDT5903","UAL1983","SWA4805","NKS450","N330KP","N160BR","SWA2407","N765MM","SWA2150","UAL2245","N274KA","AAY937","KLM251","SWA945","N350CK","FDX1162","N4069F","QXE2403","N155AP","SKW5373","AAL979","SWA403","LDX700","SWA123","WUP615","UAL1694","AAL2367","ASA225","ENY3682","JBU183","UAL272","FDX1571","AAL1157","SWA2923","JRE807","N675PC","N456RF","N3704G","SWA2863","N683TM","JRE840","SWA555","N552MW","WST103","MXY527","PWR9","JBU579","EJA407","N4988V","CSQ345","UPS1084","SKW5348","SKW5300","RPA5788","N234RN","N70HH","DAL783","SWA1243","RPA4469","JBU1595","AMX1682","N302KC","N4114P","AAY757","ATN3317","UAL2410","VTM904","FDX435","AAL573","AAL45","CMF2288","SWA1166","KAI42","SLI472","NKS217","NKS543","AAL859","N247MP","VTE3801","N298SA","FDX1355","N429GM","WST201","AAL1332","AAL2591","NAC8202","KII730","EJA836","EJA810","ASA440","PJC45","N769TM","GTI7835","SWA3492","AAL2643","SWA1107","N128VT","AVL121","FFT1162","JBU488","FFT3663","UCA4262","EDV5025","N371RA","ENY3335","JBU2930","UAL1642","CKX546","N26AA","AAL433","SWA983","AAL1653","UPS1131","TAI311","AAL1499","AAL179","UAL2124","UPS662","ASP875","JIA5157","ASA301","AFR4P","AWI6109","LXJ415","UAL2160","XBMCB","TPA4031","UAL2433","JBU2876","SWA1406","N7945G","SKW3930","N518JJ","UPS5909","FRV754","AAL2110","LTA819","DAL309","SKW5548","EJM210","N18387","EJA175","ANZ6","EVA055","ETH500","N53424","ENY3835","DAL3140","SWA1085","EJM132","XFL319","FDY809","TRP9","SKW5344","AAY251","N8789T","UAL2336","N784CD","FFT2532","JTZ356","FFT97","N560SA","PDT5847","UAL1063","N61842","SWA1173","SKW5839","N91126","ENY3406","EDV5279","SWA19","N716DJ","N5626Y","AAL1321","UCA4248","N888RT","N99HS","N7487G","N3905Q","DAL754","GJS4573","UAL697","AAL2666","UAL2182","PTR2126","N400FF","JBU594","N812CP","SWA4910","AAY327","AAL1617","NKS397","N497LP","SKW4984","ASA388","DAL824","SWA1917","UAL1078","EJA402","RPA4564","UAL1607","RPA4666","FDX1627","AAL1959","SFY436","EJA703","MXY148","ASA1326","N811MK","PJC4","SKW3338","SWA2381","JIA5637","AAL2774","SWA1666","AAL3242","N939JL","LFA686","N151JM","N29CS","ABX583","SHH860","LXJ376","UAL1588","N288D","WIN901","ACA2199","PCH39W","N82BB","ETH514","DAL860","UAL1897","JBU92","AAL1668","NKS319","JBU2779","SWA100","N423EH","FFT2815","N1221M","N228RC","AAL2162","UAL2301","VXP684","DAL515","AAL2270","MTN8305","VOI700","RPA3520","SWA2354","SWA2100","SWA347","AAL3158","LXJ346","SWA1005","N915FG","LN914SH","N422JR","AAL9785","AMX427","ASA814","AIP1604","FTN5","N561CF","N527PN","SKW5426","ABX3123","AAL949","FDY489","AMX007","AIP1565","ASA1359","SWA722","JIA5541","CNS434","WJA606","N32GE","FFT2825","WJA1331","ROU1617","NOJ53","ASH6234","AAL2701","FRG1544","MRA603","N993GH","N255FP","EDV5157","N1053X","JBU1266","N725CS","DAL751","NKS338","N4175K","N350RR","CMP357","UAL2650","UAL1752","JBU1882","UAL2319","DAL784","JIA5495","AAL2585","N301KB","DAL1511","SWA623","ZKP152B","SWA3074","NKS1308","ASA1060","UAL2628","AMX426","UPS1364","SWA1682","SKW3497","AIP1595","ACA90","FDX1469","CTL15","UAL1900","UAL1678","LXJ368","DAL1354","SWA1932","N7778T","UAL722","SWA937","SKW3827","SWA1053","UAL862","JTL917","NUS530","PDT5777","SWG4520","RPA5742","AAY271","QXE2064","CMP484","CPA2082","DAL1814","CSN492","LXJ598","DAL171","EJA561","AAY453","WMN914","SWA3067","AAL1239","SWA2209","FFT1261","UCA4352","AIP1530","AAL2856","DAL1381","N61BA","LTA387","DAL3190","SCX8245","UAL1188","UAL1938","UAL878","SWA1412","N7806G","AAL662","N501AJ","ENY3993","N451R","JBU910","LN1076K","SJJ209","SKW4935","FDX1979","IRO8430","DAL2702","SKW4131","FFT1601","AVA020","SKW5412","N622PG","AAL1009","N30983","N577SP","ASH6091","N212TV","CMP446","WMN682","UPS5969","N600NM","N918AM","KLM685","ASA959","N221PL","N951E","SWA2908","AAY2111","SKW4630","AJT1030","DAL1741","N429LF","RPA4675","SWA2302","UPS907","ASH6245","SWA834","LXJ590","FDX1416","UAL2155","UAL1108","N859LL","UAL2290","N476CB","RPA4712","DAL3156","N751BC","N680EC","EDV4888","GLT4","SKW4696","SUP550","FFT2687","AAL2894","N289RZ","GTI8224","N445DU","CSN442","AAL9781","JBU381","WAL513","ABX2042","KPO44","UAL2653","GJS4420","RPA4581","JIA5344","AAL1695","FFT3456","DAL1688","CFDXQ","GJS4505","ENY4078","SWG463","N225DC","ASA1254","N390CE","CGQPM","AAL1545","CSB1002","SCX753","UAL1050","N521BB","ABX120","AAL805","CMP239","MTN8508","JBU52","IBE0308","JIA5333","N252VJ","SVL35","ACA1317","SWA3634","NKS867","DAL696","AAL329R","NKS1475","RRR9531","N200RM","IWY950","AAL1413","AJT881","RPA4806","AAL3254","UAL1026","AAL1427","DOW974","PDT5915","DAL1752","UAL2014","FFT3175","AAL1675","AAL528","AAL1518","RPA4531","RPA4739","SWA1393","NKS524","MLM241","AMX1685","FFT1875","WAL271","JBU421","SWA1493","FFT2401","N54AP","TPA4024","N385CT","NKS851","RPA3567","ASH6064","AAL1642","SWA918","AAL1004","ASA224","N7574N","VOI1716","AAL428","AAL1478","N616ST","N7225M","MRA662","UAL22","SWA1138","APZ132","RPA3480","LVJQM","RPA3527","FDX1747","NKS1331","FTN1","AMX498","VIV611","WMA9219","SWA1150","FDY125","ANA108","UAL2383","PCM8671","EDV5097","N995TM","SKW3464","MWT620","UAL776","SWA3006","CJT9004","N7772R","N998MM","DAL543","UAL2392","SIL0114","RPA4615","N1853B","AAL3036","N657CT","SCX3061","N168AG","JBU2119","GJS4382","AAL1759","N551GE","RPA3641","ENY3462","UPS844","CFNEC","JBU548","JIA5382","UAL2391","GLF94","DAL1614","FDX1729","UAL2141","ASA504","FDX53","RPA3654","WJA2040","PDT8845","FDX1879","SWA106","SWA5149","LN266RD","AAL3220","N9876T","SWA1694","N739ER","SKW4675","XAJAI","SWA912","SKW5380","SWA1241","N7198R","AMX429","N980CE","EJA507","SKW6466","N7600G","N598WC","UAL1408","N4163R","AAY19","UAL2473","N2167A","UAL3849","KNT102","BVN8359","SKW5995","JIA5136","FDX141","LN323HA","SKW3929","ASA394","N6150M","DAL2320","UPS204","N4969G","SKW5666","AAL1029","CGPUC","AAL1495","HAL18","BAW271","SWA1313","N9846W","N50CT","UAL2293","FDX1422","N3115W","AAY643","UAL476","FFT3008","VV300","CXK1080","ASH6077","UPS5946","MVP471","SIA12","FDX5163","SWR66","SWA578","CNS723","SWA1445","CMF2285","FDX1874","FFT3231","N7993W","SKW5027","FDX1502","N472MD","N170LA","AAL2884","ASA461","N126KA","N565DE","DAL1933","EDV5094","DAL1376","ENY3626","SWA2983","GTI3713","FFT1669","DAL722","EVA011","BVN8515","SWA6113","ENY3443","N414GJ","WAL771","JBU1389","FFT4354","DAL2412","RPA5677","AAL316","PTR2938","FDX150","DAL3041","AAL397","FDX1486","KAL086","N98502","EJA840","N17409","SWA1780","N628G","ACA1250","SKW5010","FFT90","UCA4296","RPA4784","N108JC","UPS610","JBU2009","GXA965","AAL1414","SWA486","N247LC","AMX649","FDX1410","SCX161","EJA216","SHH976","JIA5417","JIA5424","DLX100","DAL2221","QXE2192","AAY417","UAL474","JBU350","CFS4789","UPS1342","SKW5787","FFT1888","N62LN","AAY527","SWA1364","LXJ556","LBQ799","SWA498","N35683","EDV4915","NKS286","FLE662","N914GW","UPS1308","SKW5662","DAL2910","ROU1734","DAL1043","N712JT","DAL1628","UAL253","LTA809","UAL1045","N540M","N8843X","N7878V","DAL2399","SWA1548","VJA539","AAY574","N5115B","N746AB","N793RC","JBU2126","N531MG","SWA690","MRA685","UAL2610","SKW4029","N166WC","C8131","AAL1245","SWA2834","NKS379","LCO3607","N673WM","CXK454","N920TC","N690E","SKW4136","SWA4119","PCM8683","LTA903","N776R","N852AC","RPA4717","UPS959","JBU1584","MTN8375","UPS2975","RPA3447","AAL2472","NKS1688","FDX431","BOE22","UAL1830","ASH6167","VXP394","N682EE","CXK639","UAL2072","SWA2540","N582JA","N618AL","RJE99","TWY926","FFT1291","UPS499","ERY64","LVA442","N395WJ","CXK555","JIA5552","RPA5723","DAL1942","GPD434","UPS780","FDX1403","SWA2614","SWA881","LXJ1650","UAL2402","UPS5601","RCA591","UAL1508","JZA959","UCA4347","N6607U","FDX1712","ASA868","AAL1090","AAL446","ASA203","SWA862","AAL1590","NKS344","SWA107","UPS5636","JIA5079","FFT3055","AAL448","SWA4283","N800WF","BEZ401","WUP825","SWA4624","FLE2611","NKS559","AAL2720","EJA905","FFT4894","CGUDQ","NKS2963","N595DC","ACA1073","ENY3495","EDV5453","AAL535","N457QS","SWA1530","RPA5597","SVA038","KAL091","EJA864","JIA5046","UAL2404","AZU8708","AAL608","NKS1612","XABZN","JBU1201","N816PK","DAL2477","ACA607","N680KJ","DAL1945","WJA2276","N581EA","SKW5884","SWA478","JIA5149","DAL2134","GGT720","N5554T","QXE2200","UPS5973","SWA1159","SWG4437","N6034B","NKS292","SWA1532","N516CC","DAL1937","UAL715","N760W","AAL2980","WJA2116","AAL2345","N257CK","DAL546","MRA680","N601AF","MVP921","SWA2658","ASA195","N9224G","UAL2631","RPA3414","ENY3516","AMF1968","DAL2204","N950VM","N222JY","SWA2129","DAL367","SWA4636","N336TC","SWA2862","HAL93","AAL1160","AAL1218","TN61AP","N300SF","JBU1553","SWA2915","SWA3336","DAL638","UAL769","AAL2302","SWA4441","QXE2112","EAL9732","ARR680","DAL2397","AAL1975","N2614V","JAL68","RPA4743","LXJ92","TWY3","N191DJ","SWA1484","N7340B","NKS436","UAL2648","DAL1921","ROU1621","UAL2231","WUP675","LXJ473","UAL2021","UPS5283","SLI2722","UAL446","N26LP","SWA579","SWA1656","UAL1718","AAL2617","JBU256","N2150Y","SKW4313","N7347G","RPA4724","JIA5668","PIT6","CSB7516","XAKAD","LN212AL","RPA3511","N4410D","LN986SR","GJS4581","JBU72","GTI8209","NKS571","NKS1285","JBU514","SWA936","ASH6030","GJS4613","SKW5600","N100FZ","UAL2252","N6085F","NKS322","ACA133","DAL1090","ENY3774","RJA267","SST107","UAL1916","WFL2502","AJI9102","AAY848","SWA1146","RPA4431","SNC1256","AAL2084","IBE0347","N22NF","N961W","N681K","UAL669","QXE2154","JBU1154","PDT5929","SKW4772","ACA991","AAL971","N616CF","SWA4465","UAL1421","ASA813","JIA5111","SWA409","UAL1487","ASA732","SWA2043","N560TX","AIP1840","JIA5073","ENY3710","SWA3350","JRE844","AAL1821","JBU1060","N111ZY","UPS6379","AAL2865","N613TR","ASH6069","UAL618","DAL2637","XSR346","AAL3186","EJA109","JIA5676","AAL1613","CWG119","UPS850","RPA5696","N9580W","SWA1591","SKW4737","DAL1641","N585NJ","N981CC","Z624AL","UAL872","SKW851E","GJS4522","UPS2651","AVA139","N438E","N768JS","ENY3819","SKW6302","KAL255","NKS1680","DAL1877","SKW5928","BVN8285","FDX51","AAL2631","NKS981","VOI7770","FDX1447","ENY3446","SWA2820","N999PW","FFT1916","UAL2480","EPI231","N8901P","LXJ667","BOE103","N914DK","UAL364","N280GL","FFT1048","AAL1846","AAL1005","N83MP","N4615C","FDX1564","GJS4170","JBU1355","JBU1447","N800HC","N409LT","N516BZ","ENY3939","ENY3962","AAL2266","ETH3511","DAL2045","AAL2734","SWG5477","N899B","AAL802","RPA3693","AIC191","UAL301","JIA5350","UPS888","JIA5212","LAK620","JBU2631","AAL2620","ASA573","SWA2694","DAL2564","AAL2733","N525DM","DAL1995","NKS2001","JBU937","SWA3136","AAL2049","SLM3001","AMF1696","HAL33","UAL1988","UAL1843","UPS5837","KAP18","DAL1247","PDT5868","AAL1808","DAL2101","QXE2067","UAL688","KFS69","RPA5779","AAL498","DAL364","AAL561","ENY3469","UAL1030","AAL1622","JBU883","PRY03","DAL846","ACA992","N5DR","LN671PC","SKW4035","RKJ104","SWA1219","UAL1401","SKW4967","PHM102","DAL1570","AAL1966","SWA1092","N313SJ","SWA4005","AAL2695","JIA9957","RAX1073","AAL1838","EDV4850","N564HV","SWA3384","BVN8811","N64TZ","SWA146","WUP676","N400DL","NKS104","SWA6147","ABX2040","AAL2768","ENY3980","SKW5114","NKS2000","UPS1125","TAI531","EDV5066","AAL1608","N933PB","EJM448","SIL0100","SWA889","N393SG","FFT2609","DAL1030","CFRJZ","SWA3127","JBU367","LN796PC","UAL2106","LRC640","RPA4578","N30PN","N11EF","ATN4322","EJA908","N4428C","DAL684","N920CF","AIP3596","EDV5501","EJA695","ASH6278","AAL978","N3029H","REE120","JBU304","ROU1651","AIC116","SWA4559","SWA1296","JBU1379","KAP812","FDX1537","AAL2133","DAL2883","AAL1520","SWA4942","AAL1113","N552CK","DAL556","AAL2582","UAL2258","SWA397","N650AH","VOI7790","DAL312","JIA5373","SWA3425","SWA2355","UAL591","PDT5968","AAY69","SWA2313","SWA1624","JBU2036","N8168W","SWA1014","DAL8","ACA1287","UAL2144","FDY335","SKW6308","SKW4001","AAL1244","JBU760","VTE3756","AAL2221","EJA562","SKW385P","N622CK","AIP1522","SJX001","PGR1368","SCX3027","JBU813","N22126","LXJ605","AAL2951","CPA841","ROU1519","EJA787","JIA5596","AAL1342","ENY3838","JBU65","SWA1898","LN332CA","AAL2155","N52012","AAL1744","AAL2690","KAL213","N818BL","N32369","EJM325","N421ER","UCA4233","N2425J","ASA211","JBU1519","UAL1249","AAL2204","UAL621","ACA126","AAL1227","SWA1039","SWA1376","NKS1644","CFS8802","N651841","SKW5609","N258DV","UAL1844","DAL1009","UAL1802","ENY3517","ASA757","N699SC","ILU31N","JSX907","CPA806","N818WF","EDV5241","AAY662","AAL318","AAL298","N1842W","EDV5257","SHH975","UPS2967","FFT2168","AAL938","UPS838","UAL508","N520JG","SWA943","CXK592","ASA1188","ROU1795","N479LP","UAL1288","UAL1947","WJA229","DAL343","ASA1369","N470AS","JIA5232","Q7TVPS","N145VG","AJT855","N450L","PDT5852","JSX906","CSJ336","SWA274","N946LC","VIR156T","JRE846","N227GH","FDX1482","AWI6136","ERU816","RPA4472","SFY113","DAL507","DAL873","SWA1646","N7368F","SKW3834","NKS807","UAL372","FFT3613","SKW5671","AFR429","SKW5939","JBU1476","AMX646","CFS8699","UAL511","RPA4404","JBU571","EDV4913","N788DM","N56PD","N2710B","EJM89","EJA846","SWA4789","SKW6408","N1474W","SWA906","ENY3508","SWA801","DAL1330","TIV720","RPA3647","WMA242","AZG1331","JBU1233","UAL2039","QTR710","JBU2656","PCM8708","NCA108","FDX1485","SKW5243","ENY3615","UCA4819","VIR73Q","CSI301","SKW4288","FFT2447","SWA1696","UPS896","SKW6314","SWA3776","FDX1435","JBU842","AAL2764","UAL1057","KLM767","N2183C","EDV5468","N91HK","FFT2330","JZA876","ERU885","UAL1027","VXP532","AAL171","ELY021","LN84UP","UAL1608","DAL2418","UAL1458","N238EJ","FDX1771","UAL1961","LXJ566","FFT3791","N847BA","EJA568","CJT1804","AAL3328","SKW5595","N638EH","DAL27","AAL2624","SWA2134","DAL1794","N525MX","DAL2791","N8836U","TSC218","SKW4292","DAL651","JSX172","PDT5680","LFA463","UAL881","JBU496","RPA4701","UAL723","LN970NA","VTE3101","SWA6471","JZA984","AAL1296","SKW4710","AAL1519","FFT3053","ACA124","LN573CM","N425ML","N3193X","N3DE","ABX797","JBU360","J16","UAL1113","AAL2498","SWA6417","JBU1689","ASA39","UAL2246","JBU114","AAL2048","SKW3662","N916BD","KAL234","UAL2333","SKW5219","N98RP","WJA1448","N79113","ACA788","EDV4789","UPS556","ACA1284","SWA1777","KAP2711","SWA786","JIA5388","MJS4171","JBU426","UAL2196","KES311X","N43MS","JIA5612","VOI710","SKW6479","JUS556","DAL790","AAL304","N285PH","RPA3431","QXE2129","JTZ839","JIA5427","AAL934","SWG3512","RPA4479","UAL373","UAL8158","JBU252","DAL3009","N822MS","RPA5773","N7453G","SKW3920","KLM755","LTA977","SKW5912","AAL1490","AAY1199","SWA4834","DAL363","JBU2751","AAL555","SWA534","N650CP","FLC83","AAL592","N39NP","LAN502","LXJ373","ERY502","AAL2545","N5636F","N1103G","N33MY","AAL2210","UAL1213","SIL75","RQT10","AAL2689","VXP9300","AAL1852","UAL1586","ARE4400","RPA4623","UAL1221","NEW442","N906JR","ACW2520","UAL3895","ENY3559","SWA1185","AAL2180","AAL508","FDX1514","DAL1915","WIS101","AAL1539","UAL1935","N307WP","FFT2816","DAL8952","EDV5447","FDX1853","DAL998","WJA1992","FFT2279","JBU765","AAL1923","UCA4281","UPS513","N51AC","NKS272","SKW4787","LXJ421","UAL2437","SWA3333","AJT744","FDX1589","SWA2184","AAL1496","POE201","JBU172","HPN8786","FDX1678","JBU869","DAL1587","N818A","CTA306","ASA150","SKW3354","UAL1638","DAL942","AAL346","N12796","FFT2468","GXA6155","LYM5411","UPS752","SWG5453","RAX801","SKW5566","UAL1169","PCH38F","UAL1693","UPS1348","N883TW","LYM1848","SWA4292","DAL2623","N636TA","ASH6251","AAL1534","N98XS","SWA2537","AAL2292","QHD813","N642MM","BL450","N122ZM","JBU162","JBU355","UAL1477","AJT8542","DAL1201","NEW232","UAL1490","N8YM","BVN8554","NKS246","CMP366","N1437A","CXK659","SWA474","N234PD","N350US","UAL1257","SKW3425","N58CG","N388DH","SKW3646","TWY81","DAL1022","SWA2146","ENY4033","GTI3635","UAL1311","ACA1093","AAL2331","EDV5536","N147X","NKS266","AAY2824","UAL2260","N122MT","GLO9503","SWA3294","EJA312","UCA4371","ABX803","N9269C","AMX410","GJS4572","FFT3423","UPS5382","N248SE","N917JC","N283BS","SKW4030","WJA1867","N2947E","AMF2186","UAL2230","SKW4746","JBU138","AAY873","N751EC","SWA2102","AAL2272","AAL1716","AAL2722","AAL673","LYM3911","CKS961","UAL2060","AAL1410","AAL975","UAL1081","SFY438","N352SD","UAL2629","FDX1450","AAL1953","N177RV","FFT1150","JSX104","UAL1294","DAL304","SKW4107","LN876PA","N706PR","JBU2601","EJA244","SKW3671","JIA5291","PDT5978","NKS550","AAL847","AAL1094","AAL1601","UAL2343","AAL1106","VOI1712","AMF4278","WUP802","AAL2592","AAL1787","EJM285","MVJ33","JTL2828","N522TG","ASI402","QXE2213","WIS901","DAL1050","JAL7","N182VV","N616ZW","SWA514","SKW5743","MXY906","FDX1562","SWA284","DAL2853","N118F","AAL2846","DAL2253","CXK211","JIA5053","N456T","FDX1483","DAL1113","LXJ423","EJA511","NOJ10","AAL1780","JBU753","PCM8696","FDX3306","DAL867","UAL1613","SKW5444","N700LE","RPA3717","DAL461","FFT4872","CST84","N438Q","SWA830","N680JK","N4825N","EJA380","DAL2877","AAL1550","UAL2094","N6431N","N996LC","UAL1744","KFB15","LXJ479","GJS4177","FFT3167","N23GX","SKW5862","N434TX","AAY3167","MTN8112","SIL0115","ENY3435","NKS456","AAL1149","FDX1704","N362AS","N108LA","SWA3542","N5X","TSU802","NKS2968","RPA5827","N4717C","N6096Z","N112CZ","N541HF","UAL2202","TWY2741","SCX416","GRY6","WIA808","AAL1469","SWA651","DAL391","TAI571","SWA3230","FDX1500","CJT991","SWA3039","SKW3961","CAL5141","DAL2188","DAL3151","TSC328","JBU1036","PUE231","N221PH","SKW3652","N528MP","N999DM","AAL2455","NKS294","SWA813","N8182Q","N673E","N888HP","AAL788","SWA1356","CBC861","SWA899","N294DW","AAL1498","JIA5678","SWA245","TEK644","VTE3089","SWG9634","N409FZ","SKW4063","N647SJ","SWA2227","UAL318","KAL207","SWA3882","EJA651","N453SC","DAL3035","WIA823","AAL1824","RPA5841","N90RG","JIA5036","UCA4234","NKS262","AAL1973","VXP118","LN316PM","EDV5239","KAP828","SWA754","IWY121","UPS2988","AAL1777","JIA5321","WJA2376","UPS634","PDT5703","CGJDU","CPT8593","AVA006","EJA821","N201LJ","SWA2121","UAL914","NOAA65","AAY1458","N168ZZ","N763DB","N830NR","CCQ1580","N686SC","N709EL","FDX1709","N82SN","SWG5463","JIA5588","SKW4080","MAL7070","WJA1353","ASA158","KII585","N999VK","UAL8142","AMX686","DLH499","N118K","RPA4711","DAL2091","OUA18","AAY234","SWA444","SCW3851","CXK283","SWA3814","SKW5523","SKW5229","UAL562","SWA157","JBU1451","VTE3401","N918RM","NKS829","UPS5301","UAL718","QTR8160","POE530","ARE4401","DAL890","SWA4820","GJS4412","ETD76T","N640SP","VTE3552","UAL1503","N34513","VAR425","AAL2258","UAL148","N828CT","N490AM","NKS694","UAL1366","EJA940","SWA4413","SWA5013","AAL2638","RNI12","UCA4879","N543RJ","AWI6073","UAL2197","NKS656","N40W","N764AM","WSN95","JZA822","N899NP","OTA3","N444PA","COL921","ENY3844","N6684D","UPS5900","CKS962","PXT525","FTT1","CFS8686","NKS796","AAL471","STGRY","SKW3986","JAL2","N525RM","N753RP","N410H","N941RS","SCW3906","SWA561","VOI1821","WUP511","N80PP","DAL1534","ACA33","DAL1361","UAL1402","RPA4640","SKW6338","AAL791","FFT2912","UAL1488","VOI790","EDV5500","UAL1826","N999QH","SWA1648","MTN8110","GTI8232","DAL2059","DAL2143","NKS1660","WSN8","AAY3014","SKW6399","FFT2714","EJA266","N1681T","UAL448","N75DJ","N723GD","EJA933","RPA4560","SWA179","N40ER","YZR7426","N307MT","DAL1059","AAL3187","N54FA","CSI531","N507PD","RPA4755","CMP472","N404HR","JBU1569","SWA819","ASP574","SWA2107","N505GJ","UPS1444","XACLR","FDX1449","EDV5491","NWF338","NKS189","UPS938","ATN3391","N8198T","DAL573","N101AV","UCA4240","DAL1535","RPA9940","JBU366","JRE810","BAW82P","CFS8667","N732SB","SWA1690","EDV5361","GTX322","DAL1997","EDV5101","UAL2858","WJA1029","SWA2737","LXJ317","BMJ84","SWA2234","FDX1229","LXJ506","EJA866","UPS5061","ENY3449","SWA2391","EJA412","UAL2310","AWI6104","SWA440","AAY3052","UAL731","EDV5477","DAL741","N425TM","FFT3439","N472D","N816QS","AAL2563","LN781HC","UPS956","SKW5508","SWA930","SKW3345","N529BJ","UAL580","UPS232","NTW247","UAL1485","DAL485","RPA3678","N981BK","N1969T","ACA982","DAL2899","SWA411","DAL600","ACA948","UAL1941","ASA1079","N551VB","N904CG","EDV5525","LXJ397","SWA611","JBU2597","UCA4346","SWA168","SWA1629","FRG1510","DAL2592","DAL33","N994MT","UPS476","DAL1879","LOT6309","CRE68","WJA2424","AFR617","UPS1482","CKS228","N398MP","NKS1373","N926PN","AAL1618","JIA5535","RPA5753","HRT349","AWI6044","TWY501","SWA544","DAL2387","N508JA","UPS1198","SLI634","GLF22","SLI978","AAL3152","RPA5612","N1080M","UAL447","N523TA","SNC1103","AVL160","ENY3411","AAL588","ASA974","DAL593","VXP393","TJJ100","N500BQ","SKW4296","ROU1867","UAL2859","AAL2464","SKW5640","GJS4192","SKW6221","AAL893","N753SE","UPS2075","N30EK","NKS1635","SKW6198","AIP624","DAL683","N180MF","JBU1637","SWA2608","PIT5","LNE514","CFS8744","N662CH","LXJ563","SWA2588","N874SL","AAL2240","SWA5541","N594WA","FFT2398","N282WB","MRA666","JIA5153","N7542S","SWA1584","UAL758","EMB87","SWA1326","AAY2479","DAL1585","JBU115","N97528","WML26","DAL657","DAL1499","UAL1702","XEN15","LN65LJ","FFT1114","AAL1729","SWA1539","GJS4512","UAL1537","EDV5466","DAL909","AMX695","AAL350","KAP412","RPA4416","AAL486","LN429EM","N417JM","JBU2920","SKW5729","SWA4458","DAL1737","LAN591","VTE3804","ANA102","UAE232","AAL2093","AAL1018","N691TW","SWA1119","IJA21","SKW5314","PDT9977","FGBMQ","HAL21","EJA481","CMP809","NKS238","AAL1333","N473CA","DAL3020","DAL1901","AAL875","N7259R","NKS193","WIS600","UAL2683","N69RK","DAL2734","RPA4689","ASH6067","SCX162","DAL859","SWA160","UAL1857","AAL2869","AWI6067","SHH850","GPD152","N588SA","ENY3907","AAL1583","AAL1120","SKW5460","N8348G","RPA4389","BBQ8369","N970SF","NKS1355","UAL1225","UAL2311","ASA279","AAL1664","DAL1949","N344SK","N292CA","AAL1340","TAI556","ENY3877","AAL1572","SKW5077","SKW146A","N842WF","FFT1280","CXK325","CFS8695","JBU2455","JIA5428","AAL230G","RPA5765","N441ME","ASA1055","UAL1934","N1607L","AAL26","UWD18","JSX904","UAL1279","N131MS","SKW6268","AAL1416","SKW3629","FFT3017","SKW5434","GJS4424","DAL1202","ENY3965","UAL1924","UAL224","KOW860","ATN3359","DAL456","DAL793","SKW3350","CPA3086","RPA4543","UAL2641","NKS845","PDT5744","DAL1256","JIA5195","AIC126","N102TD","JBU1054","UAL245","N212AR","LXJ327","AAL2994","N992FG","DAL1664","N1212U","WJA1532","TMP6166","N694ST","ABX502","UAE3MX","N831GL","DAL1154","JBU939","LPE2691","N33LX","TSC608","DAL1635","N565AB","JBU261","SKW3483","NKS163","AAL3120","SIL65","N1001V","AAY2952","NKS1351","FFT2719","LXJ450","BL39J","SWA1432","AAL981","DAL1731","SWA1045","VIR26Q","N471HP","N58CX","DAL735","UCA4318","SWA1676","ENY3395","RPA4568","UAL1412","UAL2332","CXK1013","AAL1265","N5730J","N305TB","KII430","AAL1508","DAL1940","TSC340","UAL1467","SKW3406","N630MW","FRG1538","EDV5256","LXJ303","N11596","JBU1794","N999V","AAL1447","JBU1776","N6479W","UAL1911","UCA4917","PPQ310","WJA1513","ENY3428","SWA2596","RPA3593","N926TJ","BAW6B","SWA2248","UAL2482","SWA2396","WARRIOR","SWA3747","BTQ862","UAL1614","RPA4669","FFT3202","SWA1741","JBU2142","UPS1122","UPS1284","AAL414","GPD100B","DAL2074","UAL1705","N903TF","AAL2832","SWA4102","ZN9114K","JAL062","FDX1748","N371CK","N726DP","AAY2605","UAL3931","NKS1446","SKW6403","TEST","EDV5188","SKW5699","N650CW","UAL294","MXY737","WJA1865","JIA5018","CXK641","KAL8203","JRE785","JIA5054","N234DR","DCM2435","AAL2632","JIA5594","EVA025","N656LD","UPS320","DAL410","KFS34","N242AU","FFT2145","UAL1665","JBU780","FDX1455","UPS2750","ENY4095","AAL523","WFL2501","NKS982","JBU1371","N985FE","N478JW","DAL150","N5431M","AAL1287","N823ER","DAL1646","SKW5513","DAL3152","LTA533","N412MW","AAL1752","RPA4808","FDX1499","N476LP","YZR7451","N319WS","AAL2744","FFT1543","GTI8107","FDX1609","SWA1703","ABX552","UAL1359","EDV4932","PTR2231","EVA031","GLT720","N400DC","N101SN","SWA2577","DAL2446","CXK315","DAL1666","IWY211","ASH6083","SWA1603","N26HH","EJA956","KAP203","ENY3673","X254PG","N580EX","DAL1715","CXK288","DAL2587","SWA384","LNE515","N37TY","N71CH","N9796V","ERY44","SWA3554","N793FB","N166RB","AAY974","EDV5356","ASA715","N822RL","DAL2185","N3112S","NKS277","KOW818","RPA4534","AAL2904","N62SP","AAL1063","UAL2107","SCX1697","SWA1486","EJA914","ABX3115","LXJ520","SWA865","N650W","SGX719","AAY3165","LXJ342","N818LM","SKW4004","RPA4366","N437F","FFT1824","VOI7705","JIA5473","SKW4711","BLK11","N1305Z","N926MT","N71TS","UPS7985","UPS7930","SKW4244","N101JE","AAL457","N2113Q","PDT5959","GJS4493","SWA998","N1878E","JCM810","EDV5309","AVA004","CXK245","EJA576","SWA1576","CFS200","AAL1435","N6235T","TAM8086","RPA3548","JIA5613","JBU2628","XALAG","MRA655","VJA537","N3174Q","SWA4295","ENY3437","FFT2250","ASP816","JBU1601","SWA846","LXJ325","UAL1726","N340JD","N480LP","DAL458","EJM168","UAL1837","SKW5291","JBU1168","ENY3736","BHS211","NKS1449","N6234C","CGYVY","AAL2717","SKW5633","JBU948","SKW3724","N831PR","N286MH","AAL1477","N856LF","SWA1360","FDX1309","SWA2029","KLM616","N250HM","N637QS","SWA639","DLH459","VIV4148","N6534N","SWA2686","DAL2579","LXJ324","UAL1293","N208CV","TSC889","N6756J","SKW5826","UAL635","ENY3921","WUP94","UAL1796","AAL1108","CMD70","AAL2371","ASA514","N2471U","ENY4041","VOI1833","N555BK","JSX880","DAL110","FDX1292","XEN45","VJA30","SWA1048","M184NW","RPA4424","JIA5078","JBU2632","N575KT","AAL1331","VIV602","DAL1344","N8375T","UPS996","UPS3272","NKS1440","N289RT","N9302F","SWA956","DAL2800","DAL442","N805FT","UAL1634","MXY332","RPA4524","SKW5267","UAL725","SWA2086","N777ZY","AAL2041","NKS633","N94RW","AAL1361","JBU854","FDY304","N811RC","N529LH","DAL2980","MXY537","EDV4887","UAL1915","ACA1251","UAL153","CSB594","PDT5700","AAL850","VXP668","SWA1308","JZA899","AAY3053","AAL643","SKW6398","NKS521","DAL2566","AAL1363","AMX644","CSB925","N940WB","TSC113","DAL125","ASH6303","WML75","AAL2681","N721F","FDX7","DLH462","N195SV","UAL1735","AMF1692","ASA766","N276LJ","N701LU","SNC492","AAL2622","UPS1468","N90404","APZ101","N603DJ","SWA424","ZKP1969","N910BS","JBU169","FLC69","FFT103","N2242Z","UCA4856","JBU2501","FDX1323","SWA3009","NKS1451","WJA1484","BVN8549","SWA7","SWA3538","N31WC","UAL2018","UAL1486","AAL2677","VNT452","TVS7M8","FFT2016","PDT5892","AAL1404","CPA880","N821CB","CPA844","LPE2690","WEJ42","QXE2026","AAL770","UPS3770","PDT5857","LXJ465","AAL2621","FFT4181","N41GJ","ACA1074","HRT119","SWA727","JBU2104","SWA1540","N424PC","QXE2061","SWA2285","N274SG","CFS8977","UAL1616","AAL781","SWA435","LXJ332","FDX830","N719FH","RPA4415","AAL878","AAL520","N550PL","N1WC","UPS213","DAL1044","ENY4096","KAL233","CPA2081","JIA5475","FDX52","DAL2455","AMF1964","SWA1652","LXJ527","AIP251","SWA4122","ASH6154","SWA404","SWA2239","PDT5887","N241U","SCX703","JBU2824","NKS883","LN246PH","UAL1803","CXK324","AAL2327","IBE0156","JBU321","SKW3419","CFXCM","ATN3465","XSR326","IRO8312","SKW138Y","DAL2105","EJA522","AAY491","CSB805","N248PH","N232BB","UAL79","DAL351","N550TN","UAL1782","AAL1374","CVU73","EDV4827","OPN24","TAP203","CFS8675","N120GX","UAL1106","NKS300","RPA3461","EJA845","N45RC","SKW5970","DAL1145","DAL1237","UAL2033","UAL639","XBRJF","WJA2152","JBU2534","N2043A","UAL1465","N544QS","SWA4132","ASA1124","ATN3451","N8275A","GXA6126","N888DH","DAL3033","FDX142","SWA1104","UAL2452","VXP950","UPS1540","N8LJ","DAL665","N280LC","BVN8630","JRE837","ASA907","N664CE","JBU873","N325PE","SWG5535","XAMRX","DAL1052","AAL1121","BLK10","JSX503","FRG1690","JCM312","QXE2069","SWA1384","AAY486","N762PK","UPS7928","N922BZ","ACA1042","N72NG","JBU224","DAL1589","N1SN","DAL1505","UAL860","AAL168","KAL037","AAL312","SWA479","AAL429","ENY3498","FDY8050","N136MA","AMX091","WJA2378","AIC127","VOI1813","RCA141","EJM889","SWA117","NKS1769","PDT5683","N989KC","N4852N","AMF1590","N322TE","GJS4516","DAL1250","SKW104T","JBU1604","NKS903","ACA930","N240MC","JBU2729","N34DG","UAL2263","EDV5196","ASA1162","N6828J","N122JM","EJA678","N826D","HAL96","VJA315","N741NG","N41404","MRA639","FFT3517","SWA1523","JBU36","SWA3819","SWA3372","JBU523","VAR466","UAL624","UAL361","FDX1161","DAL1208","LTA673","JBU1854","DAL2716","SCX218","AAL624","N53577","SWA3803","LN248PH","N810QS","N773SW","SWA2332","ACA96","CXK501","ELY005","N498FL","N660P","SKW5076","ASA1178","SWA1979","NKS154","N350FN","DAL2089","SWA2063","N770EJ","SWA697","DAL2301","AAL1830","N882C","SKW5212","AAL2413","N914CF","DAL8930","N381DF","SWA4965","SCX3065","JAL58","N41HF","GXA4427","EJA367","GPD151B","SWG174","DAL916","MXY575","UAL1569","ASA7095","DAL2937","SKW3381","CMP867","PDT6001","N991CE","SWA2007","FDX1457","COL308","SNC2204","EJA506","AAL919","UPS990","FDX9143","N2174V","UAL2612","AAL2553","N102US","JZA939","VIV4146","FDX855","CYT90","SWA4711","AAL977","JIA5034","UAL425","ASA322","RPA4802","DAL3181","EJM2","VPCOH","SWA804","SWA3027","LBQ612","N907PH","N96SW","NKS197","N523RA","NKS289","ENY4086","EJM652","FDX1466","AAL1356","AAL2005","N50AF","N1968W","QXE2123","UPS9870","FFT2713","AAR284","FFT2341","SWA1070","ACA7247","QFA93","N9702X","NKS783","SWA2476","SLI2609","N722CC","SKW5012","JBU2001","ABD7264","SWA1607","DAL918","N730NG","AAL1480","PKW836","JZA950","SWA2712","FFT1444","LTA8100","DAL2156","N816VA","N6SA","ASA898","KII587","LN730CP","ENY3586","UAL2492","DAL1960","PHM113","N839SP","SKW4011","GTI046","N1637T","UAL311","SWA1161","SWA5073","SKW3694","SWA4537","N196CC","N358MB","N808RW","JBU1733","RAX716","SWA280","UAL1834","N199DH","SKQ74","FDX40","UAL1464","XARYK","NKS183","SWA308","N94794","UAL2661","N421LH","AAL1794","SKW5123","N224LM","FDX1437","XAA8490","ENY3724","N372DG","N742WT","DAL3036","N98LK","AAL1223","N5G","SKW386R","N963R","N665CS","UAL2000","FFT4371","UPS915","CSB129","EJA917","UPS488","PDT5755","EJA851","N88HA","SJE97","ACA40","N585SC","AMX057","VWA211","N3130W","N534WC","LFA330","DAL652","N3228W","FDX1528","MAL7062","POE200","VJA312","N71LU","YEL11","AAL797","DAL3013","FDX1706","FRG9516","JBU1368","N354WC","NJE158L","DAL1195","QXE2032","SKW9915","ASA121","UPS5790","FDX1544","VJT884","N451GP","N3867G","TRF561","N510MG","AAL1389","N1776L","UAL434","RPA4709","SKW4662","AAL968","JBU1881","FDX5503","N60VE","PIT11","UPS2773","RRR9411","DAL899","N4GL","EDV4927","AAY1717","DAL2577","AAL356","JBU2818","LXJ658","KES201K","N680EA","PDT5778","NKS795","AAL2452","JBU888","N629BC","N76VZ","UAL2256","SNC200","EJA868","DAL982","N745RP","UPS5383","ROU1662","SWA2236","SKW4722","AAL2799","VOI1811","JIA5393","SKW3357","N530SR","NKS212","FDX1728","SKW4679","ASA762","JBU1655","EJA869","UPS2845","N55S","FFT1404","NKS218","LAE1836","THY10","N763U","SKW5709","LFA464","UPS7841","SNC1466","SWA2264","CFS8976","SWA1382","SWA2869","WJA2262","SKW3463","FDX19","DAL699","ASA802","EDV5084","UAL2479","DAL2736","ASH6181","N717FH","FFT9632","DAL2247","RPA3478","PTR2939","SKW3803","N169PS","VIV510","N333MG","IRO8408","QXE2251","BHS221","XSM55","EJA771","N335RE","N216TM","CXK660","UAL1447","N78TJ","ENY4093","PDT5794","AAL3235","N556MW","QFA15","AAY213","CJT1926","N7819U","UAL2091","AAL963","PFT420","ASA512","RPA4750","DAL2854","AFR816","JTZ623","UCA4368","SWA964","SKW5041","N21YM","N5082R","SWA2263","N143TA","JIA5019","AAL882","LXJ393","N914TV","AAL2526","T7CIPRI","N210CX","UAL218","N8813C","SWA2392","N42EL","DAL2693","DAL1368","TSC539","DAL1340","UAL269","AZY500","N424W","SKW5667","CXK507","N318CM","N5004Y","DAL36","AAL1085","NKS371","N167DP","WMA27","SCX383","UAL1352","EJA947","SKW6269","DAL482","SWA1089","N299KP","DAL1650","CGUSM","N212CJ","N561SA","N44WX","DAL438","ANA105","ASA7096","JBU118","AAL3257","EJA610","KPO5","JJ14","JIA5062","SWA3460","CPT8586","ENY3529","SWA1419","THY184","NKS380","AAL2743","AAL3170","AAY737","SKW4764","DAL611","JIA5548","CXK383","VTE3711","SKW9913","LXJ380","RPA5592","CSQ103","ENY4048","DAL578","SWA1304","DAL654","N119A","N695BA","VXP543","CPA843","SWA997","LBQ197","SWA1395","DAL848","N4299B","N529NM","SWA1536","SKW4761","FFT3574","SWG5483","AAY558","FDX1453","DAL396","DAL1242","N205TM","CTL102","QXE2168","N1017L","MTN8102","CSN8032","UAL2314","PRY02","AAL2536","N990MR","ERH806","N524CC","SKW832E","EJA388","SWA4321","N584DT","NKS919","AIP1814","UAL1099","N135CG","N680G","RPA4608","ACA604","SKW6292","FDX1857","SWA416","AAR242","N299MA","UPS1070","ABX105","UCA4308","UPS300","VTE3501","JBU687","N231TG","AAL1311","JIA5197","N213JT","N688QS","MTN8401","AAL982","N122AS","PDT5772","AAY1015","EJA918","EDV4985","AAL10","KAP1738","AAL930","UAL1877","EJA804","DAL201","ENY3503","AMF1830","CXK155","JBU1503","FDX1461","DAL1224","JIA5457","UAL2306","ENY4088","EJA8634","QTR8835","DAL393","JIA5421","AAL2314","TTT547","SKW3636","AAL3089","HER808","N208TR","VJA807","N2421Y","SWA4464","AAL1825","NKS321","RPA5802","AAL276","AWI6033","NKS200","EDV5513","DCM4013","JZA921","UAL1793","BOX449","N8310N","SWA433","KAP308","PDT5923","UAL951","UAE9914","ACA560","EDV5494","AAL1146","JBU519","UAL341","TN817X","N695JH","SWA2154","AAY2246","WAL701","DAL404","CVF6","SWA287","LYM1180","N5573D","DAL2133","SKW6322","AAL1254","UAL2358","ABX505","N8WC","N605PC","AAL355","N303PC","N489CP","DAL1914","N336AP","FLE601","AAL74R","DAL1504","SWA264","N82GT","VTE3854","SKW6282","SWA1609","PTR2283","SWA407","VXP463","RPA5678","SWA762","LTA857","PDT5920","DAL1091","DAL1380","PTR2118","UAL307","N168GC","N897BA","UAL818","CNS61","N390PM","N756DF","SKW4947","N615TN","N591QS","SWA1773","UAL2387","FDX1471","SKW5957","RPA4588","N990TC","N55816","BAW225Y","DAL3106","JBU1411","AWI165A","EJA167","AFR694T","CXK1041","UPS914","ASA300","N760AD","SWA4982","AAL2750","UPS984","N4478J","RVJ275","JBU2922","UAL1861","UAL3919","FDX1715","UAL1937","JIA5590","GJS4614","N5284Q","PDT5824","AAY978","SWA992","DAL1873","EDV4944","UPS1052","Z978NC","UPS905","CGRCI","KAP1969","SWA2535","AAL3206","AAY75","SKW3303","N950P","TPA4069","AAL2203","CSB507","DAL1375","UAL2457","KPO780","AAL1655","EJA164","SWA2819","ATN3443","N900KU","JIA5042","N172TG","ASA392","UAL2027","DAL1276","ASA1181","N9417K","N982DW","CAL003","AAL2630","KES208N","RPA4657","DAL8922","FDX1481","J12","DAL979","UPS856","ASA1258","LRC671","JBU1102","N52KX","KOW125","SKW3501","AAL2156","EJM713","N1929Y","SWA244","UPS2071","AAL2343","UAL1725","HPN8774","DAL9966","TSU7132","NKS171","PDT5947","JIA5519","DAL115","N767VK","N26360","DLX24","AAL1912","N58HG","FDX1587","ENY4050","DAL3018","N345PC","WJA1230","FRG9521","TPA4101","N464TT","FFT2015","SWA349","N212AL","AAL2935","SWA5055","N910AA","UPS5592","AAL2212","FDX1523","AMX787","UAL28","ACA2312","HAL1617","AAY1284","FDX5025","JBU470","AMF1984","ZN63879","UAL544","AIP1500","CXK388","N76DZ","AAL3159","DLH543","VOI1835","MTN8103","EJA588","AFR090","BMA224","UAL1929","SWA5031","GXA3139","SWA2554","JRE843","UAL1946","SWA4370","AAY1454","UAL1976","SWA1385","AAL1836","SWG600","BHS202","JBU2584","N960DT","SWA3129","N9228Z","MXY842","N13760","UAL1971","WJA1788","CXA830","TWY800","JBU736","JSX9426","FFT4263","N897SM","AAL1849","DAL2563","DAL2643","N26370","FDX1314","AAL2077","UAL691","PWA62","NKS1626","RPA4751","UPS943","N210WV","DAL3129","RPA3643","RPA4439","N86PZ","EJA396","ENY4089","CAL007","SWA3953","N199TS","AAL2466","CTL2","AAY2026","N50VS","N687HB","SKW4311","N133GL","DAL1281","VJA309","N3802N","ACA1322","N856BB","ABX2245","SWA1987","RPA3652","N9DF","N599BG","N97MK","N8151R","N641MB","UPS5495","UAL2203","FDX1446","AAL2728","SKW5383","SWG2572","AZU8704","DAL602","SIL136","FDX1434","AWI6063","UAL1873","UAL1341","UAL1384","N581WW","JIA9969","N988CE","AAL2711","SWA958","VLY91","CMP471","ERU450","EDV5199","N513LH","RPA5846","AMX408","SWA480","SWA1601","ZN68379","N500XP","PFT250","LXJ539","N746E","UPS918","UPS9301","N555KB","SKW5657","BVN8279","RAX117","SWA2877","SWA1334","ASA1014","HRC908","AAY2872","TAI444","DCM6072","AAL2497","N10TB","JSX320","JBU411","UAL2666","AAL2144","N247FR","ENY4031","SWA4893","UAL1355","JBU1607","LXJ532","KFS198","N457CA","CFG2198","UAL1576","SKW5410","N891AA","ENY3570","DAL807","N613JR","DAL1099","JIA5183","AAL3234","DAL1607","SWA2301","SKW5698","DAL997","DAL380","LAN500","PDT5725","CKX553","N730MN","DAL3038","FFT2467","UAL18","NJM998","AAY8","HI1001","UAL581","DAL1362","AAL1968","NKS325","N930WD","AAL2811","ACA1020","SNC8107","AAL729","UPS1190","N983CE","N60PT","N862GM","VOI1791","JBU2194","GPD152S","AAL2047","DAL965","AAL2373","RPA4408","N3810Z","AAL3244","BVN8588","N35609","UAL2086","PCM8704","N299RB","SWA1490","UPS5807","DAL1781","ENY3343","N5815X","AAL1987","N545LK","PDT5917","EJA741","ATN6725","UAL1382","N127GA","FFT2041","JBU2160","SKW6473","AJT8532","EJA543","N38HF","CPT8577","SWA1088","AAL1636","AAL300","SWA3613","N15671","PDT5867","N523RS","N6466N","JBU154","CPA831","SKW4759","UAL358","RPA4447","VOI7810","POE653","N23DC","AAL1399","AAL1561","N712WB","WJA2272","DAL484","ANA112","UPS734","BMJ1694","CXK396","GTI512","DAL3149","DAL917","IWY410","SWA101","SWA2191","AAY982","SKW3901","SWA753","SKW5295","NKS386","PDT5692","ENY4122","NTW14","GLF75","UPS9603","AVA148","DAL1992","VIR103M","SCX3025","ROU1754","EJA636","AAY3189","RPA3572","SWA3994","JBU2838","UAL1167","SKW4978","AAY3173","ATN3441","N25NX","PUL106","MXY650","N909TT","AAL2831","N325MA","N373D","N7HM","RPA5649","IRO8592","UAL1093","UPS1076","N119TV","AAL1159","DAL510","RPA3718","KII517","ENY4014","AAL1107","UAL515","N612JJ","EJA738","SWA1129","UAL1594","N74618","N2C","N1834W","DAL963","AAL442","SWA2188","ENY3968","FFT1616","LXJ603","N601SP","ASA1179","DAL2","RPA4807","SKW5585","N351C","UAL225","FDX120","ACA1300","WST517","ABX414","N9628H","UAL406","N377TW","AAL2095","FDX266","AAL387","SKW4305","N967BG","DAL3173","ETD1","JZA604","CFS8775","AAL1373","N472LP","ASA1066","N9868V","UAL1789","CCA984","SWG9030","SWA278","N520KJ","SWA1343","DAL1961","UAL1959","GJS4448","FDX1885","CMP485","AAL2607","SWA742","JBU1385","LN80LJ","EJA570","UPS1384","JSX850","CMP479","N278TT","ENY3647","BAW283","UAL1176","SWA2109","UAL1100","ACA1039","GJS4399","JIA5146","ZN97AQ","SWA1094","ASA134","N539PE","N786CP","N45GX","SWA1688","SKW5822","N222LF","DAL307","N100WT","N44013","SWA955","PDT5698","FDX1383","N8GU","BVN8282","UAL334","AAL2179","IBE2624","SKW5241","N98FA","VOI1901","UAL2216","JZA652","N5528Q","UAL1012","UAL2149","N408FZ","N80EM","NKS1284","JBU830","ASA402","AAL2900","VOI1751","AAL2548","PDT5781","N600AY","DAL320","NDU60","AUU14","N141JM","N414FJ","SKW3330","N21VC","FDX1573","AAL460","AAL2124","SWA341","AAL2263","RPA5648","N199HF","N458PT","JZA636","N368QS","JIA5664","UPS5215","N3517F","IRO8569","UAL2440","BMA451","WJA2136","C6FLV","NKS786","QXE2029","FDY443","SWA1298","N91WT","UAL2623","UCA4828","DAL1763","N532EB","JIA9959","HRT556","UAL502","SWA4404","ETH553","N525JN","PXG680","REH1","DAL1101","N314VV","AAL2949","AAL3038","N4531V","AMF1990","FDX1420","ENY3734","VIV741","GTI8255","AAY2810","NKS567","WJA2514","AAL2294","NKS1657","CKX1089","ZN386US","VIR117F","N595AG","N351BC","N922AZ","ASA1266","N499WR","EDV5362","VJA310","QHD811","JIA5432","DAL243","AIP3598","DAL1415","UAL1462","AIP1943","LN915SH","ENY4094","ENY3526","AAY3228","JRE794","MAL7050","UAL456","N8QV","SIL55","KAP2230","SWA2090","N173SR","ELY027","UAL2460","LXJ485","NKS1309","WAL251","TSC891","XSR730","ACA129","UAL1373","NKS1311","N86BB","AIP1892","N82KA","N826SC","KOW301","PDT5996","BWA790","JBU1690","N72SG","QXE2126","N156PH","N306WB","RPA4379","N700TL","N545WB","SWA1399","MXY328","FEX903","N729GB","FLC79","AAL837","N813JP","DAL1078","VJA545","UAL1336","DAL1662","TTT545","ASH6133","ALZ400","N79549","UAL2488","JBU173","N75GY","JBU821","WJA1231","WUP918","UAL2483","MTN8129","N124FS","AAL725","DAL1709","VNT610","CJT588","PDU52","N80F","SWA1926","N41PR","LN90KM","N921RE","JBU1288","SKW847Y","UPS5975","N801AD","N909MM","N921JP","N104WF","N819MA","RPA4555","AAL1226","JZA709","VTM687","UPS618","JBU1852","QXE2075","LN428JE","STB941","AAL293","IRO906","N98LE","EDV5284","FDY422","CXE501","UAL1525","N578JG","ASH6032","SWA3708","UAL1456","UAL1629","SKW3942","CSB7513","NKS518","N916LF","N6355T","SWA2544","STB102S","EDV5156","N954DS","CGCGA","SKW5256","N213PC","N520ER","UAL1029","N66070","N515DB","JBU1708","N521RT","ACA600","ASA1335","DAL2510","MTN8502","EJA357","HAL79","N591SF","FFT2025","EJA254","RAX264","LTA834","N50VR","SWA1557","N850PR","JIA5579","QXE2180","AAL2335","DAL3080","N787EP","N99AC","UAL2232","SWA2092","SWA3319","N545RW","AAY692","RPA4614","LN774LK","N561R","PDT9978","CFMDF","UAL1445","O920AM","ANX1208","SFY445","AAL2958","WJA1710","UAL99","N877NG","FFT2678","FFT1267","FFT2339","N805JA","AAL2843","CAL5235","KAP14","N510CF","GTI3611","FFT1079","N824HH","N710LU","ASH6066","SNC1260","N218MD","THY6557","THY195","N114TP","N10DF","N878SA","LLN284","SWA2662","AAL2036","SWA2279","EDV4787","UAL2669","SCX3059","N44458","SWA3200","N300KC","UAL1824","JTL600","SFY205","SWA4313","N91GJ","DAL265","N4630P","JIA5246","DAL646","ERU76","SWA4429","GJS4584","AAL2956","SWA1170","ACA584","SKW113R","SWA635","N8395","UAL477","RPA5790","DAL2201","N886EC","ACA1054","ANA8402","CXK232","DAL1046","ENY3919","ENY3361","SKW4177","SNC2301","UPS560","BAW9SW","VTM300","IFL521","UAL3899","FFT2939","JRE943","JBU891","N65KL","UAL1827","FDX15","NKS2022","ENY4272","ASA1350","EJA178","AAY163","UAL1332","AAL1598","JIA5431","SKW3461","UAL844","N16FJ","AAL2771","DAL3155","ASH6105","NKS946","SWA4605","CWG529","N484EC","N90HG","AAY254","UAL2373","SWA668","RPA3423","UAL1239","AAL1592","HAL59","UAL1365","SWA434","COL1910","N294RT","AAL1053","N925AK","AAL1999","AAL2857","UAL2413","SWA1740","N984BX","N525XP","N387EB","UAL1664","N519XR","FJI810","AAL894","NKS582","JBU2286","SWA6547","SWA6127","SWA2733","BOE57A","UAL790","ASH6055","N101RX","CNS2217","N828LP","GJS4410","N161MD","AAL1290","ABX3155","FTN4","JIA5374","RPA3716","N974RK","ACA958","N878MJ","N766QS","SWA913","AAL2854","SKW4929","SWA319","AAL3111","CXE700","DAL1580","SWA4099","N1857F","SWA2714","DAL231","WST602","EJA604","SWA2160","N101GV","NKS777","FDX649","SWA3345","CMP312","N333SE","UAL1137","SWA4228","DAL1391","JBU2625","UAL797","DAL1241","FFT1491","UPS1144","EDV4936","FDX1722","IWY510","UAL2100","FDX1415","UAL505","TPA4098","DAL2347","N718RR","KAP11","VHILJ","SWA1813","TSC615","SWA138","JBU1059","LN279AH","N238E","UAL2298","DAL1808","JAL10","CXK612","N2228Z","UAL2029","SKW5768","DAL809","JBU2737","AAL1919","RPA3483","JBU286","AAL2121","AAL469","N36DE","N212AG","FFT1417","DAL1285","VXP464","N460QS","UPS474","N1PX","LFA321","UAL1418","FFT1191","DAL1704","NKS838","WJA1696","LXJ591","RPA3536","AWI6158","NKS928","SWA2361","ENY3989","JIA5353","N6311X","KAP316","N9154H","SWA1217","N92SH","UAL493","N4147G","UAL199","VOI5502","DAL1282","JBU1537","N48CG","FFT4629","FDX1608","RPA3624","AAL2693","JBU2510","CSI591","DAL1882","UAL206","N923KB","N132AS","AAL340","DAL1597","SWA5063","SWA4023","ZN90JF","FFT1116","N106VP","PDT5901","N632PB","FDX1426","LN687HB","RPA3665","N9254Y","N104QT","N907WS","AAL1955","UAL432","EJM615","DAL384","FFT206","RPA3497","RPA5770","JBU2033","AAL2967","DAL1815","DAL2195","DAL1461","JAL8","SKW5612","UAL2273","PAC947","SWA481","THY903","VJA801","FFT1446","UAL656","LN682D","STY543","SWA1426","KAL256","AAL1862","SKW4238","DAL486","JTL663","SKW5685","N87PB","AAL1127","N84366","FFT3736","SWA1797","N61557","AAL1544","DAL2884","SWA204","DAL1068","EDV4987","ENY3948","UAL1212","WUP820","AAL1491","ENY4036","AAL2229","SKW5818","NKS1783","HAL3","DAL2849","JBU2725","N77UW","SWA3420","SKW3476","IBE0222","JBU1155","FFT3210","UAL438","N441LT","GTI8792","DAL1157","N424PX","CSB7514","MXY907","UAL1514","FFT1563","FFT1204","AAL2057","AAL3022","SWA4844","RPA4501","AAY1336","N2162K","ASA1041","DAL557","JBU94","SWA2203","AAL2157","DAL2929","UAL1067","UCA4228","AMF2118","RPA5672","UPS1276","LYM1430","XSN82","AAL2183","SKW4082","AAL817","VTE3093","UAL329","NKS2962","SWA3087","DAL592","SWA4066","SKW3814","SKW3876","JIA5037","DAL2330","N949SA","DAL45","WUP900","SWA2959","TTT685","SWA2176","DAL1430","SWA3874","ERA813","FDY246","NKS509","LXJ386","AAL2710","N641BW","BTQ352","JBU1988","EJA255","ROU1735","AAL980","EJA945","UAL2016","KES305H","SKW4677","IRO8416","AAL2236","DAL2552","N232RJ","DAL2628","N1265H","ATN3331","SKW5390","JIA5124","ERY277","SKW4977","DAL636","ROU1727","SWA205","N75255","UAL2117","DAL1481","PDT5805","SWA2131","AMX664","JSX101","N86AH","NJM95","N569SG","UAL289","UAL2439","N600WA","JBU495","FDX1731","SKW6427","KII117","JIA5038","AAL1594","AAL1933","SWA2398","AAL655","AAL3131","AMF1682","AAL2712","N244ND","JBU352","AAL2450","SWA2829","DAL664","FFT3242","JBU1016","PDT5913","EJA612","N848DB","N804DL","DAL2767","JBU287","N7148A","AMX432","RPA3481","AAY951","N95TC","TTT680","AAL1013","UAL2026","AMX421","AVA014","NKS1090","MVK41","SWA1981","VOI1730","UAL1250","AAL1184","N3023T","AAL359P","N190BW","N8317Y","DAL2536","EDV5171","JBU1505","RVJ19","N620HL","N386TA","UAL1704","JIA5589","SKW5476","FFL696","AIP1597","EJM776","SWA6470","N900GN","RPA4795","PDT5833","AAL119","JAL16","FFT1203","DAL2473","N3114A","ACA661","EDV4828","ATN3455","N1150N","AAL227A","ABX1820","VJA401","FFT9432","LYM516","N36JE","AAL1816","N913QS","N378JA","UAL805","ABX3133","DAL1583","AAL1531","SWA225","N336LA","UAL1611","UAL759","ATN3389","AAL2474","N880MS","N3330X","AMF8011","N43722","DAL795","FFT1004","TWY411","N158M","RPA5590","AAL2716","SKW5515","SWA994","MXY540","AAL1259","WJA1045","N739QH","AAL354","DAL2216","SKW5273","CXK1055","N984MC","GJS4193","EVA032","EJA950","N460WS","N96UA","SKW3809","SKW6194","N525GL","SKW4708","ASP655","N981CD","EJA261","N1540","N425WT","VTM342","SWA367","N1MU","UAL300","NKS788","AAL3301","DAL1006","SKW3729","ENY3831","CXK494","VOI7811","UPS2325","FDX1650","DAL1486","AAL1782","N728QL","UAL1400","WIA362","N469ER","WAL770","LXJ501","N202EA","FFT4139","VJA102","AAL1476","FDX1411","SWA116","ATN1917","CNS914","N704SX","DAL195","UAL1709","SKW4637","FFT1807","ASA420","N3033M","AAL1770","SWA1580","DAL1990","AAL2664","EJA259","N441F","N514FT","SGX998","EDV5047","MTN8459","ERU815","ENY3386","LTA446","SWA4989","SWA3751","N560PU","JBU1055","BBQ8263","DAL743","SWA1181","AAY2315","UAL1669","SWA3157","SKW3837","JIA9664","N3233G","ASH6314","DAL1132","UPS948","UAL2618","N998GC","AAL2409","BMJ48","SWA2696","N812GH","EDW36W","N186TW","CAL5115","UAL1432","SNC8104","X774S","ABX3111","AAL1325","JBU1770","ROU1873","SWA2693","AAL2459","RPA5703","DAL504","N2037A","APC997","AAY2931","MRA636","N48900","ELY001","AFR178","N357FT","N9SC","LXJ403","N59809","AAL843","PWA127","N450MB","N744QS","AAL1391","LXJ361","ROU1627","SWA1918","N604MD","VOI5506","XBGFA","AAY2598","FFT2397","SWA938","SWA1031","SJJ945","SWA500","N53794","N921MT","NKS660","N3733T","AAL1083","UAL1886","APZ111","VIV5009","N445VL","N777WM","SWA2747","N700AD","SKW5715","AAL580","RPA4421","PTR2131","DJT100","NDU76","WJA2210","POE513","CMP803","SWG622","SWA993","VTE3853","ENY3796","DAL2547","JBU2354","CXK189","LXJ360","N644KM","N727DP","JBU240","SWA3865","SCM40","DAL329","EJA324","AWI6116","EJA552","UPS5943","JIA5335","AAL2429","JBU215","N34AH","UAL501","VOI1773","ENY3990","SWA4415","RKK36","RPA5800","N777TP","FFT1129","VOI1761","LXJ98","N172TS","NKS1328","AAL2184","N30SS","MTN8310","GXA6114","AIP1563","AAL1536","DAL2474","N359VF","FFT1165","N97TE","N17MM","N130EC","TWY2000","AAL1644","PDT5740","UAL1363","JBU80","ENY4044","AAL9827","SKW3328","SWA1672","AAL17","DAL1907","N902AG","JSX855","AAL72","SWG4429","RPA5652","DCM4241","N711JG","NKS1770","UAL187","JIA9968","FDY405","SKW4957","SWA779","N8138S","SKW5568","ASA173","UPS4777","N5WN","N1555M","AVA149","SWG4474","NKS1963","POE526","FFT2209","SWA1229","N722BD","VTM292","N982FH","UPS5585","RPA4708","SKW4988","FTN6","N312NC","ASH6224","N3827R","CLX61A","RPA3554","EDV5287","ANA113","UAL830","BWA414","AAL2385","N51VT","SWA1400","AAL1988","UAL2008","UAL996","RPA3561","AAL2430","SIL0122","NKS603","SWA3888","WIA304","UPS988","HAL4","SWA1874","JIA5218","DAL2017","EJA547","CCQ717","N410HP","DAL2380","SWA1651","AAY161","N921PP","XSR718","ENY3419","EDV5005","JBU2020","CXE502","SWA449","UAL2108","BWA601","N930FR","SWA1111","UAL1923","EDV5526","VTE3934","VXP677","DAL1015","AAL348","BTQ351","SNC385","Z106BC","N315EN","AAL2222","DAL9890","RPA3629","RPA3422","N172MX","N820JH","N91HG","AAL877","UAL2418","AAL984","AAL2880","NKS1649","RPA5757","AAY25","JBU1568","ACA1289","SCX548","JZA938","DAL73","ASA194","SWA691","N681LE","N316PJ","FFT4384","DAL1780","NOJ23","WUP453","RPA4547","MAL7051","SKW3405","CFG2116","EJA739","ACA1274","JBU100","RPA3474","SWA909","DAL1455","AAL2890","EDV4912","AAL1194","PTR2940","JBU2887","UAL487","AAL1958","N858MG","FDX1406","ASA380","LTA904","LN850DP","JIA5025","ENY4071","SKW6500","JZA93","UAL2075","UAL1763","XALMV","SWA1851","SWG770","N10RM","FDX1576","N461PS","UPS778","SWA546","SKW5216","FRG1550","RPA5776","AAL1972","SIA21","ENY3861","ENY3806","RPA4639","DAL1669","SKW6310","EDV4817","BAW9CG","JBU436","LXJ328","SWA3571","SVL3","N934CT","SWA3671","POE559","SD6","ZN138BC","J1","FDX5047","N352TS","SWA28","N623PM","AAL2887","UAL2200","THY5","AMF1301","EJA416","SKW5717","TAI450","AAL1761","UAL471","AAL60","N401RG","WAL200","N7561X","EDV4989","FFT1085","QXE2360","JBU1351","N990GR","FFT2684","N54TZ","AAL1089","FFT2621","RPA5736","DAL1678","AAY1869","DAL1112","AWI6162","VTM558","EDV5340","BXR1984","UPS1356","SWA961","DAL1674","DAL2506","N102WY","PSC582","SKW5945","JRE830","SKW3689","N770BB","JBU109","JBU551","AAL3167","UAL853","SKX5590","VOI780","RPA4450","UAL2672","AAL2485","N154EF","QXE2160","ASA1438","CXK377","SWA5304","ENY3933","ENY3785","N9635M","EJM240","NKS908","FFT1596","LN701CF","DAL570","UPS652","FFT1611","FDX258","LTA940","N126SK","EDV5255","AAY1396","DAL920","SWA2616","AAL1692","ASA867","JIA5085","N1541H","FDX1382","EDV5230","WIA406","EDV5064","JIA5456","DAL225","ACA785","RPA3702","RPA4626","SKW6390","VXP395","SKW6263","N72100","N462BM","UAL2680","SWA6461","SWA873","AAL2985","UAL2049","CXK466","ENY3662","AAL2971","JBU2732","UAL2214","STB2171","JZA690","FDX1869","ATN3321","N2863W","DAL1259","UPS814","N214MN","JBU2181","UAL1232","PDU72","JBU403","UAL2607","N15CV","EDV4867","N92908","SWA1077","AAL2797","AIP252","N1472W","SWA3199","N229AT","N258GC","ZN6ZG","ASA1171","SWA1998","AAL668","UCA4812","AVA211","FFT3215","UAL31","SWA2190","SWA454","SWA3559","PDU71","DAL1152","NDU40","SWA361","N9353P","N4QL","VOI1710","ENY4080","SWA358","WJA1360","KFB699","AAL1032","ASA1289","UAL1306","RPA4590","SWA2961","PXG103","DAL1726","N145HM","SWA6515","IBE0337","UAL2167","SKW5985","AAL383","SWA3131","AAL1208","HER976","FFT1474","EDV4824","RPA4754","SWA4615","CGIIT","UAL1966","N910JW","EJA677","ASA204","JZA540","RPA5660","ATN3351","QXE2246","N929CV","ARR689","LTA926","SWA1191","KES301B","CXK475","RCA828","N4856T","SWA1332","N661WD","EJA241","UAL1501","SKW5319","AAL3162","AAY708","N512KR","JZA541","SWA2244","DAL2039","LXJ437","VOI751","SKQ91","UPS672","AAL1449","FFT1425","AAL1503","BWA520","UAL2423","AAL489","BOX303","BAW7NY","N892AB","ROU1635","DAL3055","DAL64","N162PB","N3210Y","VXP542","EDV4977","N6950W","LXJ389","JZA872","N714QW","SWA4814","N300JK","N678EC","SWA2064","EJM71","SCX1675","TPA4014","SWA3056","AAL2605","AAR286","ABX3121","DAL835","SIL0128","BMJ1810","SWA2106","AAL2502","N54479","UPS108","ASA304","NKS1616","AAL2435","N9996U","AAL2406","UAL647","CMP382","EJA235","ASA1152","AMF1992","AAL3035","ENY3371","EJA921","LN724HF","DAL400","QXE2059","SWA1657","NKS9154","SWA2091","EDV4966","JBU1418","ASP827","N845DA","UJC13","JBU2110","NKS335","TSC158","AAL1751","N1256P","SWA2930","FFT3363","VTE3714","AAL1215","SCX745","N577RT","DAL534","NKS275","JSX302","SWA3556","CAY2909","UAL1695","SKW312Y","KAL092","N515AM","ASA1023","N30VP","TSC327","AAL1057","DAL1095","N595LL","UAL1276","EJA949","SKW3399","N4273Y","SWA1106","N17YP","UAL2853","AAL3099","WJA1232","ENY4251","FDX1497","AJT1040","N30DF","SVA020","N585QS","ASH6109","FDX1585","UPS9725","NKS192","JBU858","AAL2678","OUA27","UAL2269","KOW955","SWA661","SKW114T","AAL1793","DAL2470","N287CS","FDX1407","N78249","XSR331","FFT1106","LXJ95","DAL1349","AAL2152","AAL503","DAL349","PDT5972","JBU715","SKW6417","JIA5543","SWG6451","JBU201","CXA849","SKW5019","N541QS","PBR155","N731GA","SWA1404","N601QS","N133RF","SWA501","DAL569","AAL537","ASI230","VWA118","UAL2154","N7707X","UAL261","DAL2066","EJA554","JBU1691","DAL1002","DAL698","SWA1222","FFT4000","SWA4309","RPA5787","UAL2472","POE662","UAL2058","SKW4058","RPA5824","N95RS","SKW4105","QTR8R","NKS611","NKS555","UPS32","N143RE","JIA5511","N214JT","SD2","SKW3804","SWA2602","SWA499","N595CB","N80283","RPA4493","SKW4714","EDV5280","SWA1634","N910TK","FFT9770","NKS1993","AAL1957","ENY3759","CGHCE","UAL1415","N331HA","UAL2143","HAL15","CJT583","UAL1833","N75TX","AFR24U","WUP314","BHS229","OAE3052","AMF1302","SWG519","EDV5177","N621TG","DAL1582","N80PM","N5215E","SWA2218","WJA1863","BVN8751","N992CE","N777NT","N2PG","AAL2040","EVA619","SWA3128","AMX658","BQB6","RPA4451","FFT1684","UCA4955","EJA904","SKW3468","N7TK","N21FE","SWA1204","DAL1698","N720LU","SKW6189","RPA4385","UAL114","ZVES8VL","SWA5907","JBU621","N951NB","POE510","N4687K","AMF1305","UAL1368","N3073G","JIA5528","LXJ476","N500JH","N926JK","WUP855","SKW5678","VIV731","SWA183","ASA172","ABX397","SWA1590","N12940","XARAA","UCA4910","JBU382","UAL2444","PDT5835","N998BM","JBU1145","FTO388","N4125K","N6882M","N38AD","SWA1837","SKW5602","EJM81","SWA554","N3308C","AAL2473","N218LG","AAL886","EVA027","ACA131","DAL2465","DAL2326","CES583","UAL324","N2745P","JBU952","AAL2414","RPA5873","N868EM","FFT3862","UAL593","JBU1194","ENY3544","SKW5234","EJA537","EDV5095","POE536","FYG712","ASA957","DAL2596","SWA505","N45JG","AAL1669","N414EH","UAL1450","N5193S","SKW3953","LFA533","DAL435","SKW3320","EJM333","QXE2141","FFT2675","N300VJ","UAL649","FFT1872","JBU1157","N393BZ","AAL3002","AAY1530","N460AA","SWA1477","EJA542","N54HG","UAL2642","AAL2044","CKK221","FRG9699","EJA394","DAL1676","NKS309","AAL51","AAY1988","JZA949","UAL1817","BVN8575","QXE2095","JIA5444","UAL1888","DLH460","AAL2721","SWA851","N910AB","BOE601","ENY3756","UPS5903","CJT921","LTA943","EJA474","DAL2829","RPA5697","N429RM","AVA030","FDX1488","N569ER","EJM401","N719TH","SWA1848","RPA4448","SKW3369","NKS1618","SWA737","JBU125","SWA2417","DAL934","N29MW","AIC105","DAL1460","UAL1821","JBU2152","JIA5537","AAL1719","EJM41","SWA2791","KII2023","N257DV","NKS813","AAL2996","DAL3103","ENY3875","SWA1433","SWA733","RPA4682","EJA395","JBU1775","N263RY","FDX1444","SKW3756","AAL999","RPA3524","SWA2240","SWA3382","DHK369","N707PC","DAL1166","UPS7831","N616DA","SWA4288","SWA1378","N904SH","ENY3843","N616EM","FDX67","N868XL","ROU1653","N121HC","AAY2975","RPA3419","UPS9824","DAL761","FDX77","WJA2108","N484HB","UPS620","AAL1807","JBU41","N95TD","N5CM","TSC609","N759QS","N350GA","N809CV","N397ME","BHS382","JBU1410","N540LF","DAL1581","UAL1343","AAL792R","SWA1030","ENY3445","SKW6213","WST603","JBU667","SKW6431","SKW6330","ATN3217","NKS227","FWK915","SKW4229","ASA216","ACA944","QTR29Y","UAL1024","SWA1109","N981CE","N451SJ","N530PM","DAL857","N820UT","EJA471","SKW5119","LXJ546","SIS69","SWA331","UAL2477","SKW5017","CXK405","N91W","SKW3510","LTA351","EJA419","AAL1282","UAL3918","AAL3316","ENY3959","COL202","SWA1715","JIA5040","N5055A","TSC341","N691SC","AAL3194","DAL2915","EJA705","UAL1500","AAL1708","N48JM","N711BH","SKW5226","UAL332","N60US","N548PR","ACA920","SWA1778","KAL251","VJA538","UPS1326","SWA575","SWA1758","LBQ641","MXY247","SCX527","NKS1659","AAL525","N237VP","UAL1799","N717TV","SKW4106","EDV5008","CNS577","RPA3627","N742TX","N300B","DAL346","N87FL","SKW4663","UAL2601","DAL359","FTD503","AAL3083","UAL575","SWA1347","DAL547","ACA107","NKS2965","N4144K","SWG6829","DAL1629","SWA269","SD1","UAL551","KAL017","LN348PC","SWA3216","N617CB","N412DJ","WSN9","N930AK","DAL1389","AAL117","SKW6423","VJA668","NKS1381","N1224T","N118RF","N804SW","SWA3602","FFT1500","N98CH","RJE38","FDX140","UAL1996","TCN118","EDV5366","RPA5608","UAL545","N2336V","ENY4054","N81TG","SKW5536","SCU18","NKS797","ABX199","HRT246","SWA571","CFS8511","RPA5833","EJA376","GTI3617","SKW5724","JRE715","SWA5906","N22CP","LOT6545","HAL7","SWA1257","JTZ419","AAL1567","CTP881","AAL1704","AAY1438","SWA2447","EJA538","JBU1936","XEN20","ENY3389","DAL1547","AAL3055","AAL581","N722SR","WAL702","N871RV","ASH6148","CXK409","FFT4455","N203AP","N7CS","PDT5979","ACA551","UAL2432","EJA486","AAL2552","SKW3792","HAL65","UAL339","WMA1534","EDV5046","SWA887","FFT3606","UAL459","GJS4477","N6337X","DAL897","UAL335","ENY3625","N627AA","RPA4659","DAL2314","JBU782","ENY3341","DAL2711","TTT541","JBU2842","UAL791","EJA351","FFT180","AAL2478","UPS5328","PDT5702","UAL1380","N254DV","UAL2017","N939AM","SWA1705","ASA645","N906AJ","N910TR","N8ER","ENY4073","DAL2542","AAL2975","N2557S","DAL2401","BVN8752","UAL1424","SWA4182","N737FJ","RPA4352","SWA4235","IWY949","N518AR","BHS315","LXJ524","AMX411","VIV112","WJA1770","JTL280","UAL486","SWA6178","N305KN","NKS814","STB2141","AAL2766","DAL2947","FDX130","VJA643","PCM8706","SKW811R","N918TA","N490KM","N144MH","UAL2","AAY1434","ATN3424","AAY1061","GJS4446","DAL2698","SKW5588","TSC498","AAL1154","SWA5309","SWA1371","N115BR","ACA592","WST900","SWA492","LBQ791","RPA5829","AIP1848","AAY2885","AAY1328","DAL440","CGMLN","BAW196E","RPA4400","WML74","N807QS","SWA1912","SWA704","EJA473","VJA334","AAL3305","XAFZI","RPA3700","SKW5801","N288AF","N902TM","AAL1369","UAL1535","N46211","QTR73F","UPS542","CAY102","SWA2683","SWA1164","QXE2038","EDV4955","AAL3110","SWA1833","CYO448","AAL2964","UAL1778","EJA563","N1AG","EJM75","UPS682","AAL2392","SWA1425","N901MA","LXJ424","AAL1261","PDT5750","EDV5357","N765BW","LPE2469","UAL838","AIP2597","N6029Q","FTN2","SCX571","KAP88","UPS5213","AAL2172","UAL1059","N845UP","ASA408","SWA1818","EDV5186","SWA471","APC690","AWI6081","NN113RF","XSR960","AAY712","SWA3628","EDV4946","FFT1141","ATN3388","N510PT","N35VU","DAL594","SWA2061","CNS201","SWA3278","ATN3215","ASP609","SWA818","UAL2366","SWA1552","UAL2622","SWA1927","N558R","XARTS","HI1088","N98PR","FSY127","N668Z","SKW4100","JBU185","UAL1354","N749FE","N760GP","FFT1415","VXP396","CAY205","SWA738","AAL136","N242EX","NKS1008","AAL2636","SWA4715","EDV5351","AAY3023","N577K","AAL524P","DAL185","WST702","ASA65","AAL2881","N3022U","N3664Y","N91GL","N83KM","VTE011","SWA1001","HRF154","N8514C","AAL2304","UPS1478","UAL574","JIA5332","DAL1399","JBU2063","MXY708","FFT3001","EJA132","EJA548","LXJ455","PAL103","SKW4050","N1220W","FFT4113","PCM8710","AAL2390","FDX1774","AAL1820","SWA3916","NKS1775","BHS427","N268RB","EDV5072","UAL149","SWA1162","SNC1201","N224DE","SKW5136","FTN9","N6579Y","UAL693","SWA1158","N828QS","SWG5407","N876UC","DAL2538","ASA34","SWA951","CMP408","WJA2051","AAL1470","DAL1854","SWA3377","LXJ663","UAL905","N829CB","SCX8774","FLE2613","DAL1108","SKW5367","PDT5980","AEA097","CJT2250","DAL1198","UAL1241","N55182","RPA3491","N788TS","SWA2357","DAL1773","UAL632","CGCGR","N300SC","EIN6PE","N373EF","AWI6087","DAL1184","SKW5937","EDV5207","UAL1101","AAL615","UAL1677","N312F","CXK503","JBU2065","CXK418","JJ03","N2702S","N8105R","UPS612","FDX1707","UAL414","OCN978","FDX1545","JBU311","CXK623","DAL562","N118ES","CXK335","UAL779","JIA5347","BMJ1806","DAL2798","SKW3333","SFY430","CJT990","DAL2863","N97SU","ROU1994","AAR203","AAL3071","SLH384","AAL2359","UPS62","N318JH","AAL1839","EDV5149","DAL1384","RPA3679","UAL883","ENY4075","DAL874","DLH5Y","SWA482","FDY152","LXJ352","UPS1232","OPN99","DAL1077","ATN3467","AAY1379","ACA945","UAL550","GTI038","ERU807","JBU1062","UAL1335","SWA1992","AAY3780","CKS368","NKS172","EJM529","AAL808","JBU1169","DAL2019","N7412P","N281CE","N886LF","SKW3905","N12XN","SWA574","NKS540","SWA3211","HI1087","XAXAX","AAL737","SKW3771","WST701","N997WC","JSX206","NKS1646","N327PD","MXY525","N636NN","N859WE","N9568L","SWA6406","UPS5049","SWA3282","N122E","DAL1571","SWA2269","JKR85","TRF534","EJA206","SWA1936","AAY565","UPS5541","AAL2211","AAL1907","N58DH","SKW4969","N50HN","AMF1570","DAL1129","NKS869","JIA5406","FDX5038","DAL2363","AAL1765","ATN3311","N640SC","HAL1637","N560LS","SCM7","JBU2577","GJS4426","LYM515","EJA127","XFL409","ATN4323","N488PC","YEL12","AAL1382","AAL604","N72SM","N5A","MTU98","N560CX","SKW4915","KII574","SWA1571","AAL3200","N561MA","SWA844","SKW6211","DAL3057","LN400SL","N555GQ","EJA424","EJA379","ZN880RC","DAL341","N525TM","SKW5405","N12WH","X603CR","RPA4678","N887SW","LXJ606","WJA2288","ACA1003","SWA550","N695ST","NKS258","N8EP","JBU2164","N888FG","CXK334","BMJ72","FDY227","DAL1462","EJA516","GTI3721","N6444V","SWA494","SKW3396","CCQ477","UAL1622","N99508","PCM8973","DAL3047","UAL855","N61FA","JSX9115","N712RX","LN90JX","N5160E","ATN3453","SWA1284","AAL1140","N475CB","FDX1448","SKW3691","AAL2039","SWA437","ACA7242","N9885V","SKW5015","SKW4108","ENY4072","SWA3014","SCX569","RPA4516","MRA633","RPA5656","AMX693","AAL2564","SWA2691","N110GD","SCX213","N130HL","AAL2903","AAL2633","AAL2998","LTA663","N207P","UAL1898","BHS230","DAL28","UCA4315","SKW4918","N996FG","PCM8697","AAL2066","RPA3588","ASA1003","QXE2219","CGULF","N236RP","N105J","ASA1372","SKW5213","N250HR","CNS98","IRO8578","NKS1632","UAL1775","LBQ611","TPA4002","FDX108","ASA387","UAL861","DAL910","ENY4098","AIP2601","EJA158","QXE2111","RPA3420","NKS980","FRG1689","AAL2888","SWA1516","RPA4418","SWA441","RPA4594","UAL236","N411V","AAL2303","N655MC","DAL2923","HI1111","FDX1767","DAL1523","N312AR","SWA2667","AAL118","DAL768","ANA110","DCM6317","SWA4890","SKW5339","DAL1212","UPS5963","KAP1972","AAY1827","JRE755","AAL495","SCX384","AAY876","SWG7349","GTI658","UAL1954","JIA9970","KAL081","JSX900","DAL2909","RPA4412","SWA1553","EDV4947","Z880RC","VOI1890","N217DC","JBU159","AAL418","ATN3387","UAL246","FFT3281","UPS572","FDX1476","CGLTG","ASA711","SKW5328","N1646H","AIP1832","AIP2691","JRE735","N60451","N460SJ","UAL489","TFF965","JBU870","N445NG","N80BL","N252EA","LBQ420","N897A","UAL1217","DAL1788","DAL1964","SWA4068","UAL651","DAL2556","BHS316","ASA576","UAL1146","N30HC","CFS8812","ASH6174","SWA277","UAL1095","ASH6364","N921TX","JRE390","SKW3708","AAY1142","DCM6510","AMX036","AAL1289","N135RF","UAE8ER","SKW140C","ATN3345","SWA1877","UAL1860","SKW5130","N312AE","SKW6304","DAL227","N906WA","SWA1654","SWA2410","UAL1600","N212WP","AJT851","BVN8754","SWA1143","N259MM","ENY3547","XAGAT","HER110","N7SN","UAL549","SWA538","SWG601","ACA1245","NKS1697","N57UM","UAL469","UAL563","DAL3001","JBU251","CXK213","AAL1134","JSX370","UAL2268","AAL1197","AAL2978","N359KF","FDX1247","UCA4317","AAR201","N420JG","DAL648","ENY3556","N360SJ","PVL2285","N136SH","UPS5195","FDY404","N2122G","SWA1681","UAL1774","UAL2345","DAE5232","DAL2657","ERU466","IRO8406","SKW3867","GPD438","SWA1589","TSC356","BOE14","DAL2231","UPS5681","ENY3372","EDV5436","GJS4487","UAL1407","ASA605","SWA1701","SWA3249","EJA370","VJT539","UAL1238","EDV5499","SWA382","AAL3178","N217CP","AAL2344","BEL1CP","DAL1479","N360SM","AAL380","UAL1841","N704DR","N462DH","AMF1362","UPS312","ASA1322","GJS4396","AIC103","ASH6242","AAL3012","PAL127","JBU734","SKW5794","SKW4893","UAL1111","N427WA","SKW6538","WIS380","SKW5396","FLC90","DAL3028","JBU1087","ASA305","AAL2538","N822NP","FFT4989","WMA9218","N486WC","N858ME","N138FJ","N441CR","RPA4435","SKW4734","CMP443","UAL2234","PTR2230","SWA2343","UAL609","UAL204","RPA3590","UPS7869","SKW3790","RPA4490","DAL1373","SIL135","UAL1328","UAL1064","UAL1014","DAL1180","UAL1727","DAL2060","N390QS","N410RK","JSX401","ASH6189","N1222G","NKS1695","GJS4444","EDV4939","FFT3355","AAL858Q","N80RP","N411MV","WJA2170","UAL1978","FDX22","N444YG","SWA2896","FDX49","ASA501","EJA503","UAL2384","N426SB","JBU2432","RVJ448","N119DT","AVA255","KOW883","JIA5564","AAL909","AAL1256","JBU1347","UAL1868","SKW4678","N26OFL","AAL2740","N6038B","CMP451","UAL1901","EVA017","ASA7","SWA1413","N38CP","RPA4335","N999MM","JBU671","YEL87","FDX1405","ACA7077","N2027D","UAL1536","RUC2205","SLI2708","VIR163","UAE4M","N250LG","UAL1767","DAL869","DAL820","N111Y","SKW4939","LXJ560","SWG444","AAL28","SWA4026","JBU1952","TIV710","AAL998","JBU608","UAL1771","N590MS","SKW6333","CXK146","PDT5937","JBU2802","PXG600","SWA1678","ENY3904","N4816S","NKS293","QXE2093","SKW4099","WJA1249","LXJ345","N9200Q","N415PR","N478CV","N500NB","SWA2023","DAL550","UAL1670","JBU1621","SWA2525","SKW5994","AAL2382","AWI6086","DAL1869","UAL1741","UPS207","SWA1500","N48816","FSR435","SWA1476","GLH3","UAL283","GLF23","N11NX","UAL1003","SKW6329","SKW3719","LXJ575","N818KF","N79AJ","JBU438","NJE567F","UAL659","AAL2858","SKW4763","FFT3156","FFT3397","SWA2583","ASH6296","FFT2083","N706RT","JZA922","CPA2071","DAL1618","UPS1302","AAL693","DAL2797","N4SD","JIA5489","ENY3572","DAL2165","N476JD","N247SR","SKW6204","EJA520","EJA824","UAL1154","CXK553","JIA5638","SWA2065","N7388R","N963LK","ACA856","EJA581","KII835","AIP1601","SWA3665","EJA954","EJA526","LN223LG","SWA1980","DAL2941","SWA3196","N510FH","JBU2201","UPS1066","UAL573T","ASA1306","JBU1704","UAL595","N399W","DAL2530","FDX1859","TSC219","AJT8511","N4308P","CXK295","N585EJ","UPS592","N320WC","GJS4183","AAL1915","SWA3504","AAL1136","LXJ462","WUP905","JBU397","PDT5995","CSB529","UAL7","RPA3673","AAL2699","DAL1246","SKW4783","N3066G","UAL1036","AAL1461","WST102","SWA1908","FFT3291","LFA453","SWA3695","RPA4414","SWA3365","POE263","SWA2569","UAL764","LXJ514","AVA043","N167HJ","N414M","N652WE","N424U","LN745TX","HPJ367","SWA489","AAL2217","SWA3303","SWA1893","FDX5262","RPA3496","UAL444","DAL1987","WUP302","SWA1793","WJA653","AAL1429","GJS4434","N861AT","N66P","EMD44","ASH6012","N509SB","AAL1960","N179KC","FDX41","N865PS","DAL797","N5257R","DAL1396","UAL276","UAL1625","CPT300","MSR985","FFT4997","NKS209","GXA601","AFR474","N787PM","AAL1686","CXE304","UAL2219","N52HR","UAL1484","N421DC","SJX005","UAL1918","LXJ374","CXK364","SWA1559","GPD113C","UAL1129","N518JF","UPS2911","ASA116","LXJ93","N430SL","VTS2875","N903PS","FDX853","AAL2762","FDX1542","SWA976","SKW4128","N120JJ","UAL2443","N9356H","N2940F","N423JT","N33MM","IWY251","AMX026","AAL2376","JBU1117","HGE2","GTI8196","ACA127","UAL1096","SWA2568","UAL2189","FFT3018","SKW3317","TTE499","AMX645","AAL402","AAY3067","SWA3388","N8126P","IRO8559","DAL1213","N251VA","FFT1650","DAL1845","NKS374","AAL2426","UAL1544","DAL1924","SKW102P","RPA4477","AAL1418","AAL1853","AAY1799","UPS1442","N585PL","ACA1084","SDU244","SWA3217","GTI5528","WJA1940","SKW5870","N64PF","SWA1737","AAL2676","N7250T","UAL1452","N703CX","N8427D","N615QS","AAL756","UAL2048","SWA990","AAL1640","AAL31","SWA968","PTR2296","SWA2271","N696JS","RPA9936","ASA537","CPA807","GJS4539","N755PG","AAL1976","WMA5264","N6780V","SKW6335","RPA5816","CTL19","PCM8684","MTU96","SIL62","MRA605","N9BN","SKW3822","NKS1033","ASH6011","N41492","N711JT","N931GG","UAL255","SKW4794","RPA3432","AAY3782","N674WM","UPS1038","SWA1261","FDX1559","UAL820","DAL794","DAL2023","N129WS","WUP901","SWA880","EDV5316","UAL1227","TSC199","AAL970","AAL2639","UCA4895","ENY3825","ASP565","JBU904","BPX295","N205BN","UAL721","EDV5407","ELY003","CNS1209","SKW3763","LYM4122","AAL2524","FDX1473","GEC8225","UCA4333","N5529W","TIV80","AAL1367","N812TR","IRO8645","DAL513","N963AD","SWA2140","SWA5998","PAL104","UPS837","SWA2880","N38CG","SWA420","SWA4038","DAL1683","N314KD","ASA1279","NKS1758","JIA5375","SWA497","SKW5258","N75WH","SWA2200","RPA4625","N610GG","UAL2225","MTN8120","N307PH","N69CH","FDX1735","JBU1365","SWA2388","KES312Z","JBU2784","ENY3478","ENY3534","N70EW","UAL34","SWA108","RPA4557","JTL42","DAL2104","AAL2275","AMF1948","ASA612","UAL1039","AAL2558","EJA314","EJA924","FFT3216","DAL777","GJS4472","UAL640","ENY3772","SWA1351","FFT181","N12NF","N534CD","DAL962","GPD132B","AAL3046","N866CD","AMF1930","UAL1896","JBU1015","SKW4115","UAL1765","HAL50","SWA2689","N808DF","SCX8267","LAE1822","WSN1","UAL1210","LXJ653","SWA1153","N11DG","SWA4467","HAL29","SWA920","FFT3955","SWA16","UAL1881","AAL2274","DAL656","UAL359","AAL1464","FDX1439","AAY1282","AAL1354","N405PA","DHK051","SWA1860","SKW5467","ENY3592","JBU892","UAL2716","N748JA","EJA811","AAL2","SLI2783","FFT4927","AMF4276","N812GR","ENY4056","BHS138","SWA2205","JBU1318","WJA2521","N568RZ","ROU1520","N6223V","DAL1891","FDX883","UAL2649","NKS842","CAK1395","AAL2173","NKS894","EJM119","JTZ715","N25CX","UAL1714","HAL8","BXR1966","SKW3392","N63HS","AAL2282","WUP886","RPA3597","SVW51XL","JBU2265","N123BM","BAW91C","ENY3848","UAL2046","AMX642","JSX654","FFT4855","GJS4503","RPA5767","NKS1374","SWA2395","GTI3633","JZA960","JSX670","LEX8089","JBU2224","SWA1010","AAL1300","N993FG","SKW4010","PTR2124","SWA363","N6692M","LN897MD","DAL721","SWA380","CPA883","N447RM","ASA1154","N806LR","FLE2811","UPS9309","AAL3146","JBU2254","DAL952","ACA269","JBU2615","N320VC","QXE2452","RPA4597","AAL1875","UPS2344","N16TM","JBU2024","JZA902","N4925K","DAL2417","ACA689","UCA4920","UAL2684","N419ST","DAL1148","RPA3498","N226BG","SWA1194","EDV5114","EJA606","SKW5002","ASA376","DAL1401","JIA5665","AAL358","AAR202","N587G","N32303","SCA28","EDV4835","N180TL","LXJ90","AAL3263","ENY3407","N817SA","N888KR","SWA2195","N215AL","UAL517","UAL1994","SKW4061","FDX1377","N130KA","N679TS","N340AH","JBU152","VXP719","JBU1555","ASA520","JBU1941","DAL564","AAL1080","PDT5875","DAL1136","FDX195","N417FC","JIA5105","N7444T","FDX1558","KQA002","ABX110","AAL1532","AWI9904","N5462X","FFT2635","EDV4813","HAL39","AMX681","SKW4123","J3","N553BW","PDT5738","N7338F","SWA1585","N5LN","VPMNI","MAL7079","SKW6275","GTI8240","LN1273A","FFT2593","SWA2128","WJA2377","ASA474","JSX170","VOI1812","LXJ521","ACA098","FFT4464","QXE2131","N321JJ","BOX472","SWA892","ENY3482","FFT2720","TAI561","UAL1549","N611JW","UAL1849","SWA6598","AAL866","UAL1316","AAL1906","SWA1061","AAL1645","AAL3253","FDY442","RPA3682","N46EH","DAL1410","EJA122","UAL1798","ENY3591","SWA3360","EDV5523","N5568P","SYB163","FFT2146","AAL1346","TTT686","N286CX","SWA1514","DAL1326","DAL1466","N59019","JBU513","N2136Y","JBU1780","FRG897","VIV764","N9145N","N987MK","UAL2264","AAL2746","SWR64E","ZN61011","SWA2379","N201WR","VIV152","N9685U","VOI1801","ASA7006","UPS370","SIL54","SKW5374","N364MA","JBU472","EVA051","DAL819","N851FS","KAL214","EVA015","NKS538","JIA5419","N991TV","UAL2188","N815CE","SWA562","SWA2457","DAL532","DASSY","AIP9","UAL499","JBU641","RPA4388","JBU841","APC2001","N529NG","SKW3735","N827RJ","VJA343","AAL692","SWA1076","SKW3648","N325GH","FFT2448","FFT3546","RPA4622","LPE2480","JZA482","N223JF","DAL800","AAY969","UAL1720","EDV5275","UAL2466","FFT1554","N140PP","N7LA","N933EY","FDX1621","VJA02","ASH6134","RPA4654","UPS946","SWG6404","AAL2021","N612PR","RPA4381","DAL1103","ENY3374","N17JK","SWA2944","REE410","SWG6347","ENY3689","SCM3","NDU82","RPA5607","N2690T","EDV4909","N655MW","TSC986","VIV675","KAL094","N370RV","SWA450","PCM8749","UAL737","UAL1913","EDV5418","AXL869","KII830","UAL2635","UAL592","JSX173","UAL1823","ENY3512","N2112","TWY381","SWA3448","AAL1604","TAI522","DAL658","JBU947","DAL1444","TAI562","AAL972","AAL307","N55WP","N12679","AAL1124","AAL680","SWA527","SWA1172","N2326B","JZA982","NKS3930","AAL2313","DAL2049","N555DF","ENY4009","N277GL","LAE2534","CVU28","LXJ660","ASA1323","ZKP4522","AAL369","JBU2431","UAL1184","DAL734","N822BD","N958PB","UPS916","TFF997","ASA1077","SKW6392","NDU51","N3007B","N85MX","N313AR","AAY1206","N642DP","SKW6311","UAL1939","DAL2752","AAL3130","JBU2596","DAL49","N229FG","JBU1144","PXG453","SWA3611","DAL3091","WCP254","DAL2178","N399WS","SKW3314","RPA4377","UAL1685","N707WL","MTN8300","SWA775","GXA6112","SKW4885","UAL1867","FFL808","SWA915","UAL2010","N610L","N502RP","NKS818","SWA1225","N12MG","PTR2128","KOW910","SKW5936","NKS801","SWA1073","SIA7404","SKW3307","ENY3409","N595TG","AZU8706","N6243C","ASA1290","WUP325","N340N","N8910P","AWI6153","N891L","SCX3019","CKS817","EJA564","N101NY","N4639R","DAL943","N370AF","NKS285","SWA2815","SKW5131","SWA874","AAL2366","JBU881","UAL1053","EJA383","WGN4875","VPCEJ","EJA231","PAL102","FDX1580","DAL166","VJT741","AWI6065","UAL412","SWA2958","N167AR","ALZ40","SKW131A","EJA756","UPS197","ENY3489","N940TW","SWA925","DAL1298","ASA643","N2921Q","AAL2273","ASA1117","DAL1730","DAL1318","DAL2258","JBU2655","N2804W","ENY3715","UAL793","AAL1652","VOI7773","AAL2297","ASI213","DAL1958","SKW6414","QXE2331","SCX8012","JBU269","SLM482","LN535LL","SKW5882","UAL2064","DAL1938","DAL8838","N734FM","UAL1556","WWI72","NKS532","AAL1633","LN610TA","AAL339","DAL2147","N7303P","N217LJ","N387MM","N313TR","SWA133","UAL785","EJA873","AAL3179","CXK626","NKS245","EDV4901","UPS842","UAL806","FFT2667","N453BB","SWA8504","EJA837","SKW4649","N76PW","SKW5491","UAL1732","AAL1564","DAL2118","SKW4078","N156WL","SWA2917","N470SE","AAL2471","AFR6722","IRO712","DAL2946","SCX3067","CPA884","UAL1442","N441CC","JBU1186","UAL1903","ROU1883","JIA5193","SWA1930","N23EW","SWG780","FCE72","ASH6203","CSQ951","AAL3097","DLH519","LN911VM","RPA5599","CSQ211","N969RS","EDV5217","UPS9796","UAL1584","LXJ331","N422ML","EJA566","FDX2","N743DS","ACA738","LXJ458","ASA929","AIC180","DAL2888","SWA3022","DAL1787","UAL1385","NKS1364","SKW4751","AAL2378","JBU2677","N1989B","SWA4265","N301DL","N16YE","DAL2886","SFY428","N53703","SWG627","FFT2082","CMP360","FDX1487","CAL011","NKS874","DAL2503","SWA1233","N196SB","N777ZL","AIP828","TAP223","N403TB","N55LP","TAI392","NDU79","N36829","JSX301","N6149N","VOI7760","DAL1261","UAL2207","N1919","ACA1326","SKW4943","VIV521","SKW4657","JAL6","NKS276","N3318V","N2745G","N54RE","ENY3496","N821BD","UAL1158","KAL018","AAL1285","ATN3383","N365QS","N518AF","NKS1636","ENY3954","SCW4717","UPS104","DAL922","DAL2557","NKS932","SWA1096","UAL2493","EDV4949","AAL1841","FDX1521","SKW4640","KAL041","ENY3657","UPS7821","TSC623","N336MD","FDX1700","N633MG","YEL83","N28RK","FDX1498","NKS516","N1517U","AMX652","SKW3721","ENY3470","AAL167","UAL1124","NKS929","DAL1717","PDT5793","N906MB","CNS521","N344LC","N462PC","JBU503","N21692","EJA773","DAL924","AAL315","SWA483","UPS5191","SKW5078","PDT5734","N351AF","SWA1314","CFS8790","SKW4964","DAL2719","N401KM","FDX1588","LXJ445","EJA647","SWA2052","SKW5820","VOI1748","SWG9990","GJS4506","JBU127","AAL3082","X127BR","N649T","JAL11","SCX415","SWA1845","N333JW","TGCCH","N1444S","UCA4875","N3853J","AAL1415","WST901","EJA805","BBQ9840","VIV547","JIA5231","N920AC","MXY254","SKW4926","DAL791","GTI7832","EJA916","SWA1","AAL2087","N310JP","UAL356","LXJ482","FFT3905","COL919","RPA5772","ASH6094","DAL68","DAL1176","N827PA","SCW3823","JTZ814","BHS320","N500TH","SWA1482","AAY513","N897T","RPA3435","N9WW","ASA892","UAL1472","N9120P","AAL804","N601QA","N797SP","ASH6199","N2765W","AVA036","UPS5591","SKW4950","UPS2983","J7","SWA1063","UAL2398","AAL675","ASA1032","SWA1485","JIA5274","TSU7567","CYC56","N321HX","MTN8509","AVA216","GJS4392","AVA246","EJA479","GXA1620","SWA8505","FDX1442","UAL1353","BVN8793","N521TM","LYM5407","N7241B","UAL1291","CXK481","N27KY","DAL1137","DAL127","N13ER","N550MS","RPA3584","AAL1180","SWA4560","N100YB","RPA5828","XADSK","SWA3949","NKS1643","BXR8603","N937BB","AVA009","N703RK","AAL3143","NKS1380","DLH453","AAL1065","NKS895","ERH328","SKW5617","CVU88","SKW5071","N1049J","HAL35","N733SH","JBU1406","UAL1061","FBU85A","CFHKO","JIA5198","UAL2287","SKW3121","N2JJ","UAL1145","JBU1462","SWA4201","SWA1783","IRO8587","RPA3493","N529LL","N590CL","N951P","UPS869","N655PG","UAL1883","UCA4844","KAP1223","JBU35","N739NL","ENY3509","SWA362","N54810","SKW4057","ACA954","MXY736","N62745","DAL650","N276EB","JBU273","N50GP","ERY74","FXC33","N1471R","N280RM","FDY392","VWA212","SHH3913","CGBLJ","DAL1801","JIA5367","N6542C","AAL652","RPA4409","AAL3279","AAL2161","CMP253","ECJ80","RPA3503","SKW3641","EDV5333","AWI6148","SKW5054","LXJ451","N26DP","FFT1482","UAL2850","CXK562","N755TS","PDU63","SKW4742","ASA866","N899ME","SWA1003","RPA4426","SWA1195","UPS1188","N414KW","AAY2651","KAP912","ENY4254","SWA2122","N379HP","SCX1629","CPA826","TWY98","N2447B","N147Z","AAL3091","AAL2016","N508AC","ENY4012","DAL1038","TSU8122","EDV5208","CFFRY","AMF1906","ASA1320","AMF1902","EJA472","N44995","SKW175C","FFT2677","AAL453","SKW5375","UAL1256","N786SC","JTL936","JBU766","DAL1565","JSX854","N231CE","EDV5116","ACA941","DAL586","UAL1619","DAL1073","PJC22","PTR2280","SKW5082","LXJ467","SWA3059","DAL1590","LYM118","DAL901","N600BW","AAL2919","XSR450","NKS1447","JBU1320","CPA881","UPS5321","LBQ870","N506PG","TSU811","PRY01","DAL772","WEJ57","PDT5990","AAL1119","N737HY","SWA939","N53FJ","UCA4958","N898TX","LN617BG","GXA6160","UAL1645","DAL935","UAL100","LXJ500","N131EA","SKW4983","SWA3475","IRO8552","N252RH","ACA7260","MVK79","N584R","DAL2695","DAL912","LYM5872","AAL1328","N95SJ","FFT3312","EJA446","AAL3016","UPS987","GTI7803","SLH560","SKW5034","N4378Q","UPS208","JBU405","UAL2409","AAL786","SWA1754","UPS922","N900CC","EAL9112","N733RC","NKS340","FDX1901","SWA2907","FFT3282","N415LE","NCA101","JSX500","SKW3488","VXP932","SWA807","AAL880","JZA874","FFT3302","N871DG","DAL425","RKJ868","N523DB","SBU42","N850UW","UPS1340","EJA253","VTM940","UAL2153","SKW5355","AWI2901","CKS818","N477GJ","RZO247","SWA1734","N9241L","AAL2898","N631BA","ASH6222","SWA2326","SWA2642","N796TH","DAL1826","IRO8334","NKS401","DAL540","N905WR","N651CV","N5379F","AZY100","AAL401","ASH6050","FFT4436","JBU371","QXE2084","LNE511","DAL354","AAL3018","ACA1023","N6422J","N24XZ","N353LS","N534CB","DAL2158","AAL1614","UCA4288","SWA343","SKW5618","UAL1235","UPS390","SWA4811","N510FA","N943CB","N43KA","UPS1382","SKW5254","AAL895","N7MW","UAL2403","UAL1323","AAL711","UAL1788","JIA5422","N6851E","N67971","AAL3246","JSX470","N302VM","EJA793","AAL2962","DAL2722","FFT1542","N711HT","N994MZ","N624CM","AAL1870","SWA3300","AAL1703","N98NF","WJA1853","DAL2142","EDV5321","N15TZ","DAL2440","SKW186E","N258JG","AAL2136","SKW499U","AAL2187","SD12","N4639Q","SWA2283","N929FE","SWA3445","UAL2242","JRE852","CXK153","AAL1249","SWA1446","AAY50","N600CL","DAL2077","SWA4316","N84WC","AAL2058","ASI5965","FLE2665","N20932","N724RK","DAL1967","SWA1372","ROU1726","UPS1058","JIA5405","N466F","EDV4851","N51ND","NJM1933","UAL1912","AAY851","JIA5354","GXA3392","ENY3890","AMX623","UAL1075","N414NC","SWA1904","CXK467","DHR4","DAL888","ASA312","SWA743","N318JG","LXJ484","N119TP","DAL1029","LTG8440","AMF1970","N2244F","AMF7170","EDV5269","DAL2328","PCM8662","AAL1979","N4376A","N723JM","CXK482","N18HJ","AAL2793","AAL1587","UAL1460","LN207P","KAL093","N96UT","ENY3931","JME613G","N371JL","N85KM","AAL683","DAL1146","UAL2051","N201CE","DAL737","AAL324","VXP229","CNS182","JBU1480","SWA3468","ENY4019","RPA5858","WST800","N850TJ","N324SL","N121TR","AAL493","UAL1786","ILU49G","TSC538","AAL851","N840SE","DAL306","UAL708","JBU1802","LTA944","ASA675","CXK173","AAL1150","ROU1844","DAL118","ASH6070","N513AC","N250LH","MXY6348","SKW4661","EDV5051","DAL2554","ASA442","N698PM","JIA5204","DAL1917","FDX90","SKW5449","XFL511","N8133Z","NKS1280","UAL2050","NKS551","N72NE","AAL2215","N60GG","N224GP","UAL29","UAL2005","N299HP","N422WD","RPA4372","MTN7102","SCX3022","AAL2600","UAL2722","SKW4659","AAL879","SWA1182","HAL31","NKS574","WUP450","UAL97","N550J","VXP533","UAL1116","N672AR","SWA421","SKW5898","UPS796","EDV5510","RPA4631","N591GT","ASA1340","CKS697","FDX1889","DAL1493","SCX556","LXJ587","JBU2163","N422EM","FFT3796","SKW602X","MTN8105","SWA1640","AAL2932","JBU815","N76ZA","UAL1459","N56NG","N700TX","DAL226","UAL2498","JBU51","SKW3456","NKS1242","ATN4333","JBU648","BHS141","SWA1712","JZA381","PDT5767","FFT1876","AAL2027","CXK274","TOM064","N646AF","RPA3706","SKW5532","SKW5284","TSU069","CJT949","AAL2739","UAL560","NKS1765","N850GV","SWA6001","SWG774","RPA3680","N333W","AAL2883","AAL2342","AAL1485","N2974N","UPS853","JZA783","DAL1177","N350GD","N226B","SWA2066","EJA217","N875JT","SWA2062","UAL2083","RPA3658","N972TP","UPS9733","SWA791","JBU200","AAL2277","UAL2468","FFT1820","RVJ412","SWA2795","UAL278","EJA819","UAL1659","FFT3137","UAL1599","GJS4553","DAL2294","AAL734","AAL702","IBE0333","OME77","WIS700","N57810","FDX1474","ASA9","SWA509","NKS890","N8772R","JBU111","CNS568","UAL2062","SWA3896","AAL1528","N874CP","UAL2267","SWA4249","JIA5591","SWA1015","KPO444","SWA761","SKW6451","UAL572","SCX3015","AAL1562","UPS768","AAL1271","SWA202","ENY3514","EAU31E","NKS210","JRE875","N801SW","JBU1433","UAL1783","PDT5811","N230DF","JTL15","ENY3542","EDV4869","EJA835","T7MLT","SWA2563","FFT4611","MXY330","FDX1418","JBU2537","THY11","UAL2643","DAL417","SWA1556","ASH6248","QXE2270","N22336","SWA696","SWA1157","SWG3513","MXY774","N19PP","FDX1445","EJA423","SKW5065","SWA1982","SWA383","AAY444","AAL2151","HAL47","GPD122B","XSM26","JIA5461","SWA4308","AMX441","LXJ540","RPA5740","ASI70","SWA1373","UAL2041","ASA326","N2051U","ACA591","RPA5654","NKS213","N209MG","N143RS","UAL604","DAL2952","RPA3438","N230UH","SKW4700","UPS5285","JBU2141","DAL1054","N8430B","AAL1511","GXA1623","SWA1471","JBU607","SKW864W","NKS345","N71753","N2734U","ENY3582","BAW4LV","VJA356","N460PA","EJM328","LXJ5599","UPS2190","ASA1177","TTT602","SKW5382","EDV5414","AAL2855","VOI7704","UPS371","SWA4670","N311WM","SWA5184","N925DC","AAL2053","AAL2086","DAL2292","SKW3778","NKS1353","FFT1966","BTQ861","N191CZ","JRE833","UAL1008","ENY4011","AAL1707","SWG764","LXJ664","WJA4120","JBU1384","UPS909","SCX214","SKW6461","SKW5497","DAL2882","UAL2171","N38TV","EJA669","N317AF","RPA4442","N585JC","CJT951","FJI891","LXJ554","EJA309","UAL1301","N323HA","DAL135","MXY6354","AAL134","LTA855","ENY3676","LXJ355","N4336X","UAL8111","TIV241","CPT8815","DAL347","N605TF","GLF39","N793BC","N3976M","AAL698","SIL82","SWA2416","UPS711","SKW3863","JBU1759","DAL2681","SKW5005","JIA5471","SWG7590","KAL038","N535JF","UAL1520","N5322M","UAL2150","NKS1776","EDV5432","AAL1527","VTE3385","LXJ384","SCX8187","FFT3262","FFT1416","AAL942","VOI1851","AAL1440","UCA4350","DAL760","AAL3102","UAL2038","FFT3735","SWA4317","UAL2169","SWG305","N843SA","N44SH","N865PC","AAL395","N971MT","SKW3775","N682DR","SCX8719","UAL1933","ANA101","SWA4580","SWA2174","FDY361","RDN094","MTN8109","N410AS","SWA1256","SWA285","AAL785","DAL717","UPS1322","SWA4858","TTT601","AAL1712","JBU2423","EDV4820","GTI3715","CTF163","RPA4550","ASA912","NKS1091","SFY773","RPA5785","N503LM","UAL2434","JIA5252","SWA3696","ACA742","SWA882","JBU1593","EDV5312","ARG1305","SWA724","SWA2711","SKW6331","N861PS","N925JA","SKQ65","ASA770","EJA799","VIV633","N31609","SWA1997","N201CG","SWA3643","N1992H","UAL1347","UAL2448","SKW6312","N2076G","SWA328","SWA1921","EJM871","N75XL","JBU1638","WJA2042","UAL2436","N111VV","GJS4604","SWA235","CXK1011","REH18","DAL1395","RPA4707","UPS954","AAL644","ASA59","N437KM","DAL1496","LN775CC","RPA3703","SWA6499","WIA506","EJA625","KAL082","EDV5183","X722PL","AAL2436","JSX8501","N117CS","DAL1553","NKS534","EDV5031","SKW4750","QXE2210","SWA1137","N44290","UAL583","N265BF","ASA1125","N505BG","N242CT","N9650A","EJA859","UAL1185","CNS87","DAL1487","DAL3100","CVR613","EJM445","N111TV","DAL3147","SWA1719","PXT750","ASA723","LN277PC","HI1079","QXE2384","RVJ400","TIV740","SHH861","AAY9200","JIA5103","UAL1214","WJA2250","CSG2535","ARG1302","LN984ME","FFT1230","FDY124","SWA143","LCO1507","JIA5604","DAL189","UAL1617","N817NE","EDV5264","SWA1623","SKW6192","JAS83","N6277Q","SWA3176","N1615X","DAL499","UAL1985","GJS4384","N895MM","ENY3845","AAL1969","AAL2521","UAL2217","SWA3203","NKS632","JIA5566","GJS4479","SWA4843","AIC119","N2152K","N44SA","MTN8465","DAL1334","N300CP","SKW6514","N60395","JBU320","AAL1554","JBU2322","DAL1822","N839AL","SKW5481","N776CM","SWA3267","N2468Y","N5959A","UAL2667","LTA920","DAL1770","ROU1643","AAL3190","ATN3382","SWA1574","GLO9506","AAL1993","N522G","ASA333","AAL2193","BAW243","EJA345","NKS2","ROU1810","ENY3463","DAL1708","FDX1463","JIA5507","N900YP","VXP321","JTL4","SWA760","DAL1265","DAL551","ENY3638","UCA4325","AFR428","UAL611","LXJ516","SWA4346","AAL682","SKW5112","AMF6601","BAW28F","UAL1581","ASH6076","AAL3150","N333GM","N514PG","SKW5427","N44XX","UAL274","N502PS","KLM661","N970FE","WAF8101","PDT5934","AAL1156","CBC515","FDX1742","AAY1632","UAL532","QXE2015","SWA2496","N6805P","SWA794","SWA1155","TSU801","UAL2080","SKW6224","RPA5693","DAL1085","N556AV","DAL2757","JBU709","RPA5750","UAL1091","AIP1578","FDX9144","AAL2735","ASH6127","AAL2694","SKW6397","SWA1519","AAL1754","LPE2476","AMF1994","AAL1535","CSJ672","DAL350","NKS214","DZR359","AAL1450","SKW5557","PHM108","N538RM","DAL2531","AAL1323","VIV671","DAL1315","FFT2483","N68WS","EDV5102","NKS1772","N928CF","AAL2247","AAL2020","QXE2345","AMX692","EJA788","SKW3751","N738NS","N1MR","N451GV","FDX1800","CFS4692","EJA438","AMF1722","N550AL","DAL1363","SWA4186","UAL1177","SKW5446","DAL1833","UPS1134","BFF20","AAL1578","N911VM","EJA578","THY2AY","N226PA","N892CH","DAL3118","DAL379","AIP302","ASA308","NKS1700","CKS865","SWA2874","AAL7","N86RW","UAL537","N123CW","N4144R","EDV5334","SKW123J","WUP821","DAL2069","FDX89","JIA5539","VOI7712","N92DE","UAL217","CFABN","SWA1522","N13NL","N766PD","RPA3549","DAL613","NKS533","EDV5291","EJA858","AAL1854","SWA595","AAL1286","N149JS","N626HJ","N65ST","AAL1025","ERY917","CMP478","AAY67","AAL2076","DAL628","PDT5716","SKW3838","SKW5402","ENY4029","DAL389","RPA5686","N62FF","SKW835C","DAL2706","N928FM","AAL2748","UAE244","CFS8284","BHS204","JBU236","CMP477","AAL1768","SWA800","RPA5751","DCM6083","NKS557","SKQ69","TSC841","ENY3630","N794UT","N831LS","BVN8200","AFR002","UAL1905","NKS715","SWA2670","UAL2250","SWA585","UAL1657","EDV5160","NGF0ZH","FDX428","N414CY","AAL2599","TSC329","NDU502","N225DE","DAL609","N93FS","VJA546","N93AZ","N313RW","JBU2884","N501PR","N36RB","N54DA","UAL510","UAL2199","JBU229","EJA607","ERH883","DAL1562","N456TC","N997RV","SKW5852","JBU1528","AAL790","UAL8185","CXK269","SWA2124","XOJ201","N90AM","N135SP","N2BD","PDU56","SKW3323","RPA4735","SKW4888","N324GT","AAL2786","B080","SKW5627","N196SC","N3306Q","CJT623","N247WE","SLI2723","SWA3186","AAL2725","NKS451","SWA1324","JZA543","POE570","N627DY","N927SM","TWY85","ATN3371","DAL1158","LXJ436","N101JJ","N217NP","DAL411","DAL1908","UAL326","N95PW","UAL3927","JSX510","FFT2215","N9GF","FFT2818","SKW3785","FFT3241","N328ME","JZA692","LJY540","SKX5591","FDX1441","UAL405","AAL301","N45NS","JBU717","N444KM","N69FX","UAL2279","UPS1338","AAY1041","AAL491","EJA927","QXE2148","UAL2063","AJT8416","RPA3636","JBU1703","LXJ385","JBU1924","ENY3828","UAL2446","SWA4450","JBU1702","ASH6208","FFT2836","N603HH","EMD23","AAL1526","N985CE","CXK320","AAL2697","PFT151","N519JG","DAL2042","AAL860","SWA657","CSS101","UAL435","FFT2676","EJA737","MVP442","N277KE","PDT5988","UAL1406","JBU484","SWA757","NUS991","UAL1890","SWA1231","AMF7212","UAL1940","N255SH","JBU1796","ICE5470","N646VP","BBQ9850","JBU424","UBT77U","WUP17","UAL1405","DAL1994","CNS99","PDT5786","ENY3910","UCA4357","N72446","ASA1354","QTR8836","SKW4940","AAL2870","PLT8","FFT25","UAL670","XSM25","BMJ58","QXE2205","BVN8275","NKS1365","UAL2639","GPD388","CJT620","SWA1091","ASA306","SWA191","BOX393","DAE926","UAL1443","AAL2453","ASH6175","UAL774","VXP678","SCX341","AAL2590","FDX36","SKW5292","ENY3655","SKW5286","TWY972","SKW5741","N365KM","ENY3783","AAL824","GTI8637","AMF1562","DAL2601","GLG7397","N158TB","SWA1307","ENY3640","FFT2826","SWA672","NKS1361","AAY3048","POE202","DAL2682","N947EL","N4351K","VTM356","UPS5481","FFT1306","SKW6297","LYM1502","SWA4694","SWA3419","XAOAC","SWA33","N277VC","N500DH","VOI703","UAL1539","N500VL","QXE2055","N333MX","ALZ401","CLX44K","SKW5866","EDV5265","N984AC","EJA389","SWA4298","SKW4952","N35JN","N314MR","SWA1093","UAL2181","SWA1105","JBU74","JBU722","N142CF","PDU57","N5697E","N93EP","TWY241","JIA5545","EDV5302","SWA2764","DAL1528","FFT3800","N33676","TSC519","AVA121","N9202W","NDU89","DAL1985","UPS616","DAL1019","EJA659","P229WC","SWA126","AAL2026","N43FE","AAL1026","JBU2780","N439S","SKW5285","N500RH","DAL2666","LXJ504","N908DB","UPS5385","N888HT","N822MK","JZA901","IJA70","P4RCA","PDT5717","GLG7393","FFT1978","ENY3951","UPS684","JBU1343","FDX5046","LXJ582","KPO371","FLC72","CHP11","N432RB","FDX1467","AAL336","AAL3072","N502QS","MTN8108","N256RC","AIC106","N237MP","FDX1835","FDX1705","JBU737","AAL1811","DAL1853","N46KJ","EDV5235","POE205","EMD4","SKW5450","AIC179","FDX1387","N7243A","ENY3493","SKW3403","AAL2080","AAL2326","UAL1548","NKS1478","JBU241","SKW3681","CXK673","JIA5186","JEF99","AAL1913","SWA1270","HAL30","XEN27","ASA83","NKS181","CXK622","DAL1566","UAL626","FWK33","N711VT","NKS549","CFS8757","RPA4344","UAL2128","GJS4611","N588TN","SKW6519","VAR498","UAL1754","N1044M","WST707","AAY957","CSQ503","N852ND","SWA1795","N304ND","CFS8690","SCX343","N379DS","SKW6503","EJA767","JBU1328","PTR2123","SWA1766","DAL694","SKW4930","SWA1121","DAL2475","SWA586","LFA548","DAL1592","N989B","N364NY","JBU561","N401AA","CSS102","SWA1160","MTN8313","JBU1529","UAL445","N318GL","N510SW","DAL1121","AAY1028","XAZYZ","UAL1879","N3002U","ASA292","TPA4257","DAL1269","SKW4945","N306BH","SWG668","N85SG","UAL2184","UAL1668","SKW3996","JBU1979","KAP1221","N965AC","AAL601","UPS923","ASA318","N131BV","N159JA","SWA1731","UAL1984","N400JZ","N500FE","AAL3215","JBU2317","DAL3114","FFT1492","LN322PR","VIV632","UPS2751","AAL2355","OPN63","VOI1880","UAL2401","UCA4372","EJA699","SWA1375","AAL1058","N626QS","ACA1040","ANA109","SKW5251","DAL1948","CGFJA","N8092E","UAL216","XSM52","ATN3365","SWA2162","WSN3","UPS1314","SWA2619","AAL1211","SWA878","LXJ565","UPS9601","N439ER","RPA4405","N284HS","WDY600","EDV5016","UAL743","N604SG","N32RW","AMF2850","UAL2251","DAL2285","UAL2031","MRA643","N450KV","N605QS","ENY3468","FFT1963","SKW5982","SWA2706","MXY128","JTL224","VOI1836","UAL2670","UAL275","SWA3110","DAL1548","N51TE","AAL452","SWA2951","SWA822","AAL1087","HAL36","ASA698","DAL2911","DAL2411","SWA4830","N6020Q","DAL1200","AAL1575","SWA2515","N166BV","JBU2873","NKS1330","COL31","AAL2737","ACA681","SWA1293","SWA2532","AWI6141","AAY3009","AAL3003","AAL2088","N379SM","XFL150","N870EA","SWA1749","LTA927","NKS251","SCX535","JSX102","HAL1615","TAP225V","SKW6303","N517DW","VOI7800","ABX510","AAL83","MTN7103","SWA2259","SWA1165","UAL2630","DAL2289","ASA395","SCX3041","QXE2122","QXE2006","RPA4781","N80WM","AAL2402","ASH6306","SWA6484","CMP403","SWA1439","AWI6170","RPA5665","IRO8551","N340KS","N316RS","JBU619","SWA802","N771BV","POE511","CFS8584","N153LM","N3QA","AAL839","UAL506","NJE825L","SWA2921","AAL2054","DAL2532","N900EM","SNC380","ASH6034","SWA3148","THY4MS","AAY649","ELZ3","QXE2043","AAL2013","ENY4022","UPS64","SWA1767","SKW5782","AVA219","EJA572","DAL978","CFS8673","TSU8135","N38254","AAY351","LXJ428","N4752S","FDY487","DAL789","N300KR","JTL275","UPS1214","N284BG","EDV5352","RJA272","UAL678","UAL1977","CSB841","FFT1086","ENY3942","N3LG","RPA4801","PDT5682","UAL1031","RPA4668","SWA2118","UAL430","UPS400","ASH6213","N345MA","N4859F","VTE3094","N799FA","UCA4283","UAL1431","EJA409","FDX65","DAL764","BBQ8262","QXE2211","FFT1666","UAL1013","LXJ609","N6PS","N108CR","RPA3669","N406BL","EIN141","DAL8916","N124CC","FDX1414","N68WT","UPS9853","N7372J","N282MK","EVA008","DAL448","AAL3074","N560WE","N998G","FDX104","SWA2546","DAL1527","NKS892","AMX402","FDX1714","PTR2211","N51LH","DAL305","N53KG","EJA444","JZA948","SWA519","AAL912","AAL2202","ENY3338","LTA368","N8164L","MXY151","C6AMA","SWA1668","UAL1187","EJA130","N260NK","AAL2745","AAY2850","FFT1714","N754PS","RPA3591","N838QS","UAL879","N5760","VJA536","N92161","AVA008","OKC470","JBU409","SKW5855","SWA4810","N652QS","N432KC","EDV5454","N5324W","FDX1759","AAL2055","N107SB","RPA5634","DAL2851","RPA3610","N939ET","JRE214","N480ER","SKW3796","CVR327","AAL2814","SWA2178","N31AJ","FFT1755","N816FC","N8GQ","SWA1301","FTH831","N4108P","ENY3612","N991KP","N733TQ","QXE2263","UAL858","GTI8349","N15TF","ENY3609","N958FE","N615WB","AAL853","N302SJ","EDV4994","RPA5749","CFEXD","FDX8","FLC70","UAL310","SWA2769","N176AM","VOI1861","SKW3376","N260FL","ASA572","AAL1066","KAL031","SKW3438","UAL1891","DAL1071","FDX1552","SWA4900","N818BK","NKS1921","N1161L","N812SA","UAL1376","SKW4251","N171DC","UAL2465","AAY4924","AAL2078","FDX1408","AAY2718","AAL1851","AAL2767","AAL609","UAL2238","SWA1697","SWA1984","AAL3252","TSC600","N833JB","AAL2195","CNK5","AAL1219","DAL2353","N79PA","N285EM","AMF1996","FFT1375","ACA774","FDX1703","UPS2049","JBU1334","N5465M","FFT4863","N1453S","N6833J","DAL1248","GTI3619","N5728U","N8TZ","VTM693","JBU1830","CMP236","N204WC","DAL2668","JBU807","N2117D","DAL441","UPS973","XAMPS","AMX495","JIA5651","N57HC","PDT5991","AAL2827","JTL3","RPA4455","UCA4906","VTE3397","UCA4918","ENY983G","ENY4024","ASH6284","AAL2248","N317PC","ASA132","FFL610","LN917TL","UAL2356","FDX1493","N48W","UAL1056","SWA2002","FLE2516","UAL1661","MRA700","ASH6178","ENY3878","AAL1774","AAY2737","AEA033","DAL508","DAL1920","UAL1620","SWG6450","FDX1711","RPA4378","DAL432","N629C","WJA1119","N410QS","N165LM","FFT86","FFT1441","N1932K","ASA30","SKW3302","UAL343","LXJ100","N475LC","GGT710","ENY4038","ENY3949","ASA1049","N65V","N78D","UAL698","N216SP","UAL2003","N317JT","DAL2343","SWA273","JBU2701","WUP496","N354WA","AAY434","JBU524","UCA4913","DAL169","NKS205","LLN103","JZA7921","DAL2162","AAL870","FDY415","IRO8431","TPA4015","UAL8200","SWA1507","SWA4268","FFT1887","N1706W","AAL1130","FDX1876","ENY3827","RPA3479","N671PC","AAL3154","MXY717","UAL2162","LXJ486","VJA351","KES300A","ASA1200","N833TB","VOI1840","AAL671","ACA7253","SWA1310","JZA603","N372TW","FDX1401","JBU632","CAL5312","KLM686","FFT2416","VIV533","SWA3305","CJT2401","SWA1035","N353JA","DAL641","ROU1874","WJA1593","DAL1392","N6949E","CSPJD","RPA4789","AAL2837","GJS4508","SWA5145","HRT605","N964BL","SCX685","AAL123","PCM8694","N34UT","UAL2346","AAL554","EJA392","BTQ353","FFT3547","N80TL","GJS4421","CJT302","PTR2304","GTI3627","N5695S","SWA3422","FFT4196","QXE2430","N25UT","N903SM","SIA7431","N515RD","UAL1233","LXJ441","AAL3030","NOJ21","N8586R","N4839R","ABX3165","SCX610","AAL1099","AAY83","TAI310","AMF1546","FTD557","SKW3349","SWA843","UPS5609","N458QS","N428JE","NKS530","SWA1960","WMA413","N88UA","N938PM","CXK632","JBU369","N528JA","KII174","FTN10","UAL1580","WUP306","AJT845","LN649AE","DAL1745","N944MD","AAL1075","DAL357","HAL91","UAL1201","ANA12","N868QS","N696DG","FFT3237","AAL776","N622PF","EJA320","AFR6733","ENY3977","SKW5016","UAL32","ASA291","AAL2532","N652EP","QXE2249","N662ME","N220F","EDV5139","N840AK","SWA3017","N2837X","FFT1683","NKS584","AAL127","SWA2757","PDT5890","N322GV","UAL2296","N68AB","BHS359","UAL374","N272BG","AAL2424","ENY3521","DAL553","ACA966","NKS882","UAL142","NKS1120","N19481","EDV4930","EJA658","JBU253","FDX1042","LN828FC","ASA18","QHD681","GTI8409","N910JS","PDT5831","N560RK","RPA5708","SKW4104","SWA2503","DAL3052","AAY1031","N525PV","NOJ52","JBU2536","AAL3085","N229NG","AAL823","NDU45","N100AL","SWA2364","BOX526","FFT1713","UAL2617","JBU1710","N399TC","JIA5304","GGVII","N707DH","WMN211","EJA387","CGREY","X87MD","WIA361","TSC485","N900BC","N121QS","SWA2886","SWA859","SWA556","N6650H","SWA2728","DAL830","N480RL","EDV5187","N816CA","N932H","PDT5861","N913JR","ENY3660","NKS915","N441VH","NKS3642","N339ET","UAL571","SWA1350","AEA051","AMX008","TFF936","JBU448","N748BL","ACA103","N20KT","WMN603","N938BT","UAL2329","UAL1115","SWA1353","N850KA","WJA1236","SWA1816","N930BK","UAL2067","N588JK","OLV788","AAL2684","PTR2322","SWA1169","FFT3981","PHX274","UCA4948","ASA494","FDX1454","EJM110","JBU2169","HAL10","HAL11","FFT3484","AAL1061","N160FS","JBU2834","SKW3977","UAL2271","N352DZ","SKW3394","RAX121","SKW4084","N81LB","AAL323","ENY3557","SWA2306","AIP3597","SWA502","EDV5259","N83M","VIV187","N865NG","EDV5403","MXY129","LXJ419","JIA5187","UAL1663","UAL1515","VP2","BVN8280","AAL2825","N203BP","FDX442","SWA3187","N994FG","N751QS","AAY1541","AMX027","UPS910","AAY531","TSU1016","N59FH","ENY3352","EMO75","PDT5863","DAL1427","N3406X","FDX1107","SWA4","DHK363","N210LA","SKW3910","UPS2859","ASA630","EJA958","AAL1203","SWA1914","DAL2274","N469RB","EJA587","JBU1124","N481LP","N153LG","UCA4342","EJA791","N857GA","VJA501","XSR301","RPA4809","BWA415","DAL1034","SWA2516","NKS186","ACA627","RPA4566","N821AD","N79929","AAL444","N7JD","SWA3518","AAL579","UAL1466","AAY2622","SKW5574","FDX1468","LN154RR","RPA4677","UPS5843","UAL2491","TSC820","UAL267","DAL1603","RPA3482","TIV85","AAY94","SWA161","UPS788","UPS402","SWA1442","AAL609Q","AAY2946","DAL452","N3134","SKW5009","N396BB","ASH6059","AAL560","WJA2324","N176KH","FDX156","N2921S","UPS5063","EJM217","PDT5695","LN1932K","DAL376","N6607D","N702SR","N597VF","N830BS","UAL1605","AMX665","N702FX","CFG3910","AAL2283","DAL1270","UAL1350","P626TC","AAL2511","DAL907","FDX1464","N150HB","AAL1351","CXE303","SWA3529","DAL2177","SWA2997","AAL3232","UPS5907","EJA431","ASA76","N79SG","AAL1844","EDV5099","AMX697","UJC701","JBU2186","UAL1259","N35PE","ROU1855","JRE701","AAY728","JBU163","FFT4874","UPS750","N30MC","DAL1117","N17HZ","UAL2183","EJM34","ASH6082","LN800GM","DAL670","SCX3031","EJA648","ETH518","N82BL","DAL47","NKS1691","SWA1065","N5KP","FDX1579","AAL1042","FFT2017","SWA2243","SKW5576","NKS158","DAL1017","N732WB","UPS1081","CMP187","LXJ340","DAL1408","N3609E","DAL276","N2ML","MTN8520","N30HG","SKW6265","SKW4796","LJY47","KAP328","DAL2522","LXJ503","HYS101","SWA814","N199EE","N9284C","LN910GF","EDV5317","ENY3425","UAL2159","N777TK","SKW4308","JBU967","MTN8463","DAL2214","ASH6057","AAL2820","N25615","DAL114","N2030P","HRT676","DAL326","THY4","N3152K","SIL86","EJA318","EJA156","N176JR","AXL868","AAL2051","N731LM","N6563K","N89PX","N230EA","SWA2504","SWA3190","WUP833","VIR24M","BWA015","N154DS","UAL1999","SWA2991","RPA4752","SWA2367","GJS4459","EJA642","N8687M","FFT4406","ATN3377","UAL1409","DAL494","AIP2602","IDT83","JBU277","AAL557","RPA3586","NKS562","LXJ600","CSN437","VIR163U","N81ER","HAL1629","EJA209","EDG94","LYM943","SWA4839","N359BG","JBU1722","AAL1376","ENY4134","AAL634","ACA582","RPA5797","SWG6553","FFT1821","MXY185","SWA3396","KAL042","JIA5093","FLE2100","AAY369","N505JJ","SKW3962","EJA593","SWA309","JBU315","N4346D","CXK129","DAL1952","DAL880","SWA596","FDX1746","AAL2231","JBU2259","N3121V","ENY3628","ABX2028","AAL522","UAL1818","AAL974","N333JK","KAP6347","ASA884","RPA5732","RPA4676","N372EX","UAL2399","N240NY","SKW3312","CNS52","N3557E","MTN8503","SKW4754","BAW178","SWA2427","TNO4272","N182MA","BWA521","N30GJ","JIA5098","JIA5381","UPS1446","N972WW","JBU396","EJA436","SKW4085","PCM846","GXA6110","N5287Y","AAL1649","DAL1530","UAL1865","ENY3524","SKW5560","ENY4007","N102Q","DAL1578","WML56","ENY3860","SWA2492","SWA2927","N721EN","N2623","N2251M","N97KA","NKS8804","N940CS","VJT537","SKW5464","DAL822","TSU215","UAL1228","SKW5723","SKW3480","N46338","SKW4072","HAL63","N7216R","PIT3","QXE2150","ASA817","AMX423","N806MC","DAL719","UAL2172","AAL2583","DAL1468","KAL036","DAL387","UPS1172","N747LE","AAY1080","N131JX","DAL876","SKW6420","N73AR","SKW5486","N81TF","SWA318","FFT3511","PDT5855","RPA3417","N250EX","N604M","CXK457","SWA1653","DCM6352","SKW4736","KFB444","SWA2573","AMF1650","DAL1926","SWA2548","N483LP","N86CW","JBU2801","UAL375","UPS5557","SWA4951","N7799T","AJT854","N615MS","JBU1184","NKS457","N889CG","EJM503","DAL2379","AAL2896","JBU887","DAL758","PDT5686","NKS3350","ANA107","EJA150","N518ME","NKS1480","N62828","ENY3474","CXK187","DAL632","AAL2889","SKW5127","N4631B","UPS722","DAL323","N682SB","GTX247","EJA595","GJS4606","JBU779","UAL2370","N320JM","N116QS","PRE90","PDT5841","N620BA","NKS979","VJA534","N942NG","HAL60","AAL372","DAL724","DAL2212","TSC647","N247AJ","EJM114","ASA1016","EJA693","UAL2244","UAL1378","N333DL","AAL377","EMD29","EJA447","SWA2463","DAL1910","JIA5442","NKS2987","AAL2301","ASA286","NKS381","LJY468","N7599X","DAL781","SKW5692","NKS198","IWY233","FDX420","SKW3844","ASA767","SWA624","N675AZ","UPS1142","UAL129","DAL1385","RPA3475","LLN217","JBU192","AAL1204","ASA1428","SWA4195","AAL2091","AAL2069","KLM628","UAL1639","POE313","FDX1166","FLE1774","SKW4949","SKW4925","SWA3316","N686BE","UCA4367","AAL629","LN581HC","N347CD","FDX1417","SWA4939","JIA5261","SNC1104","ENY3635","WIA835","JBU2593","LTA386","VTE3102","AAL973","X254FD","LXJ602","UPS5171","EDV5079","N724KW","BWA484","DAL1447","GPD152B","ROU1663","DAL158","UAL1889","DAL1245","SWA1405","BVN8571","UAL306","EJA539","AWI2048","N154PA","DAL1894","N352BG","AZG2122","SCX778","ACA956","FDX1539","SKW4704","ACA1030","JBU1524","SWA650","ENY3432","JBU174","SWA1925","N432CJ","GJS4167","N158EA","UAL348","AAL1920","SNC1470","JSX490","PTR2121","N104CX","DAL3061","ENY3832","DAL4","SWA2881","ANA180","AAL8","FDX1379","JBU1677","DAL2209","N78652","JIA5110","EJA931","AAL2939","UCA4841","FFT2084","LXJ396","AMX793","N7864F","UAL1349","SKW4646","DAL837","BMA601","N23KM","JBU1737","ASA24","AAY2884","SKW3722","FLC73","SNC387","SCX102","N15M","N46VE","SKW5327","NCA5141","N239RT","SWA224","ERY80","N2353","EJA200","AAL2496","N858AF","N12ML","NKS793","PIT4","IBE0221","PDT5785","SWA4448","N142JM","DAL2525","CXK277","AAL1896","SWA1124","CMP481","SKW225X","JBU1530","N863QS","N446ER","NKS790","DAL390","DAL1488","KOW733","LTA445","N61799","SWA1062","EJM805","AAL484","AWI6121","AAL2647","EJA319","DAL904","ERU856","N5GU","N75EA","NKS1654","EJM979","UAL584","N85CC","AVA138","EJA131","FFT4291","N860MA","ETH501","EJA702","DAL2986","N5529N","MXY651","DAL1069","N826TW","VIV500","FDX1430","DAL977","FFT1953","AAL2510","ASA963","SWA354","JBU2661","SWA3791","SKW5036","UPS961","FDX1526","EDV4804","SKW3972","AIP253","ASH6062","AAL988","JBU2220","N36SR","EJA839","JBU73","WST801","AIP1574","SWA188","N601JP","N825JW","NKS153","N629MM","UPS5079","UAL636","VXP187","RPA4559","AVA029","AAL1187","ENY3346","SWA212","DAL2040","DAL336","N23DS","DAL2359","MRA630","VOS4326","ASP846","N302RV","KES307D","SWA553","TAI569","N79SC","SWA3843","AAL2704","UPS5125","N587QS","FDX1717","JBU2875","N626TR","DAL1316","SWA1357","HAL41","SCX555","AAY1995","ASH6035","NKS219","UAL1791","JBU2476","UAL617","DAL693","N347BC","FLE2451","N254EA","BHS62L","ASI94","UAL570","BWA267","SWA787","AWI655A","N142EB","AAL2181","FDX1648","JBU1386","AAL990","SWG415","SWA3223","BMJ1324","EJA540","LN174AA","AWI6177","AAL3149","SWA4226","N450MM","PCM8650","DAL3128","N500UH","AIP303","SKW6307","N126MM","VXP474","SKW5115","LN560LX","N188DF","X702SR","SWA3181","RPA5628","JBU1069","NKS564","NKS2163","RPA4360","N321JE","DAL2236","AAL2337","LXJ530","AAL1334","TZP26","RPA5615","FFT1915","NKS2969","SCX1613","CKS9709","CPT8765","UAL1148","UPS834","AAL2101","FFT1667","CMP196","VTE3020","NKS1312","JBU2929","JBU533","FDX1216","ROU1854","PDT5816","TSU803","UAL2303","NKS809","UAL2415","FDX1419","AAL3259","NKS563","UAL96","NKS146","DAL627","VXP100","UPS1126","DAL1653","KES209S","SKW5048","FDX1316","AAY1708","DAL313","EJA686","N680TS","DAL159","FFT70","NKS455","ENY4129","JBU465","UAL1049","N488LF","MXY584","UPS1392","CXK453","SKW3862","UAL2429","AAL3096","FDX1785","SKW5220","AAL2234","JSX280","EJA524","SWA2277","ASH6124","RPA3578","PFT318","HRT168","N986BL","N3345M","PDT5780","N111HZ","UAL509","N7773N","NKS528","RPA3709","ATN4325","N826CF","AMF1560","AMF1962","RPA3564","UAL1326","AAL701","N527LQ","DAL973","N101EF","AAL834","NKS3643","WJA1449","UPS5364","UAL1962","DAL845","UCA4894","UAL2047","N9103J","RPA4413","ENY3580","EJA820","X521PT","UPS5893","AAL3119","SCX657","N420PP","NKS8851","QXE2227","N370ES","CNS1989","AWI6043","GJS4469","FDX1155","DAL2916","SWA1330","N1LT","N371RM","AAL2012","DAL1748","JBU85","N975SP","N560CC","NTW929","TSU1015","TAG18","SKW3311","N25798","AAL1627","SWG6578","AAY678","AAL3028","SWA1341","ENY3721","N717JJ","N870B","SKW5215","ENY3922","BWA527","TAM8190","BWA290","AAL964","PJC80","VJA759","GJE21","N157MG","DAL497","AAL1059","FDX1268","XLJ411","FDY8051","N691MS","QXE2087","PGR1592","EDV5261","PCM8260","FFT3893","N115LF","SWR41G","N59JE","DAL429","JIA5317","RPA3458","JBU1109","JBU601","AAL81G","NKS1712","N579NC","KAP127","JBU1579","AVA021","LTA664","QTR98K","SWG5454","ABX3157","RPA5690","DAL2263","N9481T","NDU15","JBU91","TTT546","SWA1641","FFT1449","SWA169","JBU88","SKW3856","N420LG","AAL2931","SKW4738","UAL1511","AAL1505","SKW3830","AAL673P","UCA4261","FFT1215","DAL455","JBU1552","RPA3457","SWA1560","CSB441","NKS1631","JBU1821","UAL2137","AAL2216","SWA339","ASH6365","SWA4867","ENY3998","SWA1319","JBU431","UPS1096","PCM8798","SKW3813","UAL1364","AAL1983","SFY768","JIA5016","SWA197","NKS2006","RPA4679","JIA5310","GJS4179","MXY738","SWA257","N5051R","DAL1023","UPS774","MXY202","SWA1962","DAL2339","DAL327","AAL2977","AAL1362","SWA251","N710BG","UAL357","JA002G","SCX1671","N512WB","CJT622","N459DM","UAL747","EJA926","SKW4803","JBU2462","NKS8879","SCX8420","UPS9738","N90PR","ENY3703","BOX391","PDT5949","SKW5900","ETD93C","ENY3397","RPA3608","UAL2487","AAL1671","SIL96","GXA3393","DAL104","AAL1114","TSU3810","JBU9","SWA4494","ASH6164","ACA153","EJA910","N213TB","N34SW","AAL1677","AAL1597","N889MA","DAL1192","ASA1101","RPA3612","AAL2462","JIA5284","EJA521","SKW4982","SCX754","AAL1020","ASA830","SKW4095","LN271SM","N7308C","N250LP","SWA4994","AAL3093","ROU1634","EDV5011","VOI760","KAP548","N94626","EJA852","UAL1168","SWA3342","SWA599","AAL1412","N795CR","FFT1691","N835MD","NKS1771","UAL637","N56MC","DAL1898","FDX1385","PCM8656","SKW4976","N850WP","AWI6041","UPS5861","SWA1944","FDY245","AAL955","ENY3820","SWA477","GLF64","BVN8829","ASA1331","N245EA","N48RT","UPS994","MRA682","NKS355","CFS8792","ACA1331","N852PS","JBU53","UPS913","SKW5111","SKW4338","N351RB","RPA3476","SWA5150","SWA811","N1494V","DAL2688","UPS782","SKW178P","MXY209","FRG898","SWA3619","FFT1668","SWA2012","SKW3632","AAL1019","UAL2098","AAY1104","RPA3513","UAL2389","PDT5921","AAL2285","N653PA","UAL1453","EJA690","N728DW","N9464Q","ACA7252","WUP807","N850PE","N34LT","N1154Z","DAL1168","QUE241","AAL1091","EDV5006","EDV5061","N489JG","DAL1509","N4147W","AAL735","N407TR","ASA555","QXE2127","N436DG","AAL1565","LXJ406","DAL55","AAL1438","JSX281","N8922V","N91PD","EJA834","N2915Q","UAL8209","EJA963","SKW6471","ACA2173","CLX24Q","N127MJ","N27FB","SIA26","VIV518","SWA3106","AAY1601","UAL1395","FFT1758","UAL1836","MTN8106","N75132","ASH6188","NKS1375","DAL331","RPA3576","SWA698","RAX131","AAL1216","N329RT","AAY2864","NKS1360","N4678A","PDT5881","UPS9729","AAL1940","EJA750","AMF1548","SKW3380","EJA829","SKW4652","FDX134","BMA302","LNE516","FDX1459","AAL2851","KAL005","N771CA","N2370J","KPO560","AAR216","LTA923","SKW3831","VIV674","GTX248","N13405","AAL3109","AAL3139","SWA3568","JIA5128","DAL531","RPA3452","N181FC","N622WP","AAY1059","DAL422","ENY3607","SWA4417","N786NM","UPS5980","AMX420","N509RH","DAL2605","DAL1211","TAP24U","DAL3105","N800JS","AAL2407","N984SP","UAL1722","N154ER","JAL4","SKW3428","SHH905","RPA5662","AAY4902","DAL9979","N68BT","UAL1097","AAL2562","AAL1079","QXE2267","JBU2404","ASH6274","N1886G","SWA1655","JIA5314","CFS8739","SKW4791","RPA3606","SWA1317","SWA1665","SWA1809","N497EC","AAL574","N42HT","N745KD","XAVOA","BHS380","JIA5015","JBU285","EDV4857","UPS1304","N220LC","AAL1172","JBU1678","SKW5093","N10FZ","AAL835","PDT5856","SWA281","JIA5143","DAL1574","AAY1341","JIA5361","CAL5311","DAL280","UPS5333","MXY6429","UAL1283","N824CA","NKS1658","N8686Y","N14BM","ASA135","UAL2676","SIA52","HAL1650","N255BV","ENY3552","AAL63","N476JC","SWA247","IBE0379","CMD07","DAL3104","NKS1370","N18HL","N586MC","AAL2075","LJY234","UAL2490","SWA1625","DAL1767","AAL3268","UAL2662","PSBTG","N976V","UPS203","N6252E","EJA571","AAL2795","FDX1582","UPS2987","AVA042","N503SP","AAL474","AAL3","DAL879","UAL2074","N9323E","SWA305","ENY3782","LXJ583","SKW5309","LN36LJ","AAL271","FNX930","UAL2368","JRE869","CNS1111","N600CA","SKW3874","AAL2936","DAL8969","N389SR","LYM3219","SWA3220","UAL213","DAL342","ASH6220","FLE1889","N956MC","FTN7","PXG210","BT119","AAL1183","UAL1192","DAL2985","NKS913","DAL1682","JBU1707","SWA1869","N583CJ","N7PG","RRR9534","SKW5793","AAL3088","SWA453","NKS222","SKW4730","N77SF","SWG4464","CBT80","N550HJ","AAL2929","SLH385","NKS1645","N423R","MPH6142","CFEMT","N984ME","UPS1274","CKT320","N340TB","N563MA","N640AK","JRE809","CFG2228","ERY69","AAL345","LTA672","JIA5387","N9XF","UPS1334","FDX1409","SKW3755","SWA4442","GPD205B","FFT3938","AAL1390","SWA522","JTM211","UAL2431","AAL1595","JIA5097","ERH692","RAX715","SKW5876","GJS4571","N64FF","GPD101B","UAL2416","AAL2899","KAP8","UAL2679","UAL1322","AAL1031","SWA4626","N775CC","UAL2657","RPA4716","JBU2028","EJM163","SWA1197","N883GB","AAL1098","AAL628","FDX1512","AAL2925","EIN950","JIA5026","ASA198","ENY3604","JRE928","UAL1080","AVL126","LBQ792","KPO280","N800PK","AAL1242","DAL870","UAL1851","N501XP","ASA5","AAL509","AAL1690","JBU310","JBU149","DCM4224","AAL1407","LN886LF","UAL703","JSX580","MTU97","AMX405","AMX412","DAL464","DAL1163","FDX1432","JAL3","SWA276","UPS1114","EJA600","CGBOQ","AAY23","SWA4398","PWA413","MRA695","N549CF","AVA254","UAL539","N707JK","AAL1625","EDV5131","N279PH","AAL1881","VIR3N","VTE3456","UAL1196","FRG896","SKW3975","AAL2758","BHS235","EJA943","ASA1017","SWA4876","LTA941","MAL8062","AAL2099","WJA1394","GTX490","FDX1701","AAY217","SNC1301","ASA368","DAL815","N682K","DAL2325","N76UK","DAL1258","EJA463","SWA3983","PDT5806","AAL2705","SWA2636","N174JS","CFS8820","CXK498","N272LC","N680WP","NKS904","SKW103C","SWA971","DAL1291","UAL729","TAF701","N516PC","N1491T","DAL2420","N471AE","JTZ417","FDY233","DAL2081","SKW4110","DAL2121","SKW5001","QXE2023","CMP383","JBU2285","UAL711","N889FE","UPS5793","FFT4757","AAL1812","SKW4778","TSU804","SWA6148","CMP306","N417WD","SKW3625","SKW3815","N815RB","SIL53","N4110S","KII518","N606ZZ","LBQ650","LFA459","SKW6256","ACA1085","LXJ548","JBU1287","DAL197","EJA439","UPS5327","EDV5365","UAL694","KFS17","N847TX","JBU1360","UAL1986","N6348W","UAL226","FFT80","N1BB","SKW3851","JIA5562","N261TD","KAL032","N957RC","DAL893","EDV5437","N712RM","SKW5549","N5824Q","N214TF","N207CJ","SWA1570","SWA1401","N280RH","LXJ483","SKW5468","SWA2611","N437PS","PSFMV","UAL202","AAL3008","SWA821","AAL836","N265NX","DAL356","VTE3087","AAL1867","TSC300","N2CF","UAL1107","QXE2172","AAY4914","SWA2011","SKW3363","SKW3457","AAL1482","SWA664","JBU2940","CXK105","PAC991","N505CN","SWA4896","JBU604","N180LD","SWA1558","N348PC","EJA960","N504P","UAL1125","JIA5190","N758XP","CFS8783","EJA867","DAL2469","N604LB","VLJ","SKW6290","AAL3196","AAL2252","DAL1115","DAL477","LFA325","N256SG","DAL1331","N65VM","AAL386","AMF2132","DAL1089","SWA812","SWA2811","REE400","JBU1855","UPS5331","N110BM","JBU778","N454CR","UAL875","DAL2808","ENY3561","ENY4047","UAL1230","EDV5226","FTH154","UAL1690","SWA647","IBE0243","UPS5944","SWA750","VOI1871","NKS519","AMX004","FDX1223","UAL548","SKW3337","SWA2307","BL058","SWA940","VJA67","BEZ321","XSR361","DAL2755","N31770","FDY243","N270PC","N420GE","DAL2684","DAL1203","UAL766","LXJ564","CGYDM","LTA671","UAL1428","EJA434","UAL2675","FFT1376","JRE832","ASA460","SWA1721","CPT8589","RPA4375","ABX3112","JIA5027","SWA2247","AAL1400","UAL381","YEL9","DAL2958","SKW9934","DAL1040","EJA818","N400EC","AAL1743","N528PA","N21PK","N9650V","QXE2117","N680CS","DHK368","N601GS","SWA3719","ENY3401","N421VC","PTR2723","DAL3196","UAL1383","N340L","LTA905","N748MK","EJA790","N97CL","LXJ545","AAR221","N7268J","SKW5510","UAL1524","UPS1220","HRT421","UAL2059","SWA1501","SWA1838","DAL3051","RPA3559","SWA616","JBU1533","TWY989","CFS8789","N5840K","QXE2012","N9TJ","SWA236","MVGAL","N8383V","SKW5353","DAL2838","SIS522","KES200J","N18YP","JBU1137","N218TC","AAL1151","N5157Z","SD8","EJA590","DAL2897","AAL408","AAL3171","SWA1440","CFS8801","N950QS","AAL2067","N1824X","RPA5733","ROU1692","ENY583G","FRG1642","SWA3312","FDX1230","DAL980","RPA5619","KAL012","DAL92","SWA4498","N479W","SKW4176","SKW4088","N3001K","AAY1092","N754WC","AAL2158","AWI6111","N591DK","NIT297","SKW3687","BBQ8368","AAL406","KES306C","N8247D","SWA9005","JBU927","FFL885","N956RM","N62AC","N770PC","DAL2071","JBU511","N588PS","SWA3218","N96UJ","CNS308","AAL321","PIT10","UPS1124","SIA7403","FLC75","SWA2814","N745FX","BAW17R","UPS1016","SWA3013","CJT1810","N773AB","SKW4927","FFT4884","AAY668","FFT1192","EDV5450","EJA485","N420QS","N165AS","DAL2102","N8352U","SWA1057","FDX1380","SKW4146","N111RF","ENY9871","UAL2417","WAL512","UPS7860","THY41J","RPA4686","AAL2348","GAO306","HAL1","ERU878","JIA5268","JJ51","AAL2776","FFL537","N47119","AAL3202","DAL961","N38NG","N8238V","SWA1828","UCA4813","RPA3577","DAL1887","N904RM","AAL2515","N708DC","N560LX","FRG372","SWA1009","CNS412","TSC805","LN422PM","DAL1832","VJA549","SWA2665","JBU217","RPA5808","ENY3918","SWA646","EJA263","AAL3060","N630TK","N587SL","UAL700","SWA468","SWA2094","DAL539","JBU502","IBE0124","JRE752","DAL2651","DAL501","SWA2748","N363FW","AAL2494","JBU257","AVA028","N958PG","SKW5091","SKW3334","FDX1804","ASA516","UAL488","SKW4111","N380KM","N779JW","EDV5137","FFT4058","SHH781","KFS12","EJA779","JIA5595","DAL1517","UPS5775","CJT584","J14","OKC618","SWA5234","SKW3740","N345WL","AVA210","HER329","AAL1580","DAL87","UAL83","SKW4090","QXE2042","N351VA","DAL1973","CYO208","N8051V","FLE1602","N510WP","DAL371","UAL1338","RPA3589","N18GB","ATN3349","RPA3615","CTL361","AAL176","TTT687","VIV603","SWA969","JIA5358","LN170SE","LYM3712","SWA756","N2428J","UCA4949","ASH6040","N511TC","UCA4314","KII524","AAL2014","SWA4326","RPA3528","FFT4251","UAL1825","CXK568","SWA1736","UAL2464","UAL512","SWA2679","N692AS","SWA323","DAL937","SWA1139","SWA4185","SIL0129","UAL1449","N157PW","SWA2016","N664TT","SIS28","GJS4190","SWA4136","UAL2633","N173AW","AAY356","PAL105","SKW5096","N9298J","N79LG","N2417","DAL889","DAL1292","AAL1021","AIP357","SWA2110","N137JT","SKW4079","AAY267","SWA910","HAL6","FDX114","EMB47","N279PM","SWA373","LAK655","UAL2022","ENY3830","SWA8501","AAL1551","N321GK","RPA5860","GPD729B","PKW649","EMD1","SKQ25","SWA120","DAL1759","TAI581","DAL2973","ASA1170","SWA3574","SWA615","EDV5215","AMX647","DAL1160","EDV5058","AAL1736","UAL1646","SWA3212","DAL2486","DAL1929","UAL1892","FFT1823","LAE2880","EJA549","BAW3PB","JIA5107","CFS8737","UAL363","SWA854","NKS284","GTI346","SWA6428","SWG5408","TSC613","N980ML","N925JD","AAL2708","AAL3115","UAL1880","N186QS","IWY775","AAL3189","UPS970","AAL1515","N422CP","AAL64R","DAL801","DAL2018","ASA543","DAL1143","AAL2261","SNC1106","DAL854","AAY2942","SKW5062","LXJ377","JBU1554","CJT918","BAW2203","SWA1423","CTL101","N510SE","N900BB","AAL3209","IRO993","ASA804","DAL2444","WJA1385","FDX1801","JBU177","JBU913","AAR222","UAL1321","N47719","SKW5976","SHH3914","JIA5544","CNS72","N506RP","N87XC","SKW3635","UAL260","DAL84","UAL561","SWA1008","ENY3420","JZA7719","AAL906","DAL619","AAY2142","EJA582","VOI1703","UAL1252","DAL2229","N32CA","N95","N299PW","LXJ513","N565HV","NKS1764","CXK577","DAL886","ASH6058","DAL270","TAI396","SWG782","N7277G","SWA3463","DAL782","JBU1693","VOI7848","ASH6142","SWA4576","SWA6315","EJA689","DSS2","JBU1457","N536JA","N161CE","N87TM","N6928P","N255DV","UAL2084","TAM8189","JIA5576","EJA558","N464TA","N458G","ENY3834","DAL2953","ASA462","QXE2017","UPS378","SWG6405","N231TJ","N26913","N855AC","N4322X","UPS798","N876LF","BHS282","DAL3107","DAL1070","SWA1583","JBU231","LJY241","ENY3749","N509PM","JBU1668","UAL1598","SWA540","N83GG","EJM36","PDT5843","SWA1567","AAL1347","PDT5820","N338TX","N667AF","EJA853","SYB183","UAL923","UPS5445","AAL848","AAL1471","GXA6154","CTL18","N1219M","AAL2114","JIA5338","SWA2913","NKS204","DAL1889","SWA1410","N800QS","UAL3034","CJT574","VNT560","N855ED","ASA1091","DAL2335","JBU1717","N6087F","ACA598","N946TS","CGOAI","DAL2691","PDT5681","UAL742","LNE1454","DAL1063","SKW6512","N91966","REH50","CAL023","DAL2951","FDX9999","N567AF","UPS742","RPA4805","CPA840","MXY739","N56HP","DAL273","N537RB","N9730Q","UAL2071","N933CL","SWA2368","SWA3948","DAL1544","UAL2087","N858CB","SKW4273","RPA5774","UPS605","ASA1057","SWA458","NKS587","DAL2705","N7KC","NKS710","AAL417","SKW5297","AAY2078","CXK263","FFT1670","FFT686","AAY4903","N801MM","EJM129","DAL2856","DAL8937","UAL163","N5727V","N86ME","SWA493","NKS702","SIA22","FDY305","N850PK","N4596S","SWA898","N444LP","SWA1842","N24QA","SWA1198","N478DR","AAL2387","DAL2544","ACA1303","SWA3840","EDV5254","UCA4225","NKS177","UAL1117","DAL2085","SWA3787","N535ME","DAL120","JIA5214","SWA1871","AAL1935","FFT239","AAL1195","CMP246","AAL1831","AWI6127","JBU1322","TCF643","BWA791","SWA2593","AAL2770","SKW6445","N464H","ATN3447","SWA1923","N430SA","NKS273","QXE2208","N27CN","ENY3773","ACA536","SWA560","N623RM","DAL1433","DAL1634","ASP826","EDV4991","N4974G","POE507","N77ZX","EDV5045","N405CB","N71BS","SCX942","N989KM","AIP2827","FDY362","BVN8366","DAL1039","UAL1327","SWA1064","N5817M","N790DM","RPA4536","ASA405","ASA809","AAL422","DAL849","N75LV","UPS475","IFA6117","UAL2852","SWA2561","SWA200","WIA807","N17HA","NGF6990","N50YP","SWA2824","YEL6","AAL2011","SWA272","ASA347","ENY3917","N1463F","DAL3067","UPS674","AAL2370","CXK617","NKS1523","EJA559","UAL1073","EJA842","AAL381","N525LA","P950AC","FDX421","GJS4528","WJA243","N954JS","SWA152","N6511","JIA5383","JBU2223","N421SY","CPT7762","N454CW","SKW112C","N234BZ","TWY108","AAL1755","FAC1218","N22CX","KYE9521","SKW5572","N750TS","N4912S","ACA932","PCM8658","UAL654","N264MB","SWA2739","EDV4862","UAL350","UPS895","ENY3920","N423WA","N421CW","N343CD","JIA5048","SWA2147","N9985B","SWA410","JBU1718","UPS355","DAL2137","UAL1715","RPA4747","SWA1673","XFL539","UAL2386","AAL1815","UAL1967","JIA5208","N428MC","N901SE","ENY3358","JBU1695","UCA4238","ASA682","N9131X","SKW4931","DAL2836","AAY1633","SWA1361","UAL452","ENY3606","ANZ24","AAY522","UAL1656","AAL583","N247CP","N931EC","N3750F","MTN8409","UAL1139","N650WS","SWA2663","AAL1002","DAL2794","N200WL","XAJEY","RPA4347","N456CP","EJA453","AWI6083","RJE43","AAY674","UAL2193","FFT1047","DAL1775","N42JH","N106ML","N795SP","UAL2177","CGYRI","AMX680","SKW3342","SWA2204","VIV511","SWA778","DAL576","N456GG","N39639","SKW5433","AAL2730","JBU378","N527CR","UAL842","N122VR","KCE2","ASH6312","N216KC","SWG2467","ASA280","DAL1935","SWA1505","DAL1216","RPA4467","JBU2120","UPS5933","UAL1162","LN62GA","PDU58","CXK177","SD17","AAL1269","N32876","AAL2315","EJA909","VJA505","EJA418","GTI9625","WUP508","SWA2932","RVJ933","DAL83","AAY2719","DAL892","UAL2096","VJA307","JIA5575","FDX1536","EDV5018","NJE113A","CJT569","JBU70","UAL1314","NKS843","N6015F","N19MK","VOI1780","N182AM","ENY3415","FFT3572","N334CG","AAL1763","NKS794","DAL1153","ASH6073","N154WW","DAL2303","UAL1020","RAX555","SIL0113","N5422U","FDX1438","DAL394","VTE3383","N8YU","SWA2299","SWA1052","N855TJ","DAL796","QXE2380","UPS602","UAL1040","SWA1901","DAL1844","UAL2307","EDV4863","ENY3460","EJA843","N2865R","SWA1822","EJA928","ABX3129","N10ST","N12VU","N7353X","UAL2726","DAL3015","SWA1297","JIA5189","CXK671","SCX561","N460RM","CXK1028","SKW5487","N105RZ","SKW3669","LPE2478","ASH6112","WUP507","DAL1422","QXE2098","SWA1859","UCA4264","WJA2404","N716CB","SKQ45","AFR22U","FFT1473","ACA1323","GPD102B","LTA443","AAY2752","FDX1780","VJA533","UAL1522","SWA4802","CSB7521","UAL2668","FFT1015","DAL454","LXJ553","FRG377","AAL2523","N8642H","AAL996","DAL328","AAL1516","SWA256","KLM272","EDV5110","N47508","CFS8669","SKW5350","AAL1097","NKS880","CPT8785","UAL1010","N33CA","SWA739","JIA5299","DAL2901","UPS5983","ACA1048","LXJ358","SWA1321","SWA1120","N330DP","CFS4511","SWA1541","N856HW","SWG304","N858DD","AAY468","N22PC","CNS1978","N707QS","DAL2674","AAL1524","N80187","SKW3511","FDY9446","TSU8364","DAL3164","AAL2842","SKW6279","SWA1387","DAL2262","ENY3448","UAE214","QXE2371","RPA5826","JBU2103","UAL1502","JZA746","PFT103","N383TX","N42YZ","N930MD","DAL60A","JBU914","DAL146","LXJ366","UCA4889","GJS4427","ABX3127","N26793","FDX1428","BVN8278","SKW3670","PDT5763","GTI334","SWA406","LAE2502","AMX434","PIT1","N268TM","GTI3641","DAL2652","SWA3170","N572CH","MXY735","NKS787","AAR204","BHS137","JUS532","SKW3663","N743CG","UAL1193","MXY747","UAL2285","PCM8711","AAL440","N899ST","MTN8387","RPA4700","TEK681","UAL1958","ENY3674","SKW4998","AAL2616","N717HA","UAL2316","N84BD","FFT3644","SWA1717","N562JL","N242RA","SWA491","SKW6955","N907WF","AAL109","SKW4037","FFT3209","GTI1515","UAL2174","DAL2286","WJA2092","ASH6202","ASP489","NKS695","OAE251","FFT2486","MXY765","UPS928","CRE518","N878CH","N567MD","ASA190","NKS149","N53AA","SKW6401","DAL1271","EDV5164","FLE2810","EJA731","UAL1995","NKS691","N3107U","SWA1806","EDV5129","MRA637","WUP834","DAL852","EDV5484","N414NW","CKS511","SNC386","UPS858","JBU1153","RFD32","SWA718","N4349X","FDX1404","SWA1644","SKW3639","DAL2694","N400PK","JIA5603","N954JJ","N900EZ","ZVES3VL","SWA682","N712GM","AAL353","JIA5679","CFS8693","ERH145","N777QL","WJA1883","DAL2562","ACA688","UAL2104","SKW4137","ENY3866","SWA2249","DAL590","AAL1383","JIA5466","ANA114","N88WR","SKW3807","GRB952","N577AU","AFR92K","EJA906","N280GD","DAL1138","UPS702","AAL93","AAL2672","SWA1149","SKW5246","CYO600","UAL2204","EJA951","SWA2095","STB2160","N2850Y","SWA2341","AAL2442","SWA2928","DAL1830","SWA171","JIA5300","SWA1950","N149MF","CAL5346","UAL2361","N421HR","EDV5520","SWA2789","AAL2519","N809RQ","SWA401","SKW4164","SWA2935","UAL1077","UAL1251","LTA945","DAL1424","N5023J","N681WA","AAL659","UPS843","DAL1081","DAL1979","SKW5747","AAL2818","AAL881","PCM8685","EDV4963","N67799","SKW5823","UAL1982","SNC907","N366MC","UAL565","ACA589","NKS391","RPA4463","AAL747","UPS1292","GJS4415","NFG6990","SWA2680","JZA987","FDX1865","SWA1369","N26BD","AAL2565","N504PA","SWA3576","CTA420","DAL2631","DAL302","LXJ572","JBU1057","DAL967","N13MS","DAL2490","UAL530","N8181J","UAL1138","UAL2732","FFT3203","GJS4468","ASA1246","SKW6422","N788CG","RPA5669","EJA915","AAL548","N20660","DAL572","EJA257","AFR027","ACA922","SKW4261","ROU1755","ARG1304","SWG2571","N91AP","NKS100","SKW5221","AAL753","JBU394","SWA3392","JIA5656","ENY3812","DAL971","UAL1215","N737MM","N18007","N529TS","AAY2581","AAL2787","RPA4348","JBU2161","RPA3708","N35CV","N992DC","DAL1304","N760AF","JBU98","JIA5389","UAL237","N738C","SWA2458","N479JA","N432WB","VTE3712","UAL1811","UPS1234","AAL2927","N123PC","UAL1760","N91071","N1111K","EJA854","MVP347","N44UD","N968BS","SWA206","JBU2734","SWA2918","UAL3930","CVR300","VJA338","EJA482","PCM8712","FDY309","NKS716","UAL2621","AAL2357","SWA3276","VOI5515","DAL1692","SWA3135","FLE2650","LYM5107","JRE890","DAL805","N67BK","RPA4511","CTE585","AAL234","IFA6109","N3682L","AAL1593","DAL383","VTE3096","N212PF","NKS581","FFT1734","JIA5600","N101LA","UPS942","N400TC","ACA105","N650RA","EJA832","XBRSC","ASI68","UAL500","SWA1267","N734ST","N137JQ","P123","LXJ369","DAL2319","JIA5614","SWA445","N211FD","JBU1700","N9492W","SWA1333","RPA5819","DAL407","RPA4514","NKS1962","RPA3543","SWA1771","SWA3761","CXK265","SKW3673","WUP452","GTI8986","SKW4230","RPA5874","AAL607","DAL855","DAL588","ENY3751","UAL2239","N685MB","UAL710","UAL3922","AAY2526","QXE2106","N703CD","NKS1784","UAL1589","PDU60","CPA081","AAL2544","RPA3648","RPA4487","N540PL","N829JS","SKW5395","WUP962","ACA585","AMX440","EJA337","N8853K","SKW4966","PDT5688","N650GU","HAL45","SKW3926","SKW3301","DAL1494","N785MT","SWA609","VJA551","AAY464","LTA817","AAL280","SWA659","DAL1378","JBU1752","UAL683","DAL774","UPS5778","SWA2766","DAL1510","N421PP","JBU1023","AAL1343","SWA487","N471FD","NKS1369","N836FS","N484HP","N401DR","SKW5997","UAL1603","SWA110","SKW4036","JAL12","SWA1763","AMF1746","N954S","N301RM","RPA5684","SKW3699","TSC821","EDV5344","SNC1102","LXJ417","WJA1974","N6288J","IRO8604","N900WK","AAL2560","UAL2004","JBU1111","N416G","JZA820","EJM11","JIA5259","MTN8501","N781FM","SWA263","JBU640","RPA4355","LXJ526","DAL1314","SWA1660","SKW3960","AAL1817","SWA3838","SWG623","SLI2781","SKW6273","N901WF","AAL172","UAL660","NKS3998","SLI2710","N511CD","SWA1627","AAL625","FLE2945","N60266","ATN3367","N2462T","SWA2510","SKW5288","N8YZ","DAL1552","JRE770","DAL2872","N6CP","AAL1746","TFF959","BWA482","EVA006","ASA769","SWA429","FFT1743","AAL127D","N1221J","UAL623","AMX003","EJA809","N519LM","N271SB","EJM566","CSB9201","LYM5870","SWA3594","N2722W","FFT3313","HAL49","AAL2841","AAY93","BXR1948","DAL2429","CXK438","DAL502","N636QS","N632BA","EDV4802","EJM621","JTL604","N869RH","SWA1755","JBU498","UAL526","UCA4941","ASA61","N5955V","DAL896","N4802G","N812RX","JBU373","FDX1472","SWA1122","FDX1754","AAL1199","SNC2102","N213SR","AAL1268","QXE2201","AAL1188","N34BL","KAL252","UAL1957","AAL918","SWA2136","EJA366","FFL1065","N970NA","DAL992","ASH6137","DAL2649","NKS2979","AVA231","CCK60","N568JC","LN856HW","ASA1058","DAL1236","JIA5066","N824S","SCX3023","TSC650","XEN76","UAL2299","SWA3025","WST703","SWA978","CMP435","ACA760","BHS237","DAL1302","DAL1946","SWA3225","DAL2993","SWA3950","DAL2550","SWA2645","LAE1803","NKS1986","DAL2110","SKW6298","N127EJ","DAL672","OFT66","LAN602","SCX3035","ENY3924","N514CK","AVA258","N806SQ","NKS1352","SWA443","SWA4323","N537GS","DAL361","JIA5071","ASA360","LET3022","DAL365","N490LP","XAA9111","N650SK","JBU274","N241MB","DAL1317","FFT1757","N6163R","AZY421","AAL1908","UCA4909","DAL1846","SWA1614","VJA547","SWA1251","SWA1889","ASH6107","N178GW","SWA507","DHK002","AAL1308","SWA2967","UAL2209","DAL705","UAL3910","AAL1512","DAL960","SJN6","AAL1064","SWA1546","UAL1345","LXJ326","SWA751","N503BC","UAL1448","LXJ329","UAL337","UPS810","UAL596","N808QS","AAL3182","AAL2691","N8303Q","DAL844","AAY2940","EJA806","ASA604","UPS5739","N834KC","N390TP","ASH6282","WUP835","ZN9669Q","N8299C","EDV4858","N304RJ","NKS268","ENY3373","ACA647","AAL1936","RPA5699","FDX1574","UAL1290","AAL2437","N1205M","UAL605","N93SD","SNC2201","FDX204","DAL1336","SWA1474","N236FA","EJA794","ASA1123","DAL1966","UPS919","UAL1224","RPA5735","AAL1424","FTN8","AAL468","ENY3833","ENY4261","ACA1102","DAL2333","N930RD","SWA3210","AAL2372","DAL1388","N281HL","BVN8576","UPS753","EJA611","JBU991","N351CT","AAL1217","N483MP","N83CW","CPT7780","AAL2143","DAL483","ENY4063","AAL2309","UAL717","VXP231","SWA472","RPA5711","AAL505","N978AC","PTR2323","AAY2802","AZY501","UCA4244","DAL2365","N620DF","N501FF","NJE878K","UPS683","VOI1760","JRE851","BBQ2211","N1LF","UCA4878","ASP864","N83PW","N310MD","EJM9","N700DH","DAL1965","NKS1385","ASH6063","N81TL","JBU2691","N318PP","HRT223","JBU544","HER855","AAL2127","CXK407","SWA960","UAL1712","CBC863","UCA1505","FDX169","DAL1420","SWA1972","JIA5056","UAL2376","AAL1506","GGW22","SKW5317","ACA7020","ACA540","AMF1942","JBU2130","SWA2969","ACA687","NIT279","UAL360","DAL1950","FFT3274","ACA1281","N525AQ","AFR498","DAL915","DAL566","N432CS","UCA4332","AAL461","N191N","AIC102","ENY3702","N55NC","SCX382","TAI368","SWA1642","EJA901","SKW4682","SKW4138","SWA1193","UAL2227","EDV5383","IRO994","CSB194","UAL451","N8386B","N860CB","AAL852","N855BB","GLT157","N619PA","GTI8708","N5066L","EJA417","ASA613","N550EW","AAL2490","JIA5573","SKW5851","N3819Q","UAL2185","DAL1238","AAL2911","LXJ586","TCN100","N23HD","VXP228","DHK053","RPA4403","N515WC","N913KN","DAL1218","DAL511","AAL2425","AAL2023","AAL124","SWA2741","UPS5335","AAL1684","HAL1611","N907RP","SIL0132","WJA228","EJA631","LYM3581","N1419F","N326EW","N613HK","N769","SWA1805","PVL2276","SWA2358","JSX750","UAL598","DAL2228","DWI400","SWA2530","ENY3649","ASA433","N843CP","SWA594","N585SA","GJS4504","UCA4257","AMF1988","SWA1183","JSX620","UAL1628","N199MD","RPA5673","RPA3512","STY618","DAL1777","JIA5319","N92BS","JIA5200","AAL2613","CFXAZ","ASA383","DAL2097","WSN5","N2010F","UPS7929","N873AJ","SD16","AAL1126","BXR494","UAL152","N826R","TPA4049","RPA4647","N21EU","AAL2432","AAL3057","AAY1090","N429MS","DAL2950","N663JN","N19DD","N880MB","UPS216","SKW4706","N601HW","EJA302","JIA5161","WJA2480","VJA314","DAL2184","PCM8655","FFT1105","SWA1768","LKF98","SKW4076","N8423H","UAL738","DAL1446","RPA4610","LXJ511","TIV70","ROU1688","ASA718","RPA5594","SWA1909","JIA5359","AAL2957","CXK356","FDX148","PDT5784","N273SW","JBU746","AMX448","UAL2164","HAL71","UAL1387","SKW5681","PCM8679","JIA5114","NKS1687","GXA169","N36440","UAL2140","SWA504","N62TJ","JBU40","N226SW","TKR823","DAL2864","CSN660","UAL344","N704RB","RPA4670","N827LP","DAL3198","SWA1525","SWA4192","FFT4968","SNC1401","DAL1998","DAL2960","N600RG","SWA163","UAL1173","LYM978","EJA874","SWA3116","CSB405","N526WW","HAL95","N657T","N49RH","FFT2026","VIV532","NKS301","JZA941","AAL3063","AJT8440","JBU2795","N712HA","CFLMW","DAL1968","UAL380","SWA1428","N54781","GXA6165","DAL2288","N525SK","UAL2131","HAL1635","HAL69","N3844G","AAL2208","UAL1094","UAL1832","MVK49","UAL1084","UAL620","DAL1930","AAY3195","N16YW","UPS927","SKW3881","N48AP","RPA4725","TSC761","ACA858","AAL389","FDX152","N127B","SWA2262","ACA197","DAL695","SWA699","AAL2981","DAL9950","UAL1926","ACA990","POE535","UAL2317","CMP133","RPA4428","GJS4492","ASA118","N950RF","EJA876","KES304G","KNT67","N406MB","SWA3535","N230TG","N55G","AAL646","AMX070","LXJ372","SKW5276","RGY710","SWA3173","SKQ53","N841N","NKS441","FDX5124","EJA656","N267JK","SWA4086","EJA922","EJA567","JIA9964","N278SC","WJA1564","SWA4452","SKW5596","UAL1701","FFT2668","JBU2709","ENY3349","N1907M","N749AM","JIA5363","SKW4680","SWA4386","DAL2642","NKS723","N845MA","LN642PC","DAL862","AAL3104","AAL3195","SVA021","N1926A","GTI7806","LXJ356","ROU1791","N1539E","IWY132","NKS385","AAL103","N4957A","N240DJ","BWA480","N224CJ","AAL2653","SWA2892","ASA492","SWA3374","N3644B","SKW6404","CGGLR","DAL2907","DAL2497","POE652","DAL1221","GXA3911","EJA505","N823V","FDX1779","RPA5694","GHA97","LN98KS","N86FA","N282N","RPA4371","TAI591","EJA330","N939WB","SNC1612","SKW5701","SKW3512","SWA506","AAL1189","N3CR","AIP1588","VOI1711","DAL1757","GXA540","WST110","VJA300","SKW6490","CNS120","CND599","JBU1321","N63NG","GTI364B","PDT5826","DAL1272","BOV762","SKW3372","AMX648","EDV4945","UAL1339","EJA225","AAY2932","LYM1725","N81WF","N910DF","SWA2946","JIA5433","N232SG","SWA1192","DAL3019","N170SE","ATN3341","ACA552","RPA4702","CXK449","AAL3017","N984CE","SWA1279","N458MC","DAL2027","N739RL","ENY3485","EDV5326","WST516","AAL1379","AAL2377","VOI1790","AAL1977","SWA1711","AMX641","JBU1534","AAL2461","JBU151","N709MM","PDT5842","UAL61","ASA1245","LXJ338","WST111","FFT1782","UAL1438","CGSHI","AAL769","SIL67","PDT5992","SWG4575","AMX058","UAL1072","FDX1478","N229ME","JCY626","J10","ENY3810","HAL67","DAL2998","NKS507","KAL024","PDT5941","UAL1679","SKW5315","NJE115C","SWA1230","MXY816","N277WW","SWA1937","SCX570","PKW833","DAL431","N662JN","N7818","SWA2135","FLE2854","SNC381","UAL1489","SWA1316","N3214C","N1271M","N265QS","UAL1362","CSQ343","UPS554","SWA1920","JBU577","LN912SV","SWA1265","UAL1676","UAL699","N723LU","N786FE","N155BB","UAL1493","DAL1841","AMX455","UAL837","ABX1920","JIA9966","SKW4891","AAL2154","AAY338","N28458","N525NV","UAL2302","SKW3718","AAL2191","UAL2626","NKS1386","JTZ516","DAL1185","N52C","LN498GF","TIV685","ASA536","DAL2014","UPS981","DAL1183","AAL2002","GRY2","BHS319","AAL619","SWA3610","AAL2220","N84HM","N955MC","QXE2116","DAL2439","PIT9","YEL90","LTG8451","SWA1469","SWA667","VET593","DAL1155"]

In [ ]:
# regex to match first 3 characters of the flight number as alphabets
import re
# flights = collection.find({'flightData.flight_number': {'$regex': '^[A-Z]{3}'}})
carrier = []
for i in flights:
    
    matched = re.match('^[A-Z]{3}',i)
    if matched:
        carrier.append(matched.group(0))

tots = {}
for i in carrier:
    tots[i] = tots.get(i,0) + 1
    

{'AAL': 1460,
 'JBU': 565,
 'ASA': 251,
 'ASH': 92,
 'LXJ': 178,
 'DAL': 1316,
 'UAL': 1350,
 'VAR': 7,
 'CKS': 14,
 'FLE': 25,
 'ENY': 307,
 'SKW': 797,
 'NKS': 392,
 'IRO': 26,
 'AAY': 208,
 'SJJ': 4,
 'PDT': 142,
 'CPA': 22,
 'JSX': 47,
 'CAL': 14,
 'QXE': 90,
 'RDN': 2,
 'DLH': 13,
 'EDV': 234,
 'SWA': 1584,
 'MXY': 69,
 'EJM': 53,
 'CXK': 109,
 'SUB': 1,
 'CMP': 43,
 'ACA': 145,
 'FDX': 309,
 'FFT': 328,
 'EJA': 342,
 'GJS': 90,
 'BXR': 8,
 'DWI': 2,
 'JTZ': 12,
 'LAN': 5,
 'AWI': 49,
 'TSC': 58,
 'AMX': 74,
 'RPA': 427,
 'PLZ': 1,
 'JZA': 59,
 'UPS': 388,
 'VJA': 49,
 'VXP': 34,
 'GPD': 26,
 'UCA': 85,
 'AZY': 5,
 'CVR': 5,
 'BYA': 3,
 'ASP': 17,
 'AAR': 12,
 'JIA': 243,
 'BWA': 17,
 'ARE': 4,
 'CGJ': 2,
 'XAV': 2,
 'GXA': 30,
 'BTQ': 7,
 'ABX': 41,
 'AJT': 14,
 'LFA': 15,
 'LRC': 4,
 'NCR': 2,
 'CJT': 27,
 'GGT': 3,
 'CPT': 21,
 'SVL': 3,
 'AMF': 60,
 'WUP': 41,
 'ETH': 8,
 'AIC': 17,
 'VIV': 41,
 'PAL': 8,
 'VOS': 5,
 'YEL': 12,
 'GTI': 55,
 'ANA': 21,
 'STB': 9,
 'EPI': 4,
 'I

In [17]:
# Sort the dictionary by values in descending order
tots = dict(sorted(tots.items(), key=lambda item: item[1], reverse=True))
tots

{'SWA': 1584,
 'AAL': 1460,
 'UAL': 1350,
 'DAL': 1316,
 'SKW': 797,
 'JBU': 565,
 'RPA': 427,
 'NKS': 392,
 'UPS': 388,
 'EJA': 342,
 'FFT': 328,
 'FDX': 309,
 'ENY': 307,
 'ASA': 251,
 'JIA': 243,
 'EDV': 234,
 'AAY': 208,
 'LXJ': 178,
 'ACA': 145,
 'PDT': 142,
 'CXK': 109,
 'ASH': 92,
 'QXE': 90,
 'GJS': 90,
 'UCA': 85,
 'WJA': 83,
 'SCX': 78,
 'AMX': 74,
 'MXY': 69,
 'VOI': 64,
 'SWG': 63,
 'AMF': 60,
 'JZA': 59,
 'TSC': 58,
 'GTI': 55,
 'ATN': 54,
 'EJM': 53,
 'AIP': 50,
 'AWI': 49,
 'VJA': 49,
 'HAL': 49,
 'JSX': 47,
 'ROU': 47,
 'LTA': 45,
 'CMP': 43,
 'ABX': 41,
 'WUP': 41,
 'VIV': 41,
 'FDY': 41,
 'MTN': 40,
 'CFS': 37,
 'SNC': 37,
 'AVA': 36,
 'VXP': 34,
 'VTE': 34,
 'KAL': 34,
 'JRE': 33,
 'BHS': 32,
 'POE': 31,
 'PCM': 31,
 'GXA': 30,
 'BVN': 29,
 'CNS': 29,
 'KAP': 29,
 'LYM': 29,
 'CJT': 27,
 'IRO': 26,
 'GPD': 26,
 'PTR': 26,
 'FLE': 25,
 'BAW': 24,
 'SIL': 23,
 'CPA': 22,
 'CPT': 21,
 'ANA': 21,
 'CSB': 21,
 'TWY': 20,
 'TAI': 20,
 'MRA': 20,
 'WST': 19,
 'AFR': 19,
 'F